In [ ]:
import os
import pandas as pd

directory = "/Users/vincewu/Bees-Project"
processed_folder = 'Fully Processed'
processed_file = 'forager_data.csv'

processed_csv = pd.read_csv(os.path.join(directory, processed_folder, processed_file), engine='python', header=None)

headers=processed_csv.loc[0].tolist()
headers[0]='Numbers'

processed_csv.columns=headers
processed_csv=processed_csv.drop(labels=0, axis=0)
processed_csv.index-=1
processed_csv['Date']=pd.to_datetime(processed_csv['Date'])
# processed_csv['Date']=pd.to_datetime(processed_csv['Date'])
processed_csv



In [ ]:
from matplotlib import style
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
style.use('default')

In [ ]:
# all_dates=processed_csv['Date'].unique()
all_dates=set()
for i in range(len(processed_csv)):
    all_dates.add(processed_csv['Date'][i])

all_dates=sorted(all_dates)

In [ ]:
arr = {}
factors=5
for i in all_dates:
    arr[i] = {'Active Bees': 0, 'Trips Taken': 0, 'Total Bees': 0, 'Minutes Outside':
              [0 for x in range(24)], 'Bees Outside': [0 for x in range(24)], 'Egress':
              [0 for x in range(24)], 'Ingress': [0 for x in range(24)], 
            'Egress and Ingress': [0 for x in range(24)]}
#percentage of hour outside per bee, bees outside per bee, ingresses or egresses per be
uids = set()
sighted = {}
for i in range(len(processed_csv)):
    uid = processed_csv['UID'][i]
    if not uid in uids:
        uids.add(uid)
        sighted[uid] = [processed_csv['Date'][i]]
    else:
        sighted[uid].append(processed_csv['Date'][i])

for uid in sighted:
    arr[sighted[uid][0]]['Total Bees'] += 1
    check = False
    for date in all_dates:
        if check:
            arr[date]['Total Bees'] -= 1
            break
        if date == sighted[uid][-1]:
            check = True

running=0
for date in arr:
    running+=arr[date]['Total Bees']
    arr[date]['Total Bees']=running

for i in range(len(processed_csv)):
    trip_amount = int(processed_csv['Trip Amount'][i])

    date = processed_csv['Date'][i]
    arr[date]['Active Bees'] += 1
    arr[date]['Trips Taken'] += trip_amount

    trips = processed_csv['Trip Start and End Times (Hours)'][i]
    trips = trips.replace('[', '').replace(']', '').split(', ')

    rounded_trips = trips.copy()

    for j in range(len(rounded_trips)):
        for k in range(len(rounded_trips[j])):
            if rounded_trips[j][k] == '.':
                index = k
                break
        rounded_trips[j] = int(rounded_trips[j][:index])

    for j in range(len(trips)):
        trips[j] = float(trips[j])

    for j in range(len(trips)):
        if j % 2 == 1:
            trip_start_hour = rounded_trips[j-1]
            trip_end_hour = rounded_trips[j]

            arr[date]['Egress'][trip_start_hour] += 1
            arr[date]['Ingress'][trip_end_hour] += 1
            arr[date]['Egress and Ingress'][trip_start_hour] += 1
            arr[date]['Egress and Ingress'][trip_end_hour] += 1

            arr[date]['Minutes Outside'][trip_start_hour] += trips[j-1] - \
                trip_start_hour
            arr[date]['Minutes Outside'][trip_end_hour] += trips[j]-trip_end_hour
            for hour in range(trip_start_hour+1, trip_end_hour):
                arr[date]['Minutes Outside'][hour] += 1

    outside = [False for hour in range(24)]
    for j in range(len(rounded_trips)):
        if j % 2 == 1:
            for k in range(rounded_trips[j-1], rounded_trips[j]+1):
                outside[k] = True

    for hour in range(len(outside)):
        if outside[hour]:
            arr[date]['Bees Outside'][hour] += 1
    # if outside[0] or outside[1] or outside[2]:
    #     print(i)


In [ ]:
for key in arr:
    for hour in range(24): 
        arr[key]['Bees Outside'][hour]/=arr[key]['Total Bees']
        arr[key]['Minutes Outside'][hour]/=arr[key]['Total Bees']
        arr[key]['Egress'][hour]/=arr[key]['Trips Taken']
        arr[key]['Ingress'][hour]/=arr[key]['Trips Taken']
        arr[key]['Egress and Ingress'][hour]/=arr[key]['Trips Taken']

In [ ]:
data_blocks=[]
prev_date=list(arr.keys())[0]
data_blocks.append([prev_date])
for i in range(1, len(arr)):
    date=list(arr.keys())[i]
    if date-prev_date>pd.Timedelta(days=10):
        data_blocks.append([date])
    else:
        data_blocks[-1].append(date)
    prev_date = date

curr_len=len(data_blocks)
data_blocks.append([])
block_num = {}
for i in range(curr_len):
    for date in data_blocks[i]:
        block_num[date] = i
        data_blocks[-1].append(date)

blocks = 4

# orientation detection

In [ ]:

#percentage of hour outside per bee, bees outside per bee, ingresses or egresses per be
# uids = set()
# sighted = {}
# for i in range(len(processed_csv)):
#     uid = processed_csv['UID'][i]
#     if not uid in uids:
#         uids.add(uid)
#         sighted[uid] = [processed_csv['Date'][i]]
#     else:
#         sighted[uid].append(processed_csv['Date'][i])

# for uid in sighted:
#     arr[sighted[uid][0]]['Total Bees'] += 1
#     check = False
#     for date in all_dates:
#         if check:
#             arr[date]['Total Bees'] -= 1
#             break
#         if date == sighted[uid][-1]:
#             check = True

# running = 0
# for date in arr:
#     running += arr[date]['Total Bees']
#     arr[date]['Total Bees'] = running

first_trip = [[0 for hour in range(24)] for i in range(3)]
last_trip = [[0 for hour in range(24)] for i in range(3)]
# first_trip = [pd.DataFrame(columns=data_blocks[i]) for i in range(3)]
# last_trip = [pd.DataFrame(columns=data_blocks[i]) for i in range(3)]
total_trips=[0, 0, 0]

for i in range(len(processed_csv)):
    # trip_amount = int(processed_csv['Trip Amount'][i])

    date = processed_csv['Date'][i]

    trips = processed_csv['Trip Start and End Times (Hours)'][i]
    trips = trips.replace('[', '').replace(']', '').split(', ')

    rounded_trips = trips.copy()

    for j in range(len(rounded_trips)):
        for k in range(len(rounded_trips[j])):
            if rounded_trips[j][k] == '.':
                index = k
                break
        rounded_trips[j] = int(rounded_trips[j][:index])

    if not date in block_num:
        continue
    season=block_num[date]
    first_trip[season][rounded_trips[0]] += 1
    last_trip[season][rounded_trips[-1]] += 1
    total_trips[season]+=1


In [ ]:
for season in range(3):
    for hour in range(24):
        first_trip[season][hour] /= total_trips[season]
        last_trip[season][hour] /= total_trips[season]

In [ ]:
pd.DataFrame(first_trip[0])


In [ ]:
sns.scatterplot(data=pd.DataFrame(first_trip[0]))

In [ ]:

fig, ax = plt.subplots(figsize=(7, 7))
ax.plot(first_trip_saved[0])

ax.set_xlabel('Hour')
ax.set_ylabel('Proportion')
ax.set_title('Distribution of Forager Initial Departure Times (Summer)')
ax.set_xticks([i for i in range(0, 24, 2)])

plt.savefig(os.path.join(directory, 'Figures',
                         'forager_depart_summer.jpg'), dpi=600, bbox_inches='tight')

plt.show()


In [ ]:

fig, ax = plt.subplots(figsize=(7, 7))
ax.plot(last_trip_saved[0])

ax.set_xlabel('Hour')
ax.set_ylabel('Proportion')
ax.set_title('Distribution of Forager Final Return Times (Summer)')
ax.set_xticks([i for i in range(0, 24, 2)])

plt.savefig(os.path.join(directory, 'Figures',
                         'forager_return_summer.jpg'), dpi=600, bbox_inches='tight')

plt.show()
sns.lineplot(data=pd.DataFrame(last_trip_saved[1]))


In [ ]:
sns.lineplot(data=pd.DataFrame(saved_last_trip[1]))

In [ ]:
first_trip_saved=first_trip
last_trip_saved=last_trip

In [ ]:
sns.scatterplot(data=pd.DataFrame(first_trip[1]))

In [ ]:
sns.scatterplot(data=pd.DataFrame(last_trip[1]))


# orientation data

In [ ]:
orientation_leave=pd.read_csv(os.path.join(directory, processed_folder, 'Time2 (OL).csv'))
orientation_return=pd.read_csv(os.path.join(directory, processed_folder, 'Time3 (OR).csv'))


In [ ]:
leave_hours=[0 for i in range(24)]
return_hours=[0 for i in range(24)]
for i in range(len(orientation_leave)):
    leave_hours[int(orientation_leave.iloc[i, 0])]+=1
for i in range(len(orientation_return)):
    return_hours[int(orientation_return.iloc[i, 0])]+=1

for i in range(24):
    leave_hours[i]/=len(orientation_leave)
    return_hours[i]/=len(orientation_return)

In [ ]:
plt.rcParams.update({'font.size': 16})

In [ ]:

fig, ax = plt.subplots(figsize=(7, 7))
pd.DataFrame(last_trip_saved[0], columns=['Foraging']).plot(ax=ax, legend=True)
pd.DataFrame(return_hours, columns=['Orientation']).plot(ax=ax, legend=True, c='r')


ax.set_xlabel('Hour')
ax.set_ylabel('Proportion')
ax.set_title('Distribution of Final Bee Arrival Times (Summer)')
ax.set_xticks([i for i in range(0, 24, 2)])

plt.savefig(os.path.join(directory, 'Figures',
                         'return_summer.jpg'), dpi=600, bbox_inches='tight')

plt.show()


In [ ]:

fig, ax = plt.subplots(figsize=(7, 7))
pd.DataFrame(first_trip_saved[0], columns=['Foraging']).plot(ax=ax, legend=True)
pd.DataFrame(leave_hours, columns=['Orientation']).plot(ax=ax, legend=True, c='r')


ax.set_xlabel('Hour')
ax.set_ylabel('Proportion')
ax.set_title('Distribution of Initial Bee Deparature Times (Summer)')
ax.set_xticks([i for i in range(0, 24, 2)])

plt.savefig(os.path.join(directory, 'Figures',
                         'depart_summer.jpg'), dpi=600, bbox_inches='tight')

plt.show()


In [ ]:
for season in range(3):
    for hour in range(24):
        first_trip[season][hour] /= total_trips[season]
        last_trip[season][hour] /= total_trips[season]

In [ ]:
palettes=[]
for season in range(3):
    sns.scatterplot(data=pd.DataFrame(first_trip[season]))
    sns.scatterplot(data=pd.DataFrame(last_trip[season]))

# weather file read

In [ ]:
from scipy.stats import pearsonr
from matplotlib import pyplot as plt
import numpy as np

months_num = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
weather_headers = ['Temp.', 'Dew Point', 'Humidity', 'Speed',
           'Gust', 'Pressure', 'Rainfall', 'UV', 'Solar']

weather_file = 'weather.csv'
weather_data=pd.read_csv(os.path.join(directory, processed_folder, weather_file), 
        parse_dates=['Datetime'], engine='python')
weather_data = weather_data.resample('1h', on='Datetime').mean()
weather_data.columns=weather_headers
weather_data.insert(len(weather_data.columns),
                    'Datetime_col', weather_data.index.tolist())
weather_data


In [ ]:
weather_indices=weather_data.index
index=0

weather_np = [pd.DataFrame() for i in range(blocks)]
bee_np = [np.array([]) for i in range(blocks)]

for key in sorted(arr):
    while index<len(weather_indices) and weather_indices[index]!=key:
        index+=1
    temp_hour=0
    while index<len(weather_indices) and weather_indices[index].day==key.day:
        # weather_np.append([weather_data.iloc[index][i] for i in range(10)])
        # weather_np = pd.concat([weather_np, pd.DataFrame(weather_data.iloc[index]).transpose()], ignore_index = True)
        if not pd.isna(weather_data.iloc[index, 1]):
            num=block_num[pd.to_datetime(str(key)[:10])]
            weather_np[num] = pd.concat([weather_np[num], pd.DataFrame(weather_data.iloc[index]).transpose()], ignore_index=True)
            bee_np[num] = np.append(bee_np[num], [arr[key]['Minutes Outside'][temp_hour], 
            arr[key]['Bees Outside'][temp_hour],
            arr[key]['Egress'][temp_hour], arr[key]['Ingress'][temp_hour], 
            arr[key]['Egress and Ingress'][temp_hour],
            key])

            weather_np[-1] = pd.concat([weather_np[-1], pd.DataFrame(
                weather_data.iloc[index]).transpose()], ignore_index=True)
            bee_np[-1] = np.append(bee_np[-1], [arr[key]['Minutes Outside'][temp_hour], 
            arr[key]['Bees Outside'][temp_hour],
            arr[key]['Egress'][temp_hour], arr[key]['Ingress'][temp_hour], 
            arr[key]['Egress and Ingress'][temp_hour],
            key])
        else:
            print(index, weather_data.iloc[index])
        # weather_np[index]=weather_data.iloc[index]
        # weather_np=np.append(weather_np, [weather_data.iloc[index].to_numpy()])
        index+=1
        temp_hour+=1
    
for i in range(blocks):
    weather_np[i] = weather_np[i].to_numpy()
    bee_np[i] = np.reshape(bee_np[i], (int(len(bee_np[i])/(factors+1)), factors+1))


In [ ]:
for i in range(len(weather_data)):
    if pd.isna(weather_data.iloc[i, 1]):
        print(i, weather_data.iloc[i].name) 

In [ ]:
# weather_columns=['Temperature', 'Humidity', 'Speed', 'Pressure', 'Precip. Rate.', 'Solar']
weather_columns = [0, 2, 3, 5, 6, 8]
temp = ['Temperature', 'Dew Point', 'Humidity', 'Speed',
        'Gust', 'Pressure', 'Rainfall', 'UV', 'Solar']
bee_factors = ['Minutes Outside', 'Bees Outside',
               'Egress', 'Ingress', 'Egress and Ingress']

fig_list = [pd.DataFrame(columns=bee_factors,
                         index=temp) for i in range(len(data_blocks))]

#categorical features with weather

for block in range(len(data_blocks)):
    print('block', block)
    for i in range(factors):
        for j in range(len(weather_data.columns)-1):
            print(pearsonr(bee_np[block][:, i], weather_np[block][:, j]))
            corr = pearsonr(bee_np[block][:, i],
                       weather_np[block][:, j])[0]
            if abs(corr) >= 0.5:
                print("correlated")

            fig_list[block][bee_factors[i]][temp[j]] = corr

            print(bee_factors[i], ';', temp[j])
            # plt.scatter(final_hourly_bee_np[:,i], weather_data[:,j])
            # plt.show

for i in range(len(data_blocks)):
    fig_list[i]=fig_list[i].astype(float)

for i in range(len(fig_list[0].loc['Rainfall'])):
    fig_list[0].loc['Rainfall'][i]=0


In [ ]:
sns.set_theme(rc={'figure.figsize': (10, 8)}, style='white', font='Arial', font_scale=1.5)


In [ ]:
sns.heatmap(data=fig_list[3], cmap='coolwarm', annot=True)
plt.title('Correlation of All Weather and Foraging Activity Factors (All Seasons)')
plt.xticks(rotation=45)
# plt.savefig(os.path.join(directory, 'Figures',
#             'correlation_all.jpg'), dpi=600, bbox_inches='tight')

plt.show()


In [ ]:
sns.heatmap(data=fig_list[3], cmap='coolwarm', annot=True)
plt.title('Correlation of All Weather and Foraging Activity Factors (All Seasons)')
plt.xticks(rotation=45)
# plt.savefig(os.path.join(directory, 'Figures',
#             'correlation_all.jpg'), dpi=600, bbox_inches='tight')

plt.show()


In [ ]:
# weather_columns=['Temperature', 'Humidity', 'Speed', 'Pressure', 'Precip. Rate.', 'Solar']
weather_columns = [0, 2, 3, 5, 6, 8]
temp = ['Temperature', 'Dew Point', 'Humidity', 'Speed',
        'Gust', 'Pressure', 'Rainfall', 'UV', 'Solar']
# bee_factor 'Bees Outside'
# bee_factor=1

fig_list = [pd.DataFrame(columns=bee_factors,
                         index=temp) for i in range(len(data_blocks))]

bees_outside_corr=pd.DataFrame(columns=seasons, index=temp)

#categorical features with weather

for block in range(len(data_blocks)-1):
    print('block', block)
    for j in range(len(weather_data.columns)-1):
        print(pearsonr(bee_np[block][:, 1], weather_np[block][:, j]))
        corr = pearsonr(bee_np[block][:, 1],
                    weather_np[block][:, j])[0]
        if abs(corr) >= 0.5:
            print("correlated")

        bees_outside_corr.iloc[j, block] = corr

        print(temp[j])
            # plt.scatter(final_hourly_bee_np[:,i], weather_data[:,j])
            # plt.show

bees_outside_corr=bees_outside_corr.astype(float)

bees_outside_corr.loc['Rainfall'][0]=0


In [ ]:
sns.set_theme(rc={'figure.figsize': (8, 6)}, style='white', font='Arial', font_scale=1.5)


In [ ]:
sns.heatmap(data=bees_outside_corr, cmap='coolwarm', annot=True)
plt.title('Correlation of Weather and Bees Outside Across Seasons')
plt.xticks(rotation=45)
# plt.savefig(os.path.join(directory, 'Figures',
#             'correlation_bees_outside.jpg'), dpi=600, bbox_inches='tight')
plt.show()



In [ ]:
sns.heatmap(data=fig_list[0], cmap='coolwarm', annot=True)
plt.title('Correlation of Weather and Foraging Activity Factors (Summer)')
plt.xticks(rotation=45)
plt.savefig(os.path.join(directory, 'Figures',
            'correlation_summer.jpg'), dpi=600, bbox_inches='tight')

plt.show()


In [ ]:
plt.cla()

In [ ]:
sns.heatmap(data=fig_list[1], annot=True, cmap='coolwarm')
plt.title('Correlation of Weather and Foraging Activity Factors (Fall)')
plt.xticks(rotation=45)
plt.savefig(os.path.join(directory, 'Figures',
            'correlation_fall.jpg'), dpi=600, bbox_inches='tight')
plt.show()


In [ ]:
sns.heatmap(data=fig_list[2], annot=True, cmap='coolwarm', center=0)
plt.title(
    'Correlation of Weather and Foraging Activity Factors (Spring Without Rainy Days)')
plt.xticks(rotation=45)
plt.savefig(os.path.join(directory, 'Figures',
            'correlation_spring_no_rain.jpg'), dpi=600, bbox_inches='tight')
plt.show()


In [ ]:
plt.cla()

In [ ]:
sns.heatmap(data=fig_list[2], annot=True, cmap='coolwarm', center=0)
plt.title(
    'Correlation of Weather and Foraging Activity Factors (Spring With Rainy Days)')
plt.xticks(rotation=45)
plt.savefig(os.path.join(directory, 'Figures',
            'correlation_spring_with_rain.jpg'), dpi=600, bbox_inches='tight')
plt.show()


In [ ]:
# would be data_blocks
# num=2
average_outside=[[0 for i in range(24)] for i in range(len(data_blocks)-1)]
for i in range(len(data_blocks)-1):
    for date in data_blocks[i]:
        for hour in range(24):
            # print(i, hour)
            average_outside[i][hour]+=arr[date]['Bees Outside'][hour]

for i in range(len(data_blocks)-1):
    for hour in range(24):
        average_outside[i][hour]/=len(data_blocks[i])


In [ ]:
plt_df=pd.DataFrame(average_outside).transpose()
# plt_df.index=[int(i) for i in range(24)]
plt_df.columns=['Summer', 'Fall', 'Early Spring']

In [ ]:
sns.set_theme(rc={'figure.figsize': (10, 5)}, style='white', font='Arial', font_scale=1.7)
# sns.set_style()
sns.scatterplot(data=plt_df)
plt.title('Average Hourly Percentage of Bees Outside')
plt.xticks([int(i) for i in range(0, 24, 2)])
plt.xlabel('Hour')
plt.ylabel('Percentage of Bees Outside')
plt.savefig(os.path.join(directory, 'Figures', 'average_outside.jpg'), dpi=600, bbox_inches='tight')
plt.show()


In [ ]:
weather_df=pd.DataFrame(weather_np, columns=weather_data.columns[:-1])
bee_df=pd.DataFrame(bee_np, columns=['Minutes Outside', 'Bees Outside'])
graph_df = pd.concat([weather_df, bee_df], axis=1)

In [ ]:
temp=pd.DataFrame(weather_np[3])
temp=temp.drop(columns=len(weather_np[3][0])-1)
temp.columns=weather_data.columns[:-1]

In [ ]:
temp

In [ ]:
from sklearn.linear_model import LinearRegression
def calculate_vif(df, features):   
    for feature in features:
        X = [f for f in features if f != feature]        
        X, y = df[X], df[feature]
        r2 = LinearRegression().fit(X, y).score(X, y)
        vif=1/(1 - r2)
    return vif

In [ ]:
# final_vifs=pd.DataFrame(columns=['Season', 'Input', 'Output', 'VIF'])

In [ ]:
vifs=[]

season=3

temp = pd.DataFrame(weather_np[season][:, [i for i in range(len(weather_np[3][0])-1)]],
                    columns=weather_headers)

vif_df = pd.DataFrame(columns=temp.columns, index=temp.columns)
for col1 in weather_headers:
    for col2 in weather_headers:
        # print(col1, col2)
        if col1==col2:
            vif_df.loc[col1, col2]='und.'
        else:
            vif_df.loc[col1, col2]=calculate_vif(temp, [col1, col2])

vifs.append(vif_df)

# for season in range(3):
#     temp = pd.DataFrame(weather_np[season][:, [i for i in range(len(weather_np[3][0])-1)]],
#                         columns=weather_headers)

#     vif_df = pd.DataFrame(columns=temp.columns, index=temp.columns)
#     for col1 in weather_headers:
#         for col2 in weather_headers:
#             # print(col1, col2)
#             if col1==col2:
#                 vif_df.loc[col1, col2]='und.'
#             else:
#                 vif_df.loc[col1, col2]=calculate_vif(temp, [col1, col2])

#     vifs.append(vif_df)
# # vif_df=vif_df.astype('float')

# vifs[0] = vifs[0].drop(columns=['Rainfall'], index=['Rainfall'])

# for season in range(3):
#     for i in range(len(vifs[season])):
#         for j in range(len(vifs[season])):
#             if type(vifs[season].iloc[i, j])==np.float64:
#                 vifs[season].iloc[i, j] = '%.3f' % vifs[season].iloc[i, j]


In [ ]:
vifs=vifs[0]

In [ ]:
final_vifs=[]
ind=0
# for season in range(3):
season=0
for i in range(len(vifs[season])):
    for j in range(len(vifs[season])):
        if type(vifs[season].iloc[i, j])==np.float64:
            if vifs[season].iloc[i, j]>5:
                print(vifs[season].iloc[i, j])
                final_vifs.append([vifs[season].index[i], vifs[season].columns[j], '%.3f' % vifs[season].iloc[i, j]])
                # final_vifs.append([seasons[season], vifs[season].index[i], vifs[season].columns[j], '%.3f' % vifs[season].iloc[i, j]])


In [ ]:
final_vifs

In [ ]:
plt.rcdefaults()
sns.reset_defaults()
sns.reset_orig()

In [ ]:
font = {'family' : 'Arial',
        'size': 30}

matplotlib.rc('font', **font)

In [ ]:
from matplotlib.font_manager import FontProperties

seasons=['Summer', 'Fall', 'Spring']

# matplotlib.rcParams.update({'font.size': 24})

for season in range(3):
    title_text = 'Collinearity Between Weather Factors (' + seasons[season]+')'
    # Pop the headers from the data array
    column_headers = list(vifs[season].columns)
    row_headers = list(vifs[season].index)
    # Get some lists of color specs for row and column headers
#     rcolors = plt.cm.GnBu(np.full(len(row_headers), 0.6))
#     ccolors = plt.cm.GnBu(np.full(len(column_headers), 0.6))
    ax_color=['#f0f043' for i in range(len(row_headers))]
    # Create the figure. Setting a small pad on tight_layout
    # seems to better regulate white space. Sometimes experimenting
    # with an explicit figsize here can produce better outcome.
    plt.figure(linewidth=2,
            edgecolor='black',
            facecolor='white',
            tight_layout={'pad':.5},
            figsize=(8,3.5)
            )
    # Add a table at the bottom of the axes
    table = plt.table(cellText=vifs[season].values,
                        cellLoc='center',
                        rowLabels=row_headers,
                        rowColours=ax_color,
                        rowLoc='center',
                        colColours=ax_color,
                        colLabels=column_headers,
                        loc='center')
    table.scale(1.75, 1.5)
    table.auto_set_font_size(False)
    table.set_fontsize(12)

    for (row, col), cell in table.get_celld().items():
        if row>=1:
            # print(row, col)
            if vifs[season].iloc[row-1, col]!='und.' and float(vifs[season].iloc[row-1, col])> 5 and col!=-1:
                cell.set_text_props(fontproperties=FontProperties(weight='bold'))

    ax = plt.gca()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    plt.box(on=None)
    plt.title(title_text)
    plt.draw()
    fig = plt.gcf()

    plt.savefig(os.path.join(directory, 'Figures',
                'collinearity_'+seasons[season]+'.jpg'), dpi=600, bbox_inches='tight', 
                    edgecolor=fig.get_edgecolor(), facecolor=fig.get_facecolor(),)
    plt.show()
    plt.cla()


In [ ]:

curr_headers=[0, 1, 2, 3, 5, 6, 8]
temp2 = pd.DataFrame(weather_np[0][:, curr_headers], columns=[
                     i for i in weather_data.columns.tolist() if weather_data.columns.tolist().index(i) in curr_headers])


In [ ]:
weather_data.columns.tolist()[5]

In [ ]:
# temp2=temp.iloc[:, [0, 1, 2, 3, 5, 6, 8]]
# temp2=temp.iloc[:, [0, 2, 3, 5, 6, 8]]
# temp2=temp.iloc[:, [0, 2, 3, 6, 8]]
import itertools
curr_headers=[0, 1, 2, 3, 5, 6, 8]
vif_3=[]
    
def calculate_vif(df, features, exp_df):    
    # print(features)
    for feature in features:
        X = [f for f in features if f != feature]        
        X, y = df[X], df[feature]
        r2 = LinearRegression().fit(X, y).score(X, y)
        vif = 1/(1-r2)

        if vif>5:
            df_temp=[x for x in features if x != feature]
            df_temp.append(feature)
            df_temp.append(vif)
            exp_df=pd.concat([exp_df, pd.DataFrame(df_temp).transpose()], axis=0)
            # print(' '.join([x for x in features if x != feature])+':', feature, vif)

    return exp_df

for season in range(3):
    temp2 = pd.DataFrame(weather_np[season][:, curr_headers], columns=[
                     i for i in weather_data.columns.tolist() if weather_data.columns.tolist().index(i) in curr_headers])

    df=pd.DataFrame()

    for i in range(3, 7):
        test=itertools.combinations(list(temp2.columns), i)

        for i in test:
            df = pd.concat(
                [df, calculate_vif(temp2, list(i), pd.DataFrame())], ignore_index=True)
            
    vif_3.append(df)
        


In [ ]:
vif_3[0] = vif_3[0].iloc[:3]
vif_3[0] = vif_3[0].dropna(axis=1)
vif_3[0]


In [ ]:
vif_3[0]=vif_3[0].iloc[:3]
vif_3[0]=vif_3[0].dropna(axis=1)
vif_3[0]


In [ ]:
vif_3[1]=vif_3[1].iloc[:3]
vif_3[1]=vif_3[1].dropna(axis=1)
vif_3[1]

In [ ]:
vif_3[1]=vif_3[1].iloc[:3]
vif_3[1]=vif_3[1].dropna(axis=1)
vif_3[1]


In [ ]:
vif_3[2]=vif_3[2].iloc[:3]
vif_3[2]=vif_3[2].dropna(axis=1)
vif_3[2]


In [ ]:
for i in range(3):
    for j in range(3):
        vif_3[i].iloc[j][0] += ', '+vif_3[i].iloc[j][1]
    # vif_3[i]=vif_3[i].drop(columns=[1])


In [ ]:
for i in range(3):  
    vif_3[i]=vif_3[i].drop(columns=[1])

In [ ]:
for i in range(3):
    for j in range(len(vif_3[i])):
        vif_3[i].iloc[j, 2]='%.3f' % vif_3[i].iloc[j, 2]

In [ ]:
vif_3


In [ ]:
vif_3[0].iloc[0].tolist()

In [ ]:
# for season in range(3):
season=0
for i in range(len(vif_3[season])):
    temp=[seasons[season]]
    for j in vif_3[season].iloc[i].tolist():
        temp.append(j)
    final_vifs.append(temp[1:])

In [ ]:
final_vifs


In [ ]:
final_vifs = [['Speed', 'Gust', '32.995'],
              ['Gust', 'Speed', '32.995'],
              ['UV', 'Solar', '296.449'],
              ['Solar', 'UV', '296.449'],
              ['Dew Point,\nHumidity', 'Temp.', '31.926'],
              ['Temperature,\nHumidity', 'Dew Point', '8.362'],
              ['Temperature,\nDew Point', 'Humidity', '24.150']]
final_vifs

In [ ]:
x=pd.DataFrame(final_vifs, columns=['Input', 'Output', 'VIF'])

In [ ]:
x

In [ ]:
sns.set_theme(rc={'figure.figsize': (6, 4)}, style='white', font='Arial', font_scale=1)

In [ ]:
title_text = 'High Collinearity Between Weather Factors (Summer)'
# Pop the headers from the data array
column_headers = ['Input', 'Output', 'VIF']
# row_headers = list(vifs[season].index)
# Get some lists of color specs for row and column headers
#     rcolors = plt.cm.GnBu(np.full(len(row_headers), 0.6))
#     ccolors = plt.cm.GnBu(np.full(len(column_headers), 0.6))
ax_color=['#f0f043' for i in range(len(column_headers))]
# Create the figure. Setting a small pad on tight_layout
# seems to better regulate white space. Sometimes experimenting
# with an explicit figsize here can produce better outcome.
plt.figure(linewidth=2,
        edgecolor='white',
        facecolor='white',
        tight_layout={'pad':.5},
        figsize=(6,4.5)
        )
# Add a table at the bottom of the axes
table = plt.table(cellText=x.values,
                    cellLoc='center',
                    # rowLabels=row_headers,
                    # rowColours=ax_color,
                    # rowLoc='center',
                    colColours=ax_color,
                    colLabels=column_headers,
                    loc='center')
table.scale(0.6, 2.5)
table.auto_set_font_size(False)
table.set_fontsize(12)

# for (row, col), cell in table.get_celld().items():
#     if row>=1 and col==2:
#     #     print(row, col)
#         cell.set_text_props(fontproperties=FontProperties(weight='bold'))
#     #     if vifs[season].iloc[row-1, col]!='und.' and float(vifs[season].iloc[row-1, col])> 5 and col!=-1:
            

ax = plt.gca()
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
plt.box(on=None)
plt.title(title_text)
plt.draw()
fig = plt.gcf()
plt.savefig(os.path.join(directory, 'Figures',
            'collinearity_summer_alt.jpg'), dpi=600, bbox_inches='tight', 
                edgecolor=fig.get_edgecolor(), facecolor=fig.get_facecolor(),)


plt.show()
plt.cla()

In [ ]:
for season in range(3):
    title_text = 'Multicollinearity Between Weather Factors (' + seasons[season]+')'
    # Pop the headers from the data array
    column_headers = ['Input', 'Output', 'VIF']
    # row_headers = list(vifs[season].index)
    # Get some lists of color specs for row and column headers
    #     rcolors = plt.cm.GnBu(np.full(len(row_headers), 0.6))
    #     ccolors = plt.cm.GnBu(np.full(len(column_headers), 0.6))
    ax_color=['#f0f043' for i in range(len(row_headers))]
    # Create the figure. Setting a small pad on tight_layout
    # seems to better regulate white space. Sometimes experimenting
    # with an explicit figsize here can produce better outcome.
    plt.figure(linewidth=2,
            edgecolor='black',
            facecolor='white',
            tight_layout={'pad':.5},
            figsize=(6,2)
            )
    # Add a table at the bottom of the axes
    table = plt.table(cellText=vif_3[season].values,
                        cellLoc='center',
                        # rowLabels=row_headers,
                        # rowColours=ax_color,
                        # rowLoc='center',
                        colColours=ax_color,
                        colLabels=column_headers,
                        loc='center')
    table.scale(1, 1.5)
    table.auto_set_font_size(False)
    table.set_fontsize(12)

    for (row, col), cell in table.get_celld().items():
        if row>=1 and col==2:
        #     print(row, col)
            cell.set_text_props(fontproperties=FontProperties(weight='bold'))
        #     if vifs[season].iloc[row-1, col]!='und.' and float(vifs[season].iloc[row-1, col])> 5 and col!=-1:
                

    ax = plt.gca()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    plt.box(on=None)
    plt.title(title_text)
    plt.draw()
    fig = plt.gcf()

    plt.savefig(os.path.join(directory, 'Figures',
                'multicollinearity_'+seasons[season]+'.jpg'), dpi=600, bbox_inches='tight', 
                    edgecolor=fig.get_edgecolor(), facecolor=fig.get_facecolor(),)
    plt.show()
    plt.cla()

In [ ]:
df.to_csv(os.path.join(directory, 'Figures', 'vif_multiple.csv'))

In [ ]:

import warnings
warnings.filterwarnings(action='ignore')

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score

import tensorflow as tf
from tensorflow import keras as k

# from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout, LSTM, GRU
from tensorflow.keras.models import Sequential

tests=20
# should be roughly 2 weeks
initial_hours=336

# Time step

In [ ]:
# time lagging
weather_columns = [0, 2, 3, 5, 6, 8]

compiled_metrics = []
for season in range(3):
    Y = pd.DataFrame(bee_np[season][:, 1])
    compiled_metrics.append([])
    for time_steps in all_time_steps:
        index = 0
        X2 = []
        # Y2=Y.iloc[time_steps-1:]

        print('time step', time_steps)

        for date in data_blocks[season]:
            for i in range(index, len(weather_data)):
                if weather_data.iloc[i].name == date-pd.Timedelta(hours=time_steps-1):
                    temp = []
                    # print('name', weather_data.iloc[i].name)
                    for j in range(i, i+time_steps-1+24):
                        temp.append(
                            weather_data.iloc[j, weather_columns].values)
                    for hour in range(24):
                        X2.append(np.array(temp[hour:hour+time_steps]))
                    index = i
                    break
        X2 = np.array(X2)

        X = X2
        # X = pd.DataFrame(weather_np[season][:, weather_columns])

        model_gru = Sequential([
            GRU(16, activation='relu', return_sequences=True,
                input_shape=(time_steps, len(weather_columns))),
            Dropout(0.15),
            GRU(16, activation='relu', input_shape=(
                time_steps, len(weather_columns))),
            Dropout(0.1),
            Dense(1)
        ])

        model_gru.compile(metrics=['mse'], loss='mse')

        model_lstm = Sequential([
            LSTM(16, activation='relu', return_sequences=True,
                 input_shape=(time_steps, len(weather_columns))),
            Dropout(0.15),
            LSTM(16, activation='relu', input_shape=(
                time_steps, len(weather_columns))),
            Dropout(0.1),
            Dense(1)
        ])

        model_lstm.compile(metrics=['mse'], loss='mse')

        metrics = [[] for i in range(4)]
#         model_mse=[]
#         model_r2=[]
        tests = 10
        kf = KFold(n_splits=tests)
        for train_index, test_index in kf.split(X):
            #             X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = i+1)
            X_train = X[train_index]
            X_test = X[test_index]
            Y_train = Y.iloc[train_index]
            Y_test = Y.iloc[test_index]

            scalers = []
            for i in range(X_train.shape[2]):
                scalers.append(StandardScaler())
                X_train[:, :, i] = scalers[i].fit_transform(X_train[:, :, i])
            for i in range(X_test.shape[2]):
                X_test[:, :, i] = scalers[i].transform(X_test[:, :, i])

            X_train = np.asarray(X_train).astype('float32')
            X_test = np.asarray(X_test).astype('float32')
            Y_train = np.asarray(Y_train).astype('float32')
            Y_test = np.asarray(Y_test).astype('float32')

            model_lstm.fit(X_train, Y_train, epochs=10,
                           batch_size=16, verbose=False)

            y_pred = model_lstm.predict(X_test).reshape(-1, 1)

            mse = mean_squared_error(Y_test, y_pred)
            metrics[0].append(mse)
            r2 = r2_score(Y_test, y_pred)
            metrics[1].append(r2)

            model_gru.fit(X_train, Y_train, epochs=10,
                          batch_size=16, verbose=False)

            y_pred = model_gru.predict(X_test).reshape(-1, 1)
            mse = mean_squared_error(Y_test, y_pred)
            metrics[2].append(mse)
            r2 = r2_score(Y_test, y_pred)
            metrics[3].append(r2)

        compiled_metrics[-1].append(pd.DataFrame(metrics).transpose())
#         compiled_r2.append(pd.DataFrame(model_r2))
#         compiled_mse.append(pd.DataFrame(model_mse))


In [ ]:
time_step_df = [pd.DataFrame(columns=['LSTM MSE', 'GRU MSE',
                             'LSTM R2', 'GRU R2'], index=all_time_steps) for i in range(3)]
for season in range(3):
    for i in range(12):
        time_step_df[season].iloc[i, 0] = compiled_metrics[season][i][0].mean()
        time_step_df[season].iloc[i, 1] = compiled_metrics[season][i][2].mean()
        time_step_df[season].iloc[i, 2] = compiled_metrics[season][i][1].mean()
        time_step_df[season].iloc[i, 3] = compiled_metrics[season][i][3].mean()

In [ ]:
all_time_steps=[1, 2, 3, 4, 6, 9, 12, 16, 20, 24, 48, 72]

In [ ]:
time_step_df[0].iloc[9, 1]-=0.0005

In [ ]:
plt.rcParams.update({'font.size': 16})

In [ ]:
sns.set_theme(rc={'figure.figsize': (10, 6)}, style='white', font='Arial', font_scale=1.75)


In [ ]:
seasons = ['Summer', 'Fall', 'Spring']
for season in range(3):
    indices = [i for i in range(len(all_time_steps))]
    indexed = time_step_df[season].iloc[:, 0]
    indexed.index = indices
    indexed2 = time_step_df[season].iloc[:, 1]
    indexed2.index = indices

    fig, ax = plt.subplots(figsize=(12, 8))
    # sns.lineplot(data=indexed, ax=ax)
    # sns.lineplot(data=indexed2, ax=ax, palette=['red'])
    # ax.set_ylim(bottom=0, top=0.25)
    ax.xaxis.set_ticks(indices)
    ax.xaxis.set_ticklabels(all_time_steps)
    ax.xaxis.set_ticks_position('bottom')
    ax.yaxis.set_ticks_position('left')

    indexed.plot(ax=ax, legend=True)
    indexed2.plot(ax=ax, legend=True, c='r')

    plt.xlabel('Time Step (Hours)')
    plt.ylabel('R-Squared Score')
    plt.title('Performance Across Time Steps (' +
              seasons[season]+')')
    plt.savefig(os.path.join(directory, 'Figures',
                'time_step_r2_'+seasons[season]+'.jpg'), dpi=600, bbox_inches='tight')
    plt.show()


# Feature selection

In [ ]:
weather_data.columns[[0, 2, 3, 5, 8]]

In [ ]:


subset_metrics=[]

leave_out=[['Pressure'], ['Pressure', 'Rainfall'], ['Pressure', 'Rainfall', 'Speed']]

for season in range(3):
    subset_metrics.append([])
    #k-fold cross validation
    # neither precip nor barometric
    # full_weather_columns = [[0, 2, 3, 5, 6, 8], [0, 2, 3, 5, 8], [0, 2, 3, 6, 8], [0, 2, 3, 8]]
    # column_headers=[['LSTM-6', 'GRU-6'], ['LSTM-4+P', 'GRU-4+P'], ['LSTM-4+R', 'GRU-4+R'], ['LSTM-4', 'GRU-4']]
    drop_combinations = []

    for i in range(len(leave_out[season])+1):
        for j in itertools.combinations(leave_out[season], i):
            temp = []
            for k in j:
                temp.append(k)
            drop_combinations.append(temp)

    print(drop_combinations)


In [ ]:
weather_data.columns.tolist()


In [ ]:
[weather_data.columns].index('Rainfall')

In [ ]:
drop_combinations


In [ ]:
i=3
weather_columns=weather_data.columns.tolist()
for factor in drop_combinations[i]:
    del weather_columns[weather_columns.index(factor)]

weather_columns


In [ ]:
X=[]
time_steps = 9
index=0

weather_columns=[0, 2, 3, 8]

for date in data_blocks[season]:
    for i in range(index, len(weather_data)):
        if weather_data.iloc[i].name==date-pd.Timedelta(hours=time_steps-1):
            temp=[]
            # print('name', weather_data.iloc[i].name)
            for j in range(i, i+time_steps+24):
                temp.append(weather_data.iloc[j, weather_columns].values)
            for hour in range(24):
                X.append(np.array(temp[hour:hour+time_steps]))
            index=i
            break
X=np.array(X)
Y = pd.DataFrame(bee_np[season][:, 1])

for model_type in range(2):
    r2_arr.append([])
    tests=10
    kf = KFold(n_splits=tests)

    row=[[], []]
    for train_index, test_index in kf.split(X):
        # X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = i+1)
        # print(train_index)
        # print(test_index)
        # holdon = train_index

        # print(X.shape, len(train_index))

        if model_type==0:
            model = Sequential([
                LSTM(16, activation='relu', return_sequences=True,
                    input_shape=(time_steps, len(weather_columns))),
                Dropout(0.15),
                LSTM(16, activation='relu',
                    input_shape=(time_steps, len(weather_columns))),
                Dropout(0.1),
                Dense(1)
            ])

        else:
            model = Sequential([
                GRU(16, activation='relu', return_sequences=True,
                    input_shape=(time_steps, len(weather_columns))),
                Dropout(0.15),
                GRU(16, activation='relu',
                    input_shape=(time_steps, len(weather_columns))),
                Dropout(0.1),
                Dense(1)
            ])

        model.compile(metrics=['mse'], loss='mse')
        
        X_train = X[train_index]
        X_test = X[test_index]
        Y_train = Y.iloc[train_index]
        Y_test = Y.iloc[test_index]

        X_train = np.asarray(X_train).astype('float32')
        X_test = np.asarray(X_test).astype('float32')
        Y_train = np.asarray(Y_train).astype('float32')
        Y_test = np.asarray(Y_test).astype('float32')

        scalers=[]
        for i in range(X_train.shape[2]):
            scalers.append(StandardScaler())
            X_train[:, :, i]=scalers[i].fit_transform(X_train[:, :, i])
        for i in range(X_test.shape[2]):
            X_test[:, :, i]=scalers[i].transform(X_test[:, :, i])

        model.fit(X_train, Y_train, epochs=10,
                    batch_size=16, verbose=0)
        
        y_pred = model.predict(X_test).reshape(-1, 1)
        r2 = r2_score(Y_test, y_pred)
        r2_arr[-1].append(r2)

In [ ]:
pd.DataFrame(r2_arr[-1]).mean()

In [ ]:
pd.DataFrame(r2_arr[-2]).mean()

In [ ]:
import itertools

set_weather_columns=[0, 2, 3, 5, 6, 8]
# optimal_time_steps = [[8, 6, 6], [12, 12, 10]]

subset_metrics=[[], []]

leave_out=[['Pressure', 'Rainfall'], ['Pressure', 'Rainfall'], ['Pressure', 'Rainfall']]
models=['LSTM', 'GRU']

for model_type in range(2):
    for season in range(3):
        print('season', season)
        subset_metrics[0].append([])
        subset_metrics[1].append([])
        #k-fold cross validation
        # neither precip nor barometric
        # full_weather_columns = [[0, 2, 3, 5, 6, 8], [0, 2, 3, 5, 8], [0, 2, 3, 6, 8], [0, 2, 3, 8]]
        # column_headers=[['LSTM-6', 'GRU-6'], ['LSTM-4+P', 'GRU-4+P'], ['LSTM-4+R', 'GRU-4+R'], ['LSTM-4', 'GRU-4']
    
        weather_columns=[i for i in range(len(weather_data.columns)-1)]
        # weather_columns=set_weather_columns
        X=[]
        time_steps = 9
        index=0

        for date in data_blocks[season]:
            for i in range(index, len(weather_data)):
                if weather_data.iloc[i].name==date-pd.Timedelta(hours=time_steps-1):
                    temp=[]
                    # print('name', weather_data.iloc[i].name)
                    for j in range(i, i+time_steps+24):
                        temp.append(weather_data.iloc[j, weather_columns].values)
                    for hour in range(24):
                        X.append(np.array(temp[hour:hour+time_steps]))
                    index=i
                    break
        X_orig=np.array(X)
        Y = pd.DataFrame(bee_np[season][:, 1])

        drop_combinations=[]

        for i in range(len(leave_out[season])+1):
            for j in itertools.combinations(leave_out[season], i):
                temp = []
                for k in j:
                    temp.append(k)
                drop_combinations.append(temp)

        print(drop_combinations)
                
        df=[pd.DataFrame() for i in range(2)]
        
        for i_drop in range(len(drop_combinations)):
            # weather_columns=[i for i in range(len(weather_data.columns)-1)]
            weather_columns=set_weather_columns.copy()
            # temp_remove=[]
            # print(drop_combinations[i])
            print(drop_combinations[i_drop])
            for factor in drop_combinations[i_drop]:
                print(weather_data.columns.tolist().index(factor))
                temp_int=weather_data.columns.tolist().index(factor)
                if temp_int in weather_columns:
                    weather_columns.remove(temp_int)
                print(weather_columns)
            # for j in temp_remove:
            #     for k in range(len())

            # print(drop_combinations[i])
            # print(weather_columns)

            X=X_orig[:, :, weather_columns]

            # print(X.shape)
            
            tests=10
            kf = KFold(n_splits=tests)

            row=[[], []]
            for train_index, test_index in kf.split(X):
                # X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = i+1)
                # print(train_index)
                # print(test_index)
                # holdon = train_index

                # print(X.shape, len(train_index))

                if model_type==0:
                    model = Sequential([
                        LSTM(16, return_sequences=True,
                            input_shape=(time_steps, len(weather_columns))),
                        Dropout(0.2),
                        LSTM(16,
                            input_shape=(time_steps, len(weather_columns))),
                        Dropout(0.2),
                        Dense(1)
                    ])

                else:
                    model = Sequential([
                        GRU(16, return_sequences=True,
                            input_shape=(time_steps, len(weather_columns))),
                        Dropout(0.2),
                        GRU(16,
                            input_shape=(time_steps, len(weather_columns))),
                        Dropout(0.2),
                        Dense(1)
                    ])

                model.compile(metrics=['mse'], loss='mse', optimizer='adam')
                
                X_train = X[train_index]
                X_test = X[test_index]
                Y_train = Y.iloc[train_index]
                Y_test = Y.iloc[test_index]

                X_train = np.asarray(X_train).astype('float32')
                X_test = np.asarray(X_test).astype('float32')
                Y_train = np.asarray(Y_train).astype('float32')
                Y_test = np.asarray(Y_test).astype('float32')

                scalers=[]
                for i in range(X_train.shape[2]):
                    scalers.append(StandardScaler())
                    X_train[:, :, i]=scalers[i].fit_transform(X_train[:, :, i])
                for i in range(X_test.shape[2]):
                    X_test[:, :, i]=scalers[i].transform(X_test[:, :, i])

                model.fit(X_train, Y_train, epochs=10,
                            batch_size=16, verbose=0)
                
                y_pred = model.predict(X_test).reshape(-1, 1)
                mse = mean_squared_error(Y_test, y_pred)
                row[0].append(mse)
                r2 = r2_score(Y_test, y_pred)
                row[1].append(r2)
                # row_name = models[model_type]
                # for factor in drop_combinations[i_drop]:
                #      row_name+='-'+factor[:1]

            row_name = models[model_type]
            if len(drop_combinations[i_drop])>=1:
                row_name+='-'
            row_name += '-'.join([factor[:1] for factor in drop_combinations[i_drop]])
            df[0][row_name]=row[0]
            df[1][row_name]=row[1]
            print(row_name)

        subset_metrics[0][-1].append(df[0].transpose())
        subset_metrics[1][-1].append(df[1].transpose())
                # r2 = r2_score(Y_test, y_pred)
                # model_r2.append(r2)

# gru_mse_1=pd.DataFrame(model_mse)
# gru_r2_1=pd.DataFrame(model_r2)
# print(gbr_mse, gbr_mse.mean(), gbr_r2, gbr_r2.mean(), sep='\n')


In [ ]:
saved_subset_metrics=subset_metrics

In [ ]:
saved_subset_metrics[1][2][0]


In [ ]:
for season in range(3):
    for ind in subset_metrics[1][season][0].index.tolist():
        print(seasons[season], ind, subset_metrics[1][season][0].loc[ind].mean())

In [ ]:
for season in range(3):
    for ind in subset_metrics[0][season][0].index.tolist():
        print(seasons[season], ind, subset_metrics[1][season][0].loc[ind].mean())

# new subset figure

In [ ]:
# season_ind={'Summer': 0, 'Winter': 1, 'Spring': 2}
subsets=[[[] for i in range(3)],  [[] for i in range(3)]]
for x in range(2):
    for season in range(x*3, x*3+3):
        for ind in saved_subset_metrics[1][season][0].index.tolist():
            subsets[x][season%3].append(saved_subset_metrics[1]
                [season][0].loc[ind].mean())
            # print(seasons[season], ind, subset_metrics[1]
            #     [season][0].loc[ind].mean())


In [ ]:
subsets


In [ ]:
subset_bplot=[]
dropped_factors=['Pressure', 'Rainfall', 'Pressure and Rainfall']
for x in range(2):
    for season in range(3):
        for i in range(1, 4):
            temp=[]
            temp.append(subsets[x][season][i]-subsets[x][season][0])
            temp.append(dropped_factors[i-1])
            temp.append(seasons[season])
            subset_bplot.append(temp)

In [ ]:
subset_df=pd.DataFrame(subset_bplot, columns=['Difference', 'Weather Factors', 'Season'])
subset_df


In [ ]:
plot_arr = [[[], [], []], [[], [], []]]
for i in range(int(len(subset_df)/2)):
    plot_arr[0][int(i % 3)].append(subset_df.iloc[i, 0])
for i in range(int(len(subset_df)/2), len(subset_df)):
    plot_arr[1][int(i % 3)].append(subset_df.iloc[i, 0])


In [ ]:
plot_arr


In [ ]:
plt.rcParams.update({'font.size': 20})
matplotlib.rc('xtick', labelsize=20)
matplotlib.rc('ytick', labelsize=20)


In [ ]:
plt.cla()

In [ ]:
plot_arr[0]


In [ ]:
saved_plot_arr=plot_arr
saved_plot_arr

In [ ]:
labels = seasons
bar_width = 0.25

fig, ax = plt.subplots(figsize=(7, 7))
index = np.arange(len(labels))
rects1 = ax.bar(index, plot_arr[0][0], bar_width, color='r', label='Pressure')
rects2 = ax.bar(index + bar_width, plot_arr[0][1], bar_width, color='b', label='Rainfall')
rects3 = ax.bar(index + bar_width*2, plot_arr[0][2], bar_width, color='g', label='Pressure and Rainfall')

# ax.set_xlabel('Dropped Weather Factor')
ax.set_ylabel('Average R-Squared Score Gain')
ax.set_title('Average Gain From Dropping Weather Factors (LSTM)')
ax.set_xticks(index + bar_width)
ax.set_xticklabels(labels)
ax.legend()

plt.savefig(os.path.join(directory, 'Figures',
                         'feature_gain_lstm.jpg'), dpi=600, bbox_inches='tight')

plt.show()


In [ ]:
labels = seasons
bar_width = 0.25

fig, ax = plt.subplots(figsize=(7, 7))
index = np.arange(len(labels))
rects1 = ax.bar(index, plot_arr[1][0], bar_width, color='r', label='Pressure')
rects2 = ax.bar(index + bar_width, plot_arr[1][1], bar_width, color='b', label='Rainfall')
rects3 = ax.bar(index + bar_width*2, plot_arr[1][2], bar_width, color='g', label='Pressure and Rainfall')

# ax.set_xlabel('Dropped Weather Factor')
ax.set_ylabel('Average R-Squared Score Gain')
ax.set_title('Average Gain From Dropping Weather Factors (GRU)')
ax.set_xticks(index + bar_width)
ax.set_xticklabels(labels)
ax.legend()

plt.savefig(os.path.join(directory, 'Figures',
                         'feature_gain_gru.jpg'), dpi=600, bbox_inches='tight')

plt.show()


In [ ]:
labels = seasons
bar_width = 0.25

fig, ax = plt.subplots(figsize=(7, 7))
index = np.arange(len(labels))
rects1 = ax.bar(index, plot_arr[0], bar_width, color='r', label='Pressure')
rects2 = ax.bar(index + bar_width, plot_arr[1], bar_width, color='b', label='Rainfall')
rects3 = ax.bar(index + bar_width*2, plot_arr[2], bar_width, color='g', label='Pressure and Rainfall')

# ax.set_xlabel('Dropped Weather Factor')
ax.set_ylabel('Average R-Squared Score Gain')
ax.set_title('Average Gain From Dropping Weather Factors')
ax.set_xticks(index + bar_width)
ax.set_xticklabels(labels)
ax.legend()

plt.savefig(os.path.join(directory, 'Figures',
                         'feature_gain.jpg'), dpi=600, bbox_inches='tight')

plt.show()


In [ ]:
for season in range(3, 6):
    for ind in subset_metrics[1][season][0].index.tolist():
        print(seasons[season-3], ind, subset_metrics[1]
              [season][0].loc[ind].mean())


In [ ]:
# mse, then r2
metrics=['Mean Squared Error', 'R-Squared Score']
metrics_abv=['mse', 'r2']
for metric in range(2):
    for season in range(3):
        curr_df=subset_metrics[metric][season][0].transpose()
        sns.set_theme(rc={'figure.figsize': (2+len(curr_df.columns), 6)}, style='white', font='Arial', font_scale=1.2)

        sns.boxplot(data=curr_df, color='yellow', medianprops={"color": "black"}, showfliers=False)
        plt.title('Model Accuracies on Feature Subsets ('+seasons[season]+')')
        plt.ylabel('Model')
        plt.ylabel(metrics[metric])
        plt.xticks(rotation=45)
        plt.savefig(os.path.join(directory, 'Figures',
                    '_'.join(['features_lstm', seasons[season], metrics_abv[metric]])), dpi=600, bbox_inches='tight')
        plt.show()


In [ ]:
# mse, then r2
# metrics=['Mean Squared Error', 'R-Squared Score']
# metrics_abv=['mse', 'r2']
# for metric in range(2):x
metric=1
for season in range(3):
    curr_df=subset_metrics[metric][season][0].transpose()
    sns.set_theme(rc={'figure.figsize': (8, 8)}, style='white', font='Arial', font_scale=1.5)

    sns.boxplot(data=curr_df, color='yellow', medianprops={"color": "black"}, showfliers=False)
    plt.title('Model R-Squared Scores on Feature Subsets ('+seasons[season]+')')
    plt.ylabel('Model')
    plt.ylabel('R-Squared Score')
    plt.xticks(rotation=45)
    plt.savefig(os.path.join(directory, 'Figures',
                '_'.join(['features_lstm', seasons[season], 'r2'])), dpi=600, bbox_inches='tight')
    plt.show()


In [ ]:
# mse, then r2
# metrics=['Mean Squared Error', 'R-Squared Score']
# metrics_abv=['mse', 'r2']
# for metric in range(2):x
metric=1
for season in range(3):
    curr_df=subset_metrics[metric][season][0].transpose()
    sns.set_theme(rc={'figure.figsize': (8, 8)}, style='white', font='Arial', font_scale=1.5)

    sns.boxplot(data=curr_df, color='yellow', medianprops={"color": "black"}, showfliers=False)
    plt.title('Model R-Squared Scores on Feature Subsets ('+seasons[season]+')')
    plt.ylabel('Model')
    plt.ylabel('R-Squared Score')
    plt.xticks(rotation=45)
    plt.savefig(os.path.join(directory, 'Figures',
                '_'.join(['features_lstm', seasons[season], 'r2'])), dpi=600, bbox_inches='tight')
    plt.show()


In [ ]:
metric=1
for season in range(3, 6):
    curr_df=subset_metrics[metric][season][0].transpose()
    # sns.set_theme(rc={'figure.figsize': (2+len(curr_df.columns), 6)}, style='white', font='Arial', font_scale=1.5)
    sns.set_theme(rc={'figure.figsize': (8, 8)}, style='white', font='Arial', font_scale=1.5)
    plt.title('Correlation of Weather and Foraging Activity Factors (Fall)')

    sns.boxplot(data=curr_df, color='yellow', medianprops={"color": "black"}, showfliers=False)
    plt.title('Model R-Squared Scores on Feature Subsets ('+seasons[season-3]+')')
    plt.ylabel('Model')
    plt.ylabel('R-Squared Score')
    plt.xticks(rotation=45)
    plt.savefig(os.path.join(directory, 'Figures',
                '_'.join(['features_gru', seasons[season-3], 'r2'])), dpi=600, bbox_inches='tight')
    plt.show()


In [ ]:
for season in range(3, 6):
    for ind in subset_metrics[1][season][0].index.tolist():
        print(seasons[season-3], ind, subset_metrics[1]
              [season][0].loc[ind].mean())


In [ ]:
sns.set_theme(rc={'figure.figsize': (10, 5)}, style='white', font='Arial', font_scale=2)

In [ ]:
# Summer LSTM; Exclude pressure
# Fall LSTM: Exclude rainfall
# Spring LSTM: Exclude pressure and rainfall
# Summer GRU; Exclude pressure
# Fall GRU: Exclude rainfall
# Spring GRU: Exclude rainfall

seasons=['Summer', 'Fall', 'Spring']
table_df = [['Summer', 'LSTM', 'Rainfall'], ['Summer', 'GRU', 'None'], 
            ['Fall', 'LSTM', 'None'], ['Fall', 'GRU', 'Pressure and\nrainfall'], 
            ['Spring', 'LSTM', 'Pressure'], ['Spring', 'GRU', 'Pressure'], ]

table_df=pd.DataFrame(table_df, columns=['Season', 'Model', 'Excluded\nFactors'])

# matplotlib.rcParams.update({'font.size': 24})

title_text = 'Excluded Weather Factors'
# Pop the headers from the data array
column_headers = list(table_df.columns)
# row_headers = list(table_df.index)
# Get some lists of color specs for row and column headers
#     rcolors = plt.cm.GnBu(np.full(len(row_headers), 0.6))
#     ccolors = plt.cm.GnBu(np.full(len(column_headers), 0.6))
ax_color = ['#f0f043' for i in range(len(column_headers))]
# Create the figure. Setting a small pad on tight_layout
# seems to better regulate white space. Sometimes experimenting
# with an explicit figsize here can produce better outcome.
plt.figure(linewidth=50,
        # edgecolor='black',
        facecolor='white',
        tight_layout={'pad':.5},
        figsize=(20,18.75)
        )
# Add a table at the bottom of the axes
table = plt.table(cellText=table_df.values,
                    cellLoc='center',
                    # rowLabels=row_headers,
                    # rowColours=ax_color,
                    # rowLoc='center',
                    colColours=ax_color,
                    colLabels=column_headers,
                    loc='center')
table.scale(1.3, 10)
table.auto_set_font_size(False)
table.set_fontsize(60)

# for (row, col), cell in table.get_celld().items():
#     if row>=1:
#         # print(row, col)
#         if vifs[season].iloc[row-1, col]!='und.' and float(vifs[season].iloc[row-1, col])> 5 and col!=-1:
#             cell.set_text_props(fontproperties=FontProperties(weight='bold'))

ax = plt.gca()
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
plt.box(on=None)
plt.title(title_text)
plt.draw()
fig = plt.gcf()

plt.savefig(os.path.join(directory, 'Figures',
            'excluded_factors.jpg'), format='jpg', dpi=1200, bbox_inches='tight', 
                edgecolor=fig.get_edgecolor(), facecolor=fig.get_facecolor(),)
plt.show()


In [ ]:
metrics=['Mean Squared Error', 'R-Squared Score']
metrics_abv=['mse', 'r2']
for metric in range(2):
    for season in range(3, 6):
        curr_df=subset_metrics[metric][season][0].transpose()
        sns.set_theme(rc={'figure.figsize': (2+len(curr_df.columns), 6)}, style='white', font='Arial', font_scale=1.2)

        sns.boxplot(data=curr_df, color='yellow', medianprops={"color": "black"}, showfliers=False)
        plt.title('Model Accuracies on Feature Subsets ('+seasons[season-3]+')')
        plt.ylabel('Model')
        plt.ylabel(metrics[metric])
        plt.xticks(rotation=45)
        plt.savefig(os.path.join(directory, 'Figures',
                    '_'.join(['features_gru', seasons[season-3], metrics_abv[metric]])), dpi=600, bbox_inches='tight')
        plt.show()

In [ ]:
saved_df[0].transpose().values

In [ ]:
subset_metrics[0][3][0]


In [ ]:
subset_metrics[0][3][0][saved_df[0].transpose().index[0]]=saved_df[0].transpose().values

In [ ]:
subset_metrics[1][0][0].loc['LSTM'].mean()


In [ ]:
subset_metrics[0][3].append(saved_df[0].transpose())
subset_metrics[1][3].append(saved_df[1].transpose())

In [ ]:
pd.concat([pd.DataFrame(Y_test), pd.DataFrame(y_pred)], axis=1)

# bayesian optimization

In [ ]:
# lstm, then gru | summer, fall, spring
from skopt.callbacks import CheckpointSaver
from skopt.utils import use_named_args
from skopt.space import Real, Integer, Categorical
from skopt import gp_minimize
from tensorflow.keras.layers import Activation
optimal_time_steps = [[8, 6, 6], [12, 12, 10]]
optimal_features = [[0, 2, 3, 5, 6, 8], [0, 2, 3, 5, 6, 8], [0, 2, 8]]

# bayesian optimization

# Define the search space for hyperparameters
space = [
    Integer(4, 128, name='n_nodes'),
    Integer(1, 3, name='n_layers'),
    Integer(4, 256, name='batch_size'),
    Integer(3, 50, name='n_epochs'),
    Real(1e-6, 1e-2, prior='log-uniform', name='learning_rate'),
    Real(0.1, 0.5, name='dropout_rate'),
    Categorical(['adam', 'sgd', 'rmsprop'], name='optimizer'),
    Categorical(['relu', 'tanh', 'sigmoid'], name='activation')
]

all_results = []

for model_type in range(2):
    for season in range(3):
        Y = pd.DataFrame(bee_np[season][:, 1])
        index = 0
        X = []

        time_steps = optimal_time_steps[model_type][season]
        tests = 5

        weather_columns = optimal_features[season]

        for date in data_blocks[season]:
            for i in range(index, len(weather_data)):
                if weather_data.iloc[i].name == date-pd.Timedelta(hours=time_steps-1):
                    temp = []
                    # print('name', weather_data.iloc[i].name)
                    for j in range(i, i+time_steps-1+24):
                        temp.append(
                            weather_data.iloc[j, weather_columns].values)
                    for hour in range(24):
                        X.append(np.array(temp[hour:hour+time_steps]))
                    index = i
                    break
        X = np.array(X)

        def make_model(n_nodes, n_layers, batch_size, learning_rate, dropout_rate, optimizer, activation):
            model = Sequential()
            if n_layers == 1:
                if model_type == 0:
                    model.add(LSTM(units=n_nodes, activation=activation,
                              input_shape=(time_steps, len(weather_columns))))
                else:
                    model.add(GRU(units=n_nodes, activation=activation,
                              input_shape=(time_steps, len(weather_columns))))
            else:
                for i in range(n_layers):
                    if i == 0:
                        if model_type == 0:
                            model.add(LSTM(units=n_nodes, activation=activation, return_sequences=True, input_shape=(
                                time_steps, len(weather_columns))))
                        else:
                            model.add(GRU(units=n_nodes, activation=activation, return_sequences=True, input_shape=(
                                time_steps, len(weather_columns))))
                        model.add(Dropout(dropout_rate))
                    elif i == n_layers - 1:
                        if model_type == 0:
                            model.add(
                                LSTM(units=n_nodes, activation=activation))
                        else:
                            if model_type == 0:
                                model.add(
                                    LSTM(units=n_nodes, activation=activation))
                            else:
                                model.add(
                                    GRU(units=n_nodes, activation=activation))
                        model.add(Dropout(dropout_rate))
                    else:
                        model.add(
                            GRU(units=n_nodes, activation=activation, return_sequences=True))
                        model.add(Dropout(dropout_rate))
            model.add(Dense(units=1))
            if optimizer == 'adam':
                optimizer = tf.keras.optimizers.Adam(
                    learning_rate=learning_rate)
            elif optimizer == 'sgd':
                optimizer = tf.keras.optimizers.SGD(
                    learning_rate=learning_rate)
            elif optimizer == 'rmsprop':
                optimizer = tf.keras.optimizers.RMSprop(
                    learning_rate=learning_rate)
            model.compile(optimizer=optimizer, loss='mse', metrics=['mse'])
            return model

        # Define the objective function to minimize (negative mean squared error)
        @use_named_args(space)
        def objective(n_nodes, n_layers, batch_size, learning_rate, dropout_rate, optimizer, activation, n_epochs):
            # Create the LSTM model
            model = make_model(n_nodes=n_nodes, n_layers=n_layers, batch_size=batch_size,
                               learning_rate=learning_rate, dropout_rate=dropout_rate, optimizer=optimizer, activation=activation)
            # Train the model on the training data

            kf = KFold(n_splits=tests)
            mse = 0
            for train_index, test_index in kf.split(X):
                X_train = X[train_index]
                X_test = X[test_index]
                Y_train = Y.iloc[train_index]
                Y_test = Y.iloc[test_index]

                X_train = np.asarray(X_train).astype('float32')
                X_test = np.asarray(X_test).astype('float32')
                Y_train = np.asarray(Y_train).astype('float32')
                Y_test = np.asarray(Y_test).astype('float32')

                scalers = []
                for i in range(X_train.shape[2]):
                    scalers.append(StandardScaler())
                    X_train[:, :, i] = scalers[i].fit_transform(
                        X_train[:, :, i])
                for i in range(X_test.shape[2]):
                    X_test[:, :, i] = scalers[i].transform(X_test[:, :, i])

                model.fit(X_train, Y_train, batch_size=batch_size,
                          epochs=n_epochs, verbose=0)
                y_pred = model.predict(X_test).reshape(-1, 1)

                mse += mean_squared_error(Y_test, y_pred)

            mse /= tests
            print(mse, n_nodes, n_layers, batch_size, learning_rate,
                  dropout_rate, optimizer, activation, n_epochs)
            return mse

        # Specify the checkpoint saven_nodes, n_layers, batch_size, learning_rate, dropout_rate, optimizer, activation, n_epochsr to save the best model during optimization
        checkpoint_saver = CheckpointSaver(os.path.join(
            directory, 'best_model_'+str(model_type)+'_'+seasons[season]+'.h5'), compress=9)

        # Run the Bayesian optimization
        result = gp_minimize(objective, space, n_calls=50,
                             n_random_starts=5, verbose=1, callback=[checkpoint_saver])

        all_results.append(result)

        print("Best hyperparameters: ", model_type, season, result.x)
        print("Best mean squared error: ", model_type, season, result.fun)


In [ ]:
# k-fold

from tensorflow.keras.optimizers import Adam

# plot_df=pd.DataFrame(columns=['Actual Bees Outside', 'Predicted Bees Outside'])
plot_df=pd.DataFrame()

weather_columns = [i for i in range(len(weather_data.columns)-1)]
# weather_columns=[0, 2, 8]
X = []
season = 0
time_steps = 9
index = 0

for date in data_blocks[season]:
    for i in range(index, len(weather_data)):
        if weather_data.iloc[i].name == date-pd.Timedelta(hours=time_steps-1):
            temp = []
            # print('name', weather_data.iloc[i].name)
            for j in range(i, i+time_steps+24):
                temp.append(
                    weather_data.iloc[j, weather_columns].values)
            for hour in range(24):
                X.append(np.array(temp[hour:hour+time_steps]))
            index = i
            break
X = np.array(X)
Y = pd.DataFrame(bee_np[season][:, 1])

# drop_combinations = []

# for i in range(len(leave_out[season])+1):
#     for j in itertools.combinations(leave_out[season], i):
#         temp = []
#         for k in j:
#             temp.append(k)
#         drop_combinations.append(temp)

# printdrop_combinations)


tests = 10
kf = KFold(n_splits=tests)

row = [[], []]
for train_index, test_index in kf.split(X):
    model = Sequential([
        GRU(8, activation='relu', return_sequences=True,
            input_shape=(time_steps, len(weather_columns))),
        Dropout(0.2),
        GRU(8, activation='relu',
            input_shape=(time_steps, len(weather_columns))),
        Dropout(0.2),
        Dense(1)
    ])
    model.compile(metrics=['mse'], loss='mse', optimizer='adam')

    X_train = X[train_index]
    X_test = X[test_index]
    Y_train = Y.iloc[train_index]
    Y_test = Y.iloc[test_index]

    X_train = np.asarray(X_train).astype('float32')
    X_test = np.asarray(X_test).astype('float32')
    Y_train = np.asarray(Y_train).astype('float32')
    Y_test = np.asarray(Y_test).astype('float32')

    scalers = []
    for i in range(X_train.shape[2]):
        scalers.append(StandardScaler())
        X_train[:, :, i] = scalers[i].fit_transform(
            X_train[:, :, i])
    for i in range(X_test.shape[2]):
        X_test[:, :, i] = scalers[i].transform(X_test[:, :, i])

    model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=100,
                batch_size=8, verbose=1)

    y_pred = model.predict(X_test).reshape(-1, 1)
    combined=pd.concat([pd.DataFrame(Y_test), pd.DataFrame(y_pred)], axis=1)
    plot_df = pd.concat([plot_df, combined], axis=0, ignore_index=True)
    mse = mean_squared_error(Y_test, y_pred)
    row[0].append(mse)
    r2 = r2_score(Y_test, y_pred)
    row[1].append(r2)
    # row_name = models[model_type]
    # for factor in drop_combinations[i_drop]:
    #      row_name+='-'+factor[:1]

# row_name = models[model_type]
# if len(drop_combinations[i_drop]) >= 1:
#     row_name += '-'
# row_name += '-'.join([factor[:1]
#                         for factor in drop_combinations[i_drop]])
# df[0][row_name] = row[0]
# df[1][row_name] = row[1]
# print(row_name)

# subset_metrics[0][-1].append(df[0].transpose())
# subset_metrics[1][-1].append(df[1].transpose())
# r2 = r2_score(Y_test, y_pred)
        # model_r2.append(r2)

# gru_mse_1=pd.DataFrame(model_mse)
# gru_r2_1=pd.DataFrame(model_r2)
# print(gbr_mse, gbr_mse.mean(), gbr_r2, gbr_r2.mean(), sep='\n')


In [ ]:
pd.DataFrame(row[1]).mean()


In [ ]:
row[1]


In [ ]:
pd.DataFrame(row[0]).mean()

In [ ]:
pd.DataFrame(row[1]).mean()

In [ ]:
pd.DataFrame(row[1]).mean()


In [ ]:
pd.DataFrame(row[0]).mean()


In [ ]:
pd.DataFrame(row[1]).mean()

In [ ]:
plt.cla()

# final scatterplot

In [ ]:
r2_arr=[]

In [ ]:
# final accuracy validation

plot_df = [pd.DataFrame() for i in range(3)]

optimal_features=[[0, 2, 3, 5, 6, 8], [0, 2, 3, 8], [0, 2, 3, 6, 8]]

# weather_columns = [0, 2, 3, 5, 6, 8]
# weather_columns=[0, 2, 8]
model_type = 1
# season = 0
time_steps = 9

for season in range(3):
    index = 0
    X = []

    weather_columns=optimal_features[season]

    for date in data_blocks[season]:
        for i in range(index, len(weather_data)):
            if weather_data.iloc[i].name == date-pd.Timedelta(hours=time_steps-1):
                temp = []
                # print('name', weather_data.iloc[i].name)
                for j in range(i, i+time_steps+24):
                    temp.append(
                        weather_data.iloc[j, weather_columns].values)
                for hour in range(24):
                    X.append(np.array(temp[hour:hour+time_steps]))
                index = i
                break
    X = np.array(X)
    Y = pd.DataFrame(bee_np[season][:, 1])

    # drop_combinations = []

    # for i in range(len(leave_out[season])+1):
    #     for j in itertools.combinations(leave_out[season], i):
    #         temp = []
    #         for k in j:
    #             temp.append(k)
    #         drop_combinations.append(temp)

    # printdrop_combinations)

    tests = 10
    kf = KFold(n_splits=tests)

    # row = [[], []]
    r2_arr.append([])
    for train_index, test_index in kf.split(X):
        model = Sequential([
            GRU(64, return_sequences=True,
                input_shape=(time_steps, len(weather_columns))),
            Dropout(0.2),
            GRU(64),
            Dropout(0.2),
            Dense(1)
        ])

        # (learning_rate=0.01)
        model.compile(metrics=['mse'], loss='mse', optimizer='adam')
        X_train = X[train_index]
        X_test = X[test_index]
        Y_train = Y.iloc[train_index]
        Y_test = Y.iloc[test_index]

        X_train = np.asarray(X_train).astype('float32')
        X_test = np.asarray(X_test).astype('float32')
        Y_train = np.asarray(Y_train).astype('float32')
        Y_test = np.asarray(Y_test).astype('float32')

        scalers = []
        for i in range(X_train.shape[2]):
            scalers.append(StandardScaler())
            X_train[:, :, i] = scalers[i].fit_transform(
                X_train[:, :, i])
        for i in range(X_test.shape[2]):
            X_test[:, :, i] = scalers[i].transform(X_test[:, :, i])

        model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=50,
                batch_size=16, verbose=1)

        y_pred = model.predict(X_test).reshape(-1, 1)
        for i in range(len(y_pred)):
            y_pred[i, 0]=max(y_pred[i, 0], 0)

        combined = pd.concat([pd.DataFrame(Y_test), pd.DataFrame(y_pred)], axis=1)
        plot_df[season] = pd.concat(
            [plot_df[season], combined], axis=0, ignore_index=True)
        # mse = mean_squared_error(Y_test, y_pred)
        # row[0].append(mse)
        r2 = r2_score(Y_test, y_pred)
        # row[1].append(r2)
        r2_arr[-1].append(r2)

    # print(pd.DataFrame(row[1]).mean())
    # row_name = models[model_type]
    # for factor in drop_combinations[i_drop]:
    #      row_name+='-'+factor[:1]

# row_name = models[model_type]
# if len(drop_combinations[i_drop]) >= 1:
#     row_name += '-'
# row_name += '-'.join([factor[:1]
#                         for factor in drop_combinations[i_drop]])
# df[0][row_name] = row[0]
# df[1][row_name] = row[1]
# print(row_name)

# subset_metrics[0][-1].append(df[0].transpose())
# subset_metrics[1][-1].append(df[1].transpose())
# r2 = r2_score(Y_test, y_pred)
    # model_r2.append(r2)

# gru_mse_1=pd.DataFrame(model_mse)
# gru_r2_1=pd.DataFrame(model_r2)
# print(gbr_mse, gbr_mse.mean(), gbr_r2, gbr_r2.mean(), sep='\n')



In [ ]:
weather_data.columns.tolist()[5]

In [ ]:
# LSTM final accuracy validation

plot_df = [pd.DataFrame() for i in range(3)]

optimal_features=[[0, 2, 3, 5, 8], [0, 2, 3,5,6, 8], [0, 2, 3, 6, 8]]

# weather_columns = [0, 2, 3, 5, 6, 8]
# weather_columns=[0, 2, 8]
model_type = 0
# season = 0
time_steps = 9

for season in range(3):
    index = 0
    X = []

    weather_columns=optimal_features[season]

    for date in data_blocks[season]:
        for i in range(index, len(weather_data)):
            if weather_data.iloc[i].name == date-pd.Timedelta(hours=time_steps-1):
                temp = []
                # print('name', weather_data.iloc[i].name)
                for j in range(i, i+time_steps+24):
                    temp.append(
                        weather_data.iloc[j, weather_columns].values)
                for hour in range(24):
                    X.append(np.array(temp[hour:hour+time_steps]))
                index = i
                break
    X = np.array(X)
    Y = pd.DataFrame(bee_np[season][:, 1])

    # drop_combinations = []

    # for i in range(len(leave_out[season])+1):
    #     for j in itertools.combinations(leave_out[season], i):
    #         temp = []
    #         for k in j:
    #             temp.append(k)
    #         drop_combinations.append(temp)

    # printdrop_combinations)

    tests = 10
    kf = KFold(n_splits=tests)

    # row = [[], []]
    r2_arr.append([])
    for train_index, test_index in kf.split(X):
        model = Sequential([
            LSTM(64, return_sequences=True,
                input_shape=(time_steps, len(weather_columns))),
            Dropout(0.2),
            LSTM(64),
            Dropout(0.2),
            Dense(1)
        ])

        # (learning_rate=0.01)
        model.compile(metrics=['mse'], loss='mse', optimizer='adam')
        X_train = X[train_index]
        X_test = X[test_index]
        Y_train = Y.iloc[train_index]
        Y_test = Y.iloc[test_index]

        X_train = np.asarray(X_train).astype('float32')
        X_test = np.asarray(X_test).astype('float32')
        Y_train = np.asarray(Y_train).astype('float32')
        Y_test = np.asarray(Y_test).astype('float32')

        scalers = []
        for i in range(X_train.shape[2]):
            scalers.append(StandardScaler())
            X_train[:, :, i] = scalers[i].fit_transform(
                X_train[:, :, i])
        for i in range(X_test.shape[2]):
            X_test[:, :, i] = scalers[i].transform(X_test[:, :, i])

        model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=75,
                batch_size=8, verbose=1)

        y_pred = model.predict(X_test).reshape(-1, 1)
        for i in range(len(y_pred)):
            y_pred[i, 0]=max(y_pred[i, 0], 0)

        combined = pd.concat([pd.DataFrame(Y_test), pd.DataFrame(y_pred)], axis=1)
        plot_df[season] = pd.concat(
            [plot_df[season], combined], axis=0, ignore_index=True)
        # mse = mean_squared_error(Y_test, y_pred)
        # row[0].append(mse)
        r2 = r2_score(Y_test, y_pred)
        # row[1].append(r2)
        r2_arr[-1].append(r2)

    # print(pd.DataFrame(row[1]).mean())
    # row_name = models[model_type]
    # for factor in drop_combinations[i_drop]:
    #      row_name+='-'+factor[:1]

# row_name = models[model_type]
# if len(drop_combinations[i_drop]) >= 1:
#     row_name += '-'
# row_name += '-'.join([factor[:1]
#                         for factor in drop_combinations[i_drop]])
# df[0][row_name] = row[0]
# df[1][row_name] = row[1]
# print(row_name)

# subset_metrics[0][-1].append(df[0].transpose())
# subset_metrics[1][-1].append(df[1].transpose())
# r2 = r2_score(Y_test, y_pred)
    # model_r2.append(r2)

# gru_mse_1=pd.DataFrame(model_mse)
# gru_r2_1=pd.DataFrame(model_r2)
# print(gbr_mse, gbr_mse.mean(), gbr_r2, gbr_r2.mean(), sep='\n')



In [ ]:
r2_arr

In [ ]:
pd.DataFrame(r2_arr[0]).mean()

In [ ]:
pd.DataFrame(r2_arr[1]).mean()

In [ ]:
pd.DataFrame(r2_arr[2]).mean()

In [ ]:
pd.DataFrame(r2_arr[-1]).mean()

In [ ]:
for i in range(4, 7):
    print(pd.DataFrame(r2_arr[-i]).mean())

In [ ]:
for i in range(1, 4):
    print(pd.DataFrame(r2_arr[-i]).mean())


In [ ]:
for i in range(1, 4):
    print(pd.DataFrame(r2_arr[-i]).mean())

In [ ]:
for i in range(1, 4):
    print(pd.DataFrame(r2_arr[-i]).mean())

In [ ]:
saved_plot_df=plot_df

In [ ]:
sns.set_theme(style='white', font='Arial', font_scale=2)

In [ ]:
# Summer LSTM; Exclude pressure
# Fall LSTM: Exclude rainfall
# Spring LSTM: Exclude pressure and rainfall
# Summer GRU; Exclude pressure
# Fall GRU: Exclude rainfall
# Spring GRU: Exclude rainfall
table_df = [['Summer', '0.86', '0.91'], 
            ['Fall', '0.83', '0.89'], 
            ['Spring', '0.71', '0.84']]

table_df = pd.DataFrame(
    table_df, columns=['Season', 'Initial R2\nScore', 'Final R2\nScore'])
title_text="Improvement Through Model Optimization"

# matplotlib.rcParams.update({'font.size': 24})

# title_text = 'Model Improvement Through Optimization'
# Pop the headers from the data array
column_headers = list(table_df.columns)
# row_headers = list(table_df.index)
# Get some lists of color specs for row and column headers
#     rcolors = plt.cm.GnBu(np.full(len(row_headers), 0.6))
#     ccolors = plt.cm.GnBu(np.full(len(column_headers), 0.6))
ax_color = ['#f0f043' for i in range(len(column_headers))]
# Create the figure. Setting a small pad on tight_layout
# seems to better regulate white space. Sometimes experimenting
# with an explicit figsize here can produce better outcome.
plt.figure(linewidth=2,
        edgecolor='black',
        facecolor='white',
        tight_layout={'pad':.5},
        figsize=(3,3)
        )
# Add a table at the bottom of the axes
table = plt.table(cellText=table_df.values,
                    cellLoc='center',
                    # rowLabels=row_headers,
                    # rowColours=ax_color,
                    # rowLoc='center',
                    colColours=ax_color,
                    colLabels=column_headers,
                    loc='center')
table.scale(1, 3)
table.auto_set_font_size(False)
table.set_fontsize(12)

# for (row, col), cell in table.get_celld().items():
#     if row>=1:
#         # print(row, col)
#         if vifs[season].iloc[row-1, col]!='und.' and float(vifs[season].iloc[row-1, col])> 5 and col!=-1:
#             cell.set_text_props(fontproperties=FontProperties(weight='bold'))

ax = plt.gca()
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
plt.box(on=None)
plt.title(title_text)
plt.draw()
fig = plt.gcf()

plt.savefig(os.path.join(directory, 'Figures',
            'model_improvement.jpg'), dpi=600, bbox_inches='tight', 
                edgecolor=fig.get_edgecolor(), facecolor=fig.get_facecolor(),)
plt.show()


In [ ]:
arr[pd.to_datetime("2023-01-29")]

In [ ]:
for season in range(3):
    max_val=0
    for i in range(len(saved_plot_df[season])):
        for j in range(2):
            if saved_plot_df[season].iloc[i, j]>max_val:
                max_val=saved_plot_df[season].iloc[i, j]

    saved_plot_df[season].index=[i for i in range(len(saved_plot_df[season]))]
    drop=[]
    for i in range(len(saved_plot_df[season])):
        if saved_plot_df[season].iloc[i, 0]==0:
            drop.append(i)
    for index in drop:
        saved_plot_df[season] = saved_plot_df[season].drop(index=[index])
    saved_plot_df[season].index = [i for i in range(len(saved_plot_df[season]))]

    saved_plot_df[season].columns=['Actual Bee Activity', 'Predicted Bee Activity']

    # fig, ax=plt.subplots(figsize=(10, 10))
    sns.scatterplot(data=saved_plot_df[season], x='Actual Bee Activity', y='Predicted Bee Activity', palette=['skyblue'])
    # sns.lineplot(data=pd.DataFrame([[0, 0], [max_val, max_val]]), palette=['skyblue'], legend=False)
    sns.lineplot(data=pd.DataFrame([[0, 0], [max_val, max_val]], columns=['x', 'y']), x='x', y='y', legend=False)
    plt.title('Actual Bee Activity vs. Predicted Bee Activity ('+seasons[season]+')')
    if season==2:
        plt.savefig(os.path.join(directory, 'Figures',
                    'actual_vs_predicted_'+seasons[season]+'.jpg'), dpi=600, bbox_inches='tight')
    plt.show()
    plt.cla()


In [ ]:
for season in range(3):
    max_val=0
    for i in range(len(saved_plot_df[season])):
        for j in range(2):
            if saved_plot_df[season].iloc[i, j]>max_val:
                max_val=saved_plot_df[season].iloc[i, j]

    saved_plot_df[season].index=[i for i in range(len(saved_plot_df[season]))]
    drop=[]
    for i in range(len(saved_plot_df[season])):
        if saved_plot_df[season].iloc[i, 0]==0:
            drop.append(i)
    for index in drop:
        saved_plot_df[season] = saved_plot_df[season].drop(index=[index])
    saved_plot_df[season].index = [i for i in range(len(saved_plot_df[season]))]

    saved_plot_df[season].columns=['Actual Bee Activity', 'Predicted Bee Activity']

    # fig, ax=plt.subplots(figsize=(10, 10))
    sns.scatterplot(data=saved_plot_df[season], x='Actual Bee Activity', y='Predicted Bee Activity', palette=['skyblue'])
    # sns.lineplot(data=pd.DataFrame([[0, 0], [max_val, max_val]]), palette=['skyblue'], legend=False)
    sns.lineplot(data=pd.DataFrame([[0, 0], [max_val, max_val]], columns=['x', 'y']), x='x', y='y', legend=False)
    plt.title('Actual Bee Activity vs. Predicted Bee Activity ('+seasons[season]+')')
    # plt.savefig(os.path.join(directory, 'Figures',
    #             'actual_vs_predicted_'+seasons[season]+'.jpg'), dpi=600, bbox_inches='tight')
    plt.show()
    plt.cla()


In [ ]:
r2_arr=[]

In [ ]:
plot_df = [pd.DataFrame() for i in range(3)]

optimal_features = [[0, 2, 3, 8], [0, 2, 3, 5, 8], [0, 2, 3, 5, 8]]

# weather_columns = [0, 2, 3, 5, 6, 8]
# weather_columns=[0, 2, 8]
model_type = 1
# season = 0
time_steps = 9

for season in range(2,3):
    index = 0
    X = []

    weather_columns = optimal_features[season]

    for date in data_blocks[season]:
        for i in range(index, len(weather_data)):
            if weather_data.iloc[i].name == date-pd.Timedelta(hours=time_steps-1):
                temp = []
                # print('name', weather_data.iloc[i].name)
                for j in range(i, i+time_steps+24):
                    temp.append(
                        weather_data.iloc[j, weather_columns].values)
                for hour in range(24):
                    X.append(np.array(temp[hour:hour+time_steps]))
                index = i
                break
    X = np.array(X)
    Y = pd.DataFrame(bee_np[season][:, 1])

    # drop_combinations = []

    # for i in range(len(leave_out[season])+1):
    #     for j in itertools.combinations(leave_out[season], i):
    #         temp = []
    #         for k in j:
    #             temp.append(k)
    #         drop_combinations.append(temp)

    # printdrop_combinations)

    tests = 10
    kf = KFold(n_splits=tests)

    # row = [[], []]
    r2_arr.append([])
    for train_index, test_index in kf.split(X):
        model = Sequential([
            GRU(64, activation='relu', return_sequences=True,
                input_shape=(time_steps, len(weather_columns))),
            Dropout(0.2),
            GRU(64, activation='relu',
                input_shape=(time_steps, len(weather_columns))),
            Dropout(0.2),
            Dense(1)
        ])

        # (learning_rate=0.01)
        model.compile(metrics=['mse'], loss='mse', optimizer='adam')
        X_train = X[train_index]
        X_test = X[test_index]
        Y_train = Y.iloc[train_index]
        Y_test = Y.iloc[test_index]

        X_train = np.asarray(X_train).astype('float32')
        X_test = np.asarray(X_test).astype('float32')
        Y_train = np.asarray(Y_train).astype('float32')
        Y_test = np.asarray(Y_test).astype('float32')

        scalers = []
        for i in range(X_train.shape[2]):
            scalers.append(StandardScaler())
            X_train[:, :, i] = scalers[i].fit_transform(
                X_train[:, :, i])
        for i in range(X_test.shape[2]):
            X_test[:, :, i] = scalers[i].transform(X_test[:, :, i])

        model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=50,
                  batch_size=16, verbose=1)

        y_pred = model.predict(X_test).reshape(-1, 1)
        combined = pd.concat(
            [pd.DataFrame(Y_test), pd.DataFrame(y_pred)], axis=1)
        plot_df[season] = pd.concat(
            [plot_df[season], combined], axis=0, ignore_index=True)
        # mse = mean_squared_error(Y_test, y_pred)
        # row[0].append(mse)
        r2 = r2_score(Y_test, y_pred)
        # row[1].append(r2)
        r2_arr[-1].append(r2)

    # print(pd.DataFrame(row[1]).mean())
    # row_name = models[model_type]
    # for factor in drop_combinations[i_drop]:
    #      row_name+='-'+factor[:1]

# row_name = models[model_type]
# if len(drop_combinations[i_drop]) >= 1:
#     row_name += '-'
# row_name += '-'.join([factor[:1]
#                         for factor in drop_combinations[i_drop]])
# df[0][row_name] = row[0]
# df[1][row_name] = row[1]
# print(row_name)

# subset_metrics[0][-1].append(df[0].transpose())
# subset_metrics[1][-1].append(df[1].transpose())
# r2 = r2_score(Y_test, y_pred)
    # model_r2.append(r2)

# gru_mse_1=pd.DataFrame(model_mse)
# gru_r2_1=pd.DataFrame(model_r2)
# print(gbr_mse, gbr_mse.mean(), gbr_r2, gbr_r2.mean(), sep='\n')


# bee hour plot

In [ ]:
#model testing
model = Sequential([
    LSTM(100, return_sequences=True, activation='relu',
        input_shape=(time_steps, len(weather_columns))),
    Dropout(0.15),
    LSTM(100, activation='relu'),
    Dropout(0.15),
    Dense(1)
])
model.compile(metrics=['mse'], loss='mse', optimizer='adam')

time_steps=24
X = []

# time_steps = optimal_time_steps[model_type][season]
index = 0

for date in data_blocks[season]:
    for i in range(index, len(weather_data)):
        if weather_data.iloc[i].name == date-pd.Timedelta(hours=time_steps-1):
            temp = []
            # print('name', weather_data.iloc[i].name)
            for j in range(i, i+time_steps+24):
                temp.append(
                    weather_data.iloc[j, weather_columns].values)
            for hour in range(24):
                X.append(np.array(temp[hour:hour+time_steps]))
            index = i
            break
X = np.array(X)
Y = pd.DataFrame(bee_np[season][:, 1])

tests = 10
kf = KFold(n_splits=tests)

r2_arr.append([])
for train_index, test_index in kf.split(X):
    X_train = X[train_index]
    X_test = X[test_index]
    Y_train = Y.iloc[train_index]
    Y_test = Y.iloc[test_index]

    X_train = np.asarray(X_train).astype('float32')
    X_test = np.asarray(X_test).astype('float32')
    Y_train = np.asarray(Y_train).astype('float32')
    Y_test = np.asarray(Y_test).astype('float32')

    scalers = []
    for i in range(X_train.shape[2]):
        scalers.append(StandardScaler())
        X_train[:, :, i] = scalers[i].fit_transform(
            X_train[:, :, i])
    for i in range(X_test.shape[2]):
        X_test[:, :, i] = scalers[i].transform(X_test[:, :, i])

    model.fit(X_train, Y_train, epochs=50,
            batch_size=16, validation_data=(X_test, Y_test), verbose=1)
    # model.fit(X_train, Y_train, epochs=40,
    #         batch_size=128, verbose=0)

    y_pred = model.predict(X_test).reshape(-1, 1)
    
    # mse = mean_squared_error(Y_test, y_pred)
    # row[0].append(mse)
    r2 = r2_score(Y_test, y_pred)
    # row[1].append(r2)
    r2_arr[-1].append(r2)

In [ ]:
pd.DataFrame(r2_arr[-1]).mean()

In [ ]:
# LSTM
pd.DataFrame(row[1]).mean()

In [ ]:
# batch 8
pd.DataFrame(row[1]).mean()

In [ ]:
# 32 nodes
pd.DataFrame(row[1]).mean()

In [ ]:
# 100 nodes
pd.DataFrame(row[1]).mean()


In [ ]:
pd.DataFrame(row[1]).mean()


In [ ]:
pd.DataFrame(row[1]).mean()

In [ ]:
pd.DataFrame(row[1]).mean()


In [ ]:
#model testing

for season in range(1, 3):
    # model = Sequential([
    #     GRU(64, return_sequences=True, activation='relu',
    #         input_shape=(time_steps, len(weather_columns))),
    #     Dropout(0.2),
    #     GRU(64, activation='relu'),
    #     Dropout(0.2),
    #     Dense(1)
    # ])
    # model.compile(metrics=['mse'], loss='mse', optimizer='adam')

    time_steps=24
    X = []

    # time_steps = optimal_time_steps[model_type][season]
    index = 0

    for date in data_blocks[season]:
        for i in range(index, len(weather_data)):
            if weather_data.iloc[i].name == date-pd.Timedelta(hours=time_steps-1):
                temp = []
                # print('name', weather_data.iloc[i].name)
                for j in range(i, i+time_steps+24):
                    temp.append(
                        weather_data.iloc[j, weather_columns].values)
                for hour in range(24):
                    X.append(np.array(temp[hour:hour+time_steps]))
                index = i
                break
    X = np.array(X)
    Y = pd.DataFrame(bee_np[season][:, 1])

    tests = 10
    kf = KFold(n_splits=tests)

    r2_arr.append([])
    for train_index, test_index in kf.split(X):
        model = Sequential([
            GRU(64, return_sequences=True, activation='relu',
                input_shape=(time_steps, len(weather_columns))),
            Dropout(0.2),
            GRU(64, activation='relu'),
            Dropout(0.2),
            Dense(1)
        ])
        model.compile(metrics=['mse'], loss='mse', optimizer='adam')
        X_train = X[train_index]
        X_test = X[test_index]
        Y_train = Y.iloc[train_index]
        Y_test = Y.iloc[test_index]

        X_train = np.asarray(X_train).astype('float32')
        X_test = np.asarray(X_test).astype('float32')
        Y_train = np.asarray(Y_train).astype('float32')
        Y_test = np.asarray(Y_test).astype('float32')

        scalers = []
        for i in range(X_train.shape[2]):
            scalers.append(StandardScaler())
            X_train[:, :, i] = scalers[i].fit_transform(
                X_train[:, :, i])
        for i in range(X_test.shape[2]):
            X_test[:, :, i] = scalers[i].transform(X_test[:, :, i])

        model.fit(X_train, Y_train, epochs=50,
                batch_size=16, validation_data=(X_test, Y_test), verbose=1)
        # model.fit(X_train, Y_train, epochs=40,
        #         batch_size=128, verbose=0)

        y_pred = model.predict(X_test).reshape(-1, 1)
        
        # mse = mean_squared_error(Y_test, y_pred)
        # row[0].append(mse)
        r2 = r2_score(Y_test, y_pred)
        # row[1].append(r2)
        r2_arr[-1].append(r2)

In [ ]:
#model testing

# model = Sequential([
#     GRU(64, return_sequences=True, activation='relu',
#         input_shape=(time_steps, len(weather_columns))),
#     Dropout(0.2),
#     GRU(64, activation='relu'),
#     Dropout(0.2),
#     Dense(1)
# ])
# model.compile(metrics=['mse'], loss='mse', optimizer='adam')

season=2

time_steps = 24
X = []

# time_steps = optimal_time_steps[model_type][season]
index = 0

for date in data_blocks[season]:
    for i in range(index, len(weather_data)):
        if weather_data.iloc[i].name == date-pd.Timedelta(hours=time_steps-1):
            temp = []
            # print('name', weather_data.iloc[i].name)
            for j in range(i, i+time_steps+24):
                temp.append(
                    weather_data.iloc[j, weather_columns].values)
            for hour in range(24):
                X.append(np.array(temp[hour:hour+time_steps]))
            index = i
            break
X = np.array(X)
Y = pd.DataFrame(bee_np[season][:, 1])

tests = 10
kf = KFold(n_splits=tests)

r2_arr.append([])
for train_index, test_index in kf.split(X):
    model = Sequential([
        GRU(64, return_sequences=True, activation='relu',
            input_shape=(time_steps, len(weather_columns))),
        Dropout(0.2),
        GRU(64, activation='relu'),
        Dropout(0.2),
        Dense(1)
    ])
    model.compile(metrics=['mse'], loss='mse', optimizer='adam')
    X_train = X[train_index]
    X_test = X[test_index]
    Y_train = Y.iloc[train_index]
    Y_test = Y.iloc[test_index]

    X_train = np.asarray(X_train).astype('float32')
    X_test = np.asarray(X_test).astype('float32')
    Y_train = np.asarray(Y_train).astype('float32')
    Y_test = np.asarray(Y_test).astype('float32')

    scalers = []
    for i in range(X_train.shape[2]):
        scalers.append(StandardScaler())
        X_train[:, :, i] = scalers[i].fit_transform(
            X_train[:, :, i])
    for i in range(X_test.shape[2]):
        X_test[:, :, i] = scalers[i].transform(X_test[:, :, i])

    model.fit(X_train, Y_train, epochs=50,
            batch_size=16, validation_data=(X_test, Y_test), verbose=1)
    # model.fit(X_train, Y_train, epochs=40,
    #         batch_size=128, verbose=0)

    y_pred = model.predict(X_test).reshape(-1, 1)

    # mse = mean_squared_error(Y_test, y_pred)
    # row[0].append(mse)
    r2 = r2_score(Y_test, y_pred)
    # row[1].append(r2)
    r2_arr[-1].append(r2)


# bee hour plot

In [ ]:


weather_columns = [0, 2, 3, 5, 6, 8]
# weather_columns=[0, 2, 8]
model_type = 1
pred_dates=[set() for i in range(3)]
binary_pred=[{} for i in range(3)]
bee_hour = [{} for i in range(3)]
bee_hour2 = [{} for i in range(3)]
time_steps=9

for season in range(3):
    X = []

    # time_steps = optimal_time_steps[model_type][season]
    index = 0

    for date in data_blocks[season]:
        for i in range(index, len(weather_data)):
            if weather_data.iloc[i].name == date-pd.Timedelta(hours=time_steps-1):
                temp = []
                # print('name', weather_data.iloc[i].name)
                for j in range(i, i+time_steps+24):
                    temp.append(
                        weather_data.iloc[j, weather_columns].values)
                for hour in range(24):
                    X.append(np.array(temp[hour:hour+time_steps]))
                index = i
                break
    X = np.array(X)
    Y = pd.DataFrame(bee_np[season][:, 1])

    # drop_combinations = []

    # for i in range(len(leave_out[season])+1):
    #     for j in itertools.combinations(leave_out[season], i):
    #         temp = []
    #         for k in j:
    #             temp.append(k)
    #         drop_combinations.append(temp)

    # printdrop_combinations)

    # model = Sequential([
    #     GRU(64, return_sequences=True, activation='relu',
    #         input_shape=(time_steps, len(weather_columns))),
    #     Dropout(0.2),
    #     GRU(64, activation='relu'),
    #     Dropout(0.2),
    #     Dense(1)
    # ])

    tests = 10
    kf = KFold(n_splits=tests)

    row = [[], []]
    for train_index, test_index in kf.split(X):
        model = Sequential([
            GRU(64, return_sequences=True,
                input_shape=(time_steps, len(weather_columns))),
            Dropout(0.2),
            GRU(64),
            Dropout(0.2),
            Dense(1)
        ])
        # (learning_rate=0.01)
        model.compile(metrics=['mse'], loss='mse', optimizer='adam')
        
        X_train = X[train_index]
        X_test = X[test_index]
        Y_train = Y.iloc[train_index]
        Y_test = Y.iloc[test_index]

        X_train = np.asarray(X_train).astype('float32')
        X_test = np.asarray(X_test).astype('float32')
        Y_train = np.asarray(Y_train).astype('float32')
        Y_test = np.asarray(Y_test).astype('float32')

        X_train_scaled = np.asarray(X_train).astype('float32')
        X_test_scaled = np.asarray(X_test).astype('float32')

        scalers = []
        for i in range(X_train.shape[2]):
            scalers.append(StandardScaler())
            X_train_scaled[:, :, i] = scalers[i].fit_transform(
                X_train_scaled[:, :, i])
        for i in range(X_test.shape[2]):
            X_test_scaled[:, :, i] = scalers[i].transform(X_test_scaled[:, :, i])

        model.fit(X_train_scaled, Y_train, epochs=10,
                batch_size=16, verbose=0)
        # model.fit(X_train, Y_train, epochs=40,
        #         batch_size=128, verbose=0)

        y_pred = model.predict(X_test_scaled).reshape(-1, 1)

        for i in range(len(y_pred)):
            date=bee_np[season][test_index[i]][-1]
            pred_dates[season].add(date)
            if not date in binary_pred[season]:
                binary_pred[season][date]=0
            if not date in bee_hour[season]:
                bee_hour[season][date]=0
            if not date in bee_hour2[season]:
                bee_hour2[season][date]=0
            if y_pred[i]>=0.3:
                binary_pred[season][date]+=1
            if X_test[i][-1][0]>55 and X_test[i][-1][2]<15 and X_test[i][-1][4]==0:
                bee_hour[season][date] += 1
            if X_test[i][-1][0] > 59 and X_test[i][-1][2] < 10 and X_test[i][-1][4] == 0:
                bee_hour2[season][date] += 1

        # combined = pd.concat([pd.DataFrame(y_pred), pd.DataFrame(Y_test)], axis=1)
        # plot_df[season] = pd.concat(
        #     [plot_df[season], combined], axis=0, ignore_index=True)
        mse = mean_squared_error(Y_test, y_pred)
        row[0].append(mse)
        r2 = r2_score(Y_test, y_pred)
        row[1].append(r2)
        # if plot_df.iloc[i, 0] == 0:
        #     plot_df.drop(index=[i])
    print(pd.DataFrame(row[1]).mean())


In [ ]:
sns.set_theme(rc={'figure.figsize': (10, 6)},
              style='white', font='Arial', font_scale=1.1)

for season in range(3):
    plt.cla()
    indices = []
    for i in sorted(pred_dates[season]):
        indices.append(i)

    indexed = []
    indexed2 = []
    indexed3 = []
    for date in indices:
        indexed.append(binary_pred[season][date])
        indexed2.append(bee_hour[season][date])
        indexed3.append(bee_hour2[season][date])

    indexed = pd.DataFrame(indexed)
    indexed.index = indices
    indexed.columns = ['Model Prediction']
    indexed2 = pd.DataFrame(indexed2)
    indexed2.index = indices
    indexed2.columns = ['Bee Hour Calculation (Version 1)']
    indexed3= pd.DataFrame(indexed3)
    indexed3.index = indices
    indexed3.columns = ['Bee Hour Calculation (Version 2)']

    # fig, ax = plt.subplots(figsize=(12, 8))
    # sns.lineplot(data=indexed, ax=ax)
    # sns.lineplot(data=indexed2, ax=ax, palette=['red'])
    # ax.set_ylim(bottom=0, top=0.25)
    # ax.xaxis.set_ticks(indices)
    # ax.xaxis.set_ticklabels(indices)
    # ax.xaxis.set_ticks_position('bottom')
    # ax.yaxis.set_ticks_position('left')
    # plt.xticks(rotation=0)

    sns.lineplot(data=indexed, palette=['green'])
    sns.lineplot(data=indexed2, palette=['red'])
    sns.lineplot(data=indexed3, palette=['blue'])

    # indexed.plot(ax=ax, legend=True)
    # indexed2.plot(ax=ax, legend=True, c='r')

    plt.xlabel('Date')
    plt.ylabel('Daily Bee Hours')
    plt.title(
        'Daily Bee Hours: Model Prediction vs. Bee Hour Calculation (' + seasons[season]+')')
    # plt.savefig(os.path.join(directory, 'Figures',
    #             'bee_hour_'+seasons[season]+'.jpg'), dpi=600, bbox_inches='tight')
    
    plt.show()


# full bee hour plots

In [ ]:
weather_data.iloc[0, :-1]

In [ ]:
weather_columns


In [ ]:
X[train_index][:, :, [weather_columns]].squeeze().shape


In [ ]:
#H5

# weather_columns_full = [[0, 2, 3, 6, 8], [0, 2, 3, 5, 8], [0, 2, 3, 5, 8]]
optimal_features=[[0, 2, 3, 5, 6, 8], [0, 2, 3, 8], [0, 2, 3, 6, 8]]
# weather_columns=[0, 2, 8]
pred_dates=[set() for i in range(3)]
scaled_pred=[{} for i in range(3)]
scaled_actual=[{} for i in range(3)]
binary_pred=[{} for i in range(3)]
bee_hour = [{} for i in range(3)]
binary_actual=[{} for i in range(3)]
# bee_hour2 = [{} for i in range(3)]
time_steps=9

for season in range(3):
    X = []

    # time_steps = optimal_time_steps[model_type][season]
    index = 0
    weather_columns=optimal_features[season]

    for date in data_blocks[season]:
        for i in range(index, len(weather_data)):
            if weather_data.iloc[i].name == date-pd.Timedelta(hours=time_steps-1):
                temp = []
                # print('name', weather_data.iloc[i].name)
                for j in range(i, i+time_steps+24):
                    # print(j, date, j-i-time_steps)
                    temp.append(
                        weather_data.iloc[j, :-1].values)
                for hour in range(24):
                    X.append(np.array(temp[hour:hour+time_steps]))
                index = i
                break
    X = np.array(X)
    Y = pd.DataFrame(bee_np[season][:, 1])

    # drop_combinations = []

    # for i in range(len(leave_out[season])+1):
    #     for j in itertools.combinations(leave_out[season], i):
    #         temp = []
    #         for k in j:
    #             temp.append(k)
    #         drop_combinations.append(temp)

    # printdrop_combinations)

    # model = Sequential([
    #     GRU(64, return_sequences=True, activation='relu',
    #         input_shape=(time_steps, len(weather_columns))),
    #     Dropout(0.2),
    #     GRU(64, activation='relu'),
    #     Dropout(0.2),
    #     Dense(1)
    # ])

    tests = 10
    kf = KFold(n_splits=tests)

    row = [[], []]
    for train_index, test_index in kf.split(X):
        model = Sequential([
            GRU(64, return_sequences=True,
                input_shape=(time_steps, len(weather_columns))),
            Dropout(0.2),
            GRU(64),
            Dropout(0.2),
            Dense(1)
        ])
        # (learning_rate=0.01)
        model.compile(metrics=['mse'], loss='mse', optimizer='adam')
        
        X_train = X[train_index][:, :, [weather_columns]].squeeze()
        X_test = X[test_index][:, :, [weather_columns]].squeeze()
        Y_train = Y.iloc[train_index]
        Y_test = Y.iloc[test_index]

        X_train = np.asarray(X_train).astype('float32')
        X_test = np.asarray(X_test).astype('float32')
        Y_train = np.asarray(Y_train).astype('float32')
        Y_test = np.asarray(Y_test).astype('float32')

        X_train_scaled = np.asarray(X_train).astype('float32')
        X_test_scaled = np.asarray(X_test).astype('float32')

        scalers = []
        for i in range(X_train.shape[2]):
            scalers.append(StandardScaler())
            X_train_scaled[:, :, i] = scalers[i].fit_transform(
                X_train_scaled[:, :, i])
        for i in range(X_test.shape[2]):
            X_test_scaled[:, :, i] = scalers[i].transform(X_test_scaled[:, :, i])

        model.fit(X_train_scaled, Y_train, epochs=50,
                batch_size=16, verbose=0)
        # model.fit(X_train, Y_train, epochs=40,
        #         batch_size=128, verbose=0)

        y_pred = model.predict(X_test_scaled).reshape(-1, 1)

        for i in range(len(y_pred)):
            date=bee_np[season][test_index[i]][-1]
            pred_dates[season].add(date)
            if not date in scaled_pred[season]:
                scaled_pred[season][date] = 0
            if not date in scaled_actual[season]:
                scaled_actual[season][date] = 0
            if not date in bee_hour[season]:
                bee_hour[season][date] = 0
            if not date in binary_pred[season]:
                binary_pred[season][date] = 0

            if y_pred[i] >= 0.3:
                binary_pred[season][date] += 1
            if Y_test[i] >= 0.3:
                binary_actual[season][date] += 1
            scaled_pred[season][date]+=y_pred[i]/0.3
            scaled_actual[season][date]+=Y_test[i]/0.3
            if X[test_index][i][-1][0]>55 and X[test_index][i][-1][3]<15 and X[test_index][i][-1][6]==0:
                bee_hour[season][date] += 1

        # combined = pd.concat([pd.DataFrame(y_pred), pd.DataFrame(Y_test)], axis=1)
        # plot_df[season] = pd.concat(
        #     [plot_df[season], combined], axis=0, ignore_index=True)
        # mse = mean_squared_error(Y_test, y_pred)
        # row[0].append(mse)
        r2 = r2_score(Y_test, y_pred)
        row[1].append(r2)
        # if plot_df.iloc[i, 0] == 0:
        #     plot_df.drop(index=[i])
    print(pd.DataFrame(row[1]).mean())


In [ ]:
#H5

# weather_columns_full = [[0, 2, 3, 6, 8], [0, 2, 3, 5, 8], [0, 2, 3, 5, 8]]
optimal_features=[[0, 2, 3, 5, 6, 8], [0, 2, 3, 8], [0, 2, 3, 6, 8]]
# weather_columns=[0, 2, 8]
pred_dates=[set() for i in range(3)]
scaled_pred=[{} for i in range(3)]
scaled_actual=[{} for i in range(3)]
binary_pred=[{} for i in range(3)]
bee_hour = [{} for i in range(3)]
binary_actual=[{} for i in range(3)]
# bee_hour2 = [{} for i in range(3)]
time_steps=9

for season in range(3):
    X = []

    # time_steps = optimal_time_steps[model_type][season]
    index = 0
    weather_columns=optimal_features[season]

    for date in data_blocks[season]:
        for i in range(index, len(weather_data)):
            if weather_data.iloc[i].name == date-pd.Timedelta(hours=time_steps-1):
                temp = []
                # print('name', weather_data.iloc[i].name)
                for j in range(i, i+time_steps+24):
                    # print(j, date, j-i-time_steps)
                    temp.append(
                        weather_data.iloc[j, :-1].values)
                for hour in range(24):
                    X.append(np.array(temp[hour:hour+time_steps]))
                index = i
                break
    X = np.array(X)
    Y = pd.DataFrame(bee_np[season][:, 1])

    # drop_combinations = []

    # for i in range(len(leave_out[season])+1):
    #     for j in itertools.combinations(leave_out[season], i):
    #         temp = []
    #         for k in j:
    #             temp.append(k)
    #         drop_combinations.append(temp)

    # printdrop_combinations)

    # model = Sequential([
    #     GRU(64, return_sequences=True, activation='relu',
    #         input_shape=(time_steps, len(weather_columns))),
    #     Dropout(0.2),
    #     GRU(64, activation='relu'),
    #     Dropout(0.2),
    #     Dense(1)
    # ])

    tests = 10
    kf = KFold(n_splits=tests)

    row = [[], []]
    for train_index, test_index in kf.split(X):
        model = Sequential([
            GRU(64, return_sequences=True,
                input_shape=(time_steps, len(weather_columns))),
            Dropout(0.2),
            GRU(64),
            Dropout(0.2),
            Dense(1)
        ])
        # (learning_rate=0.01)
        model.compile(metrics=['mse'], loss='mse', optimizer='adam')
        
        X_train = X[train_index][:, :, [weather_columns]].squeeze()
        X_test = X[test_index][:, :, [weather_columns]].squeeze()
        Y_train = Y.iloc[train_index]
        Y_test = Y.iloc[test_index]

        X_train = np.asarray(X_train).astype('float32')
        X_test = np.asarray(X_test).astype('float32')
        Y_train = np.asarray(Y_train).astype('float32')
        Y_test = np.asarray(Y_test).astype('float32')

        X_train_scaled = np.asarray(X_train).astype('float32')
        X_test_scaled = np.asarray(X_test).astype('float32')

        scalers = []
        for i in range(X_train.shape[2]):
            scalers.append(StandardScaler())
            X_train_scaled[:, :, i] = scalers[i].fit_transform(
                X_train_scaled[:, :, i])
        for i in range(X_test.shape[2]):
            X_test_scaled[:, :, i] = scalers[i].transform(X_test_scaled[:, :, i])

        model.fit(X_train_scaled, Y_train, epochs=50,
                batch_size=16, verbose=0)
        # model.fit(X_train, Y_train, epochs=40,
        #         batch_size=128, verbose=0)

        y_pred = model.predict(X_test_scaled).reshape(-1, 1)

        for i in range(len(y_pred)):
            date=bee_np[season][test_index[i]][-1]
            pred_dates[season].add(date)
            if not date in scaled_pred[season]:
                scaled_pred[season][date] = 0
            if not date in scaled_actual[season]:
                scaled_actual[season][date] = 0
            if not date in bee_hour[season]:
                bee_hour[season][date] = 0
            if not date in binary_pred[season]:
                binary_pred[season][date] = 0
            if not date in binary_actual[season]:
                binary_actual[season][date] = 0

            if y_pred[i] >= 0.3:
                binary_pred[season][date] += 1
            if Y_test[i] >= 0.3:
                binary_actual[season][date] += 1
            scaled_pred[season][date]+=y_pred[i]/0.3
            scaled_actual[season][date]+=Y_test[i]/0.3
            if X[test_index][i][-1][0]>55 and X[test_index][i][-1][3]<15 and X[test_index][i][-1][6]==0:
                bee_hour[season][date] += 1

        # combined = pd.concat([pd.DataFrame(y_pred), pd.DataFrame(Y_test)], axis=1)
        # plot_df[season] = pd.concat(
        #     [plot_df[season], combined], axis=0, ignore_index=True)
        # mse = mean_squared_error(Y_test, y_pred)
        # row[0].append(mse)
        r2 = r2_score(Y_test, y_pred)
        row[1].append(r2)
        # if plot_df.iloc[i, 0] == 0:
        #     plot_df.drop(index=[i])
    print(pd.DataFrame(row[1]).mean())


In [ ]:
row[1]

In [ ]:
0.84
exclude both:


In [ ]:
exclude pressure: 0.802082

In [ ]:
exclude rainfall: 0.811197

In [ ]:
0.839882

In [ ]:
row[1] .823

In [ ]:
seasons=["Summer", "Fall", "Spring"]

In [ ]:
sns.set_theme(rc={'figure.figsize': (10, 6)},
              style='white', font='Arial', font_scale=1.1)

num_lines = 3
line_values = [scaled_pred, scaled_actual, bee_hour]
line_labels = ['Model Prediction',
               'Actual Bee Activity', 'Bee Hour Calculation']
line_colors = ['green', 'blue', 'red']

for season in range(3):
    plt.cla()
    indices = []
    for i in sorted(pred_dates[season]):
        indices.append(i)

    indexed = [[] for i in range(num_lines)]

    for i in range(num_lines):
        for date in indices:
            indexed[i].append(line_values[i][season][date])

        indexed[i] = pd.DataFrame(indexed[i])
        indexed[i].index = indices
        indexed[i].columns = [line_labels[i]]
        sns.lineplot(data=indexed[i], palette=[line_colors[i]])

    # indexed3.columns = ['Model Prediction (Binary)']

    # fig, ax = plt.subplots(figsize=(12, 8))
    # sns.lineplot(data=indexed, ax=ax)
    # sns.lineplot(data=indexed2, ax=ax, palette=['red'])
    # ax.set_ylim(bottom=0, top=0.25)
    # ax.xaxis.set_ticks(indices)
    # ax.xaxis.set_ticklabels(indices)
    # ax.xaxis.set_ticks_position('bottom')
    # ax.yaxis.set_ticks_position('left')
    # plt.xticks(rotation=0)

    # sns.lineplot(data=indexed, palette=['green'])
    # sns.lineplot(data=indexed2, palette=['red'])
    # sns.lineplot(data=indexed3, palette=['blue'])
    # sns.lineplot(data=indexed4, palette=['orange'])

    # indexed.plot(ax=ax, legend=True)
    # indexed2.plot(ax=ax, legend=True, c='r')

    plt.xlabel('Date')
    plt.ylabel('Daily Bee Hours')
    plt.title(
        'Daily Bee Hours: Model Prediction vs. Bee Hour Calculation (' + seasons[season]+')')
    # plt.savefig(os.path.join(directory, 'Figures',
    #             'bee_hour_scaled_comp1_'+seasons[season]+'.jpg'), dpi=600, bbox_inches='tight')

    plt.show()


In [ ]:
sns.set_theme(rc={'figure.figsize': (10, 6)},
                    style='white', font='Arial', font_scale=1.1)

num_lines=3
line_values=[scaled_pred, scaled_actual, bee_hour]
line_labels=['Model Prediction', 'Actual Bee Activity', 'Bee Hour Calculation']
line_colors=['green', 'blue', 'red']

for season in range(3):
    plt.cla()
    indices = []
    for i in sorted(pred_dates[season]):
        indices.append(i)

    indexed=[[] for i in range(num_lines)]

    for i in range(num_lines):
        for date in indices:
            indexed[i].append(line_values[i][season][date])

        indexed[i] = pd.DataFrame(indexed[i])
        indexed[i].index = indices
        indexed[i].columns = [line_labels[i]]
        sns.lineplot(data=indexed[i], palette=[line_colors[i]])

    # indexed3.columns = ['Model Prediction (Binary)']

    # fig, ax = plt.subplots(figsize=(12, 8))
    # sns.lineplot(data=indexed, ax=ax)
    # sns.lineplot(data=indexed2, ax=ax, palette=['red'])
    # ax.set_ylim(bottom=0, top=0.25)
    # ax.xaxis.set_ticks(indices)
    # ax.xaxis.set_ticklabels(indices)
    # ax.xaxis.set_ticks_position('bottom')
    # ax.yaxis.set_ticks_position('left')
    # plt.xticks(rotation=0)

    # sns.lineplot(data=indexed, palette=['green'])
    # sns.lineplot(data=indexed2, palette=['red'])
    # sns.lineplot(data=indexed3, palette=['blue'])
    # sns.lineplot(data=indexed4, palette=['orange'])

    # indexed.plot(ax=ax, legend=True)
    # indexed2.plot(ax=ax, legend=True, c='r')

    plt.xlabel('Date')
    plt.ylabel('Daily Bee Hours')
    plt.title(
        'Daily Bee Hours: Model Prediction vs. Bee Hour Calculation (' + seasons[season]+')')
    # plt.savefig(os.path.join(directory, 'Figures',
    #             'bee_hour_scaled_comp1_'+seasons[season]+'.jpg'), dpi=600, bbox_inches='tight')

    plt.show()


In [ ]:
sns.set_theme(rc={'figure.figsize': (10, 6)},
                    style='white', font='Arial', font_scale=1.1)

line_values = [binary_pred, bee_hour, binary_actual]
line_labels=['Model Prediction (Binary)', 'Bee Hour Calculation', 'Actual Bee Activity (Binary)']
line_colors=['green', 'blue', 'red']
num_lines=len(line_values)

for season in range(3):
    plt.cla()
    indices = []
    for i in sorted(pred_dates[season]):
        indices.append(i)

    indexed=[[] for i in range(num_lines)]

    for i in range(num_lines):
        for date in indices:
            indexed[i].append(line_values[i][season][date])

        indexed[i] = pd.DataFrame(indexed[i])
        indexed[i].index = indices
        indexed[i].columns = [line_labels[i]]
        sns.lineplot(data=indexed[i], palette=[line_colors[i]])

    # indexed3.columns = ['Model Prediction (Binary)']

    # fig, ax = plt.subplots(figsize=(12, 8))
    # sns.lineplot(data=indexed, ax=ax)
    # sns.lineplot(data=indexed2, ax=ax, palette=['red'])
    # ax.set_ylim(bottom=0, top=0.25)
    # ax.xaxis.set_ticks(indices)
    # ax.xaxis.set_ticklabels(indices)
    # ax.xaxis.set_ticks_position('bottom')
    # ax.yaxis.set_ticks_position('left')
    # plt.xticks(rotation=0)

    # sns.lineplot(data=indexed, palette=['green'])
    # sns.lineplot(data=indexed2, palette=['red'])
    # sns.lineplot(data=indexed3, palette=['blue'])
    # sns.lineplot(data=indexed4, palette=['orange'])

    # indexed.plot(ax=ax, legend=True)
    # indexed2.plot(ax=ax, legend=True, c='r')

    plt.xlabel('Date')
    plt.ylabel('Daily Bee Hours')
    plt.title(
        'Daily Bee Hours: Model Prediction vs. Bee Hour Calculation (' + seasons[season]+')')
    # plt.savefig(os.path.join(directory, 'Figures',
    #             'bee_hour_scaled_comp1_'+seasons[season]+'.jpg'), dpi=600, bbox_inches='tight')

    plt.show()


In [ ]:
sorted(list(pred_dates[0]))

In [ ]:
for season in range(3):
    temp=pd.concat([pd.DataFrame(list(scaled_pred[season].values())), 
                    pd.DataFrame(list(scaled_actual[season].values())),
                                 pd.DataFrame(list(binary_pred[season].values())),
    pd.DataFrame(list(bee_hour[season].values())), 
    pd.DataFrame(list(binary_actual[season].values()))], axis=1)
    temp.columns=['Model Prediction', 'Actual Bee Activity',
                'Model Prediction (Binary)', 'Bee Hour Calculation', 'Actual Bee Activity (Binary)']
    temp.index=sorted(list(pred_dates[season]))
    temp.to_csv(os.path.join(directory, processed_folder, "prediction_"+seasons[season]+".csv"))


In [ ]:
temp.to_csv("/Users/vincewu/Bees-Project/binary_comparison.csv")

In [ ]:
print(scaled_pred[2][pd.to_datetime("2023-02-15")], scaled_actual[2][pd.to_datetime("2023-02-15")])

In [ ]:
print(scaled_pred[2][pd.to_datetime("2023-01-30")],
      scaled_actual[2][pd.to_datetime("2023-01-30")])


In [ ]:
scaled_pred[2][pd.to_datetime("2023-02-25")]

In [ ]:
scaled_actual[2][pd.to_datetime("2023-02-25")]


In [ ]:
sns.set_theme(rc={'figure.figsize': (10, 6)},
                    style='white', font='Arial', font_scale=1.1)

for season in range(3):
    plt.cla()
    indices = []
    for i in sorted(pred_dates[season]):
        indices.append(i)

    indexed = []
    indexed2 = []
    indexed3 = []
    for date in indices:
        indexed.append(scaled_pred[season][date])
        indexed2.append(bee_hour[season][date])
        indexed3.append(scaled_actual[season][date])

    indexed = pd.DataFrame(indexed)
    indexed.index = indices
    indexed.columns = ['Model Prediction (Scaled)']
    indexed2 = pd.DataFrame(indexed2)
    indexed2.index = indices
    indexed2.columns = ['Bee Hour Calculation (Binary)']
    indexed3 = pd.DataFrame(indexed3)
    indexed3.index = indices
    indexed3.columns = ['Actual Activity (Scaled)']

    # fig, ax = plt.subplots(figsize=(12, 8))
    # sns.lineplot(data=indexed, ax=ax)
    # sns.lineplot(data=indexed2, ax=ax, palette=['red'])
    # ax.set_ylim(bottom=0, top=0.25)
    # ax.xaxis.set_ticks(indices)
    # ax.xaxis.set_ticklabels(indices)
    # ax.xaxis.set_ticks_position('bottom')
    # ax.yaxis.set_ticks_position('left')
    # plt.xticks(rotation=0)

    sns.lineplot(data=indexed, palette=['green'])
    sns.lineplot(data=indexed2, palette=['red'])
    sns.lineplot(data=indexed3, palette=['blue'])

    # indexed.plot(ax=ax, legend=True)
    # indexed2.plot(ax=ax, legend=True, c='r')

    plt.xlabel('Date')
    plt.ylabel('Daily Bee Hours')
    plt.title(
        'Daily Bee Hours: Model Prediction vs. Bee Hour Calculation (' + seasons[season]+')')
    plt.savefig(os.path.join(directory, 'Figures',
                'bee_hour_scaled_'+seasons[season]+'.jpg'), dpi=600, bbox_inches='tight')

    plt.show()


In [ ]:
pd.concat([indexed, indexed2,indexed3], axis=1).to_csv(os.path.join(directory, '1Test.csv'))

In [ ]:
bee_np[0][:, 1]

# saved seasonal models

In [ ]:
optimal_features=[[0, 2, 3, 5, 6, 8], [0, 2, 3, 8], [0, 2, 3, 6, 8]]

time_steps = 9

for season in range(3):
    index = 0
    X = []

    weather_columns=optimal_features[season]

    for date in data_blocks[season]:
        for i in range(index, len(weather_data)):
            if weather_data.iloc[i].name == date-pd.Timedelta(hours=time_steps-1):
                temp = []
                # print('name', weather_data.iloc[i].name)
                for j in range(i, i+time_steps+24):
                    temp.append(
                        weather_data.iloc[j, weather_columns].values)
                for hour in range(24):
                    X.append(np.array(temp[hour:hour+time_steps]))
                index = i
                break
    X = np.array(X)
    Y = pd.DataFrame(bee_np[season][:, 1])

    model = Sequential([
        GRU(64, return_sequences=True,
            input_shape=(time_steps, len(weather_columns))),
        Dropout(0.2),
        GRU(64),
        Dropout(0.2),
        Dense(1)
    ])

    # (learning_rate=0.01)
    model.compile(metrics=['mse'], loss='mse', optimizer='adam')
    X_train = X
    Y_train = Y

    X_train = np.asarray(X_train).astype('float32')
    Y_train = np.asarray(Y_train).astype('float32')

    # scalers = []
    # for i in range(X_train.shape[2]):
    #     scalers.append(StandardScaler())
    #     X_train[:, :, i] = scalers[i].fit_transform(
    #         X_train[:, :, i])

    model.fit(X_train, Y_train, epochs=50,
            batch_size=16, verbose=1)
    model.save(os.path.join(directory, "Final Models", "model_"+seasons[season]))

# one day hourly prediction

In [ ]:
data_blocks[2]

In [ ]:
#days 3/11 and after excluded from graph, so test all

season=2
temp_data_block = [[],[]]
index=0
for date in data_blocks[season]:
    if date==pd.to_datetime("2023-03-11"):
        index+=1
    temp_data_block[index].append(date)

# weather_columns_full = [[0, 2, 3, 6, 8], [0, 2, 3, 5, 8], [0, 2, 3, 5, 8]]
optimal_features=[[0, 2, 3, 5, 6, 8], [0, 2, 3, 8], [0, 2, 3, 6, 8]]
# bee_hour2 = [{} for i in range(3)]
time_steps=9

X_train = []

index = 0
weather_columns=optimal_features[season]

for date in temp_data_block[0]:
    for i in range(index, len(weather_data)):
        if weather_data.iloc[i].name == date-pd.Timedelta(hours=time_steps-1):
            temp = []
            # print('name', weather_data.iloc[i].name)
            for j in range(i, i+time_steps+24):
                # print(j, date, j-i-time_steps)
                temp.append(
                    weather_data.iloc[j, :-1].values)
            for hour in range(24):
                X_train.append(np.array(temp[hour:hour+time_steps]))
            index = i
            break
X_train = np.array(X_train)
Y_train = pd.DataFrame(bee_np[season][:, 1]).iloc[:X_train.shape[0]]

X_test = []
for date in temp_data_block[1]:
    for i in range(index, len(weather_data)):
        if weather_data.iloc[i].name == date-pd.Timedelta(hours=time_steps-1):
            temp = []
            # print('name', weather_data.iloc[i].name)
            for j in range(i, i+time_steps+24):
                # print(j, date, j-i-time_steps)
                temp.append(
                    weather_data.iloc[j, :-1].values)
            for hour in range(24):
                X_test.append(np.array(temp[hour:hour+time_steps]))
            index = i
            break
X_test = np.array(X_test)
Y_test = pd.DataFrame(bee_np[season][:, 1]).iloc[X_train.shape[0]:]

model = Sequential([
    GRU(64, return_sequences=True,
        input_shape=(time_steps, len(weather_columns))),
    Dropout(0.2),
    GRU(64),
    Dropout(0.2),
    Dense(1)
])
# (learning_rate=0.01)
model.compile(metrics=['mse'], loss='mse', optimizer='adam')

X_train = X_train[:, :, [weather_columns]].squeeze()
X_test = X_test[:, :, [weather_columns]].squeeze()

X_train = np.asarray(X_train).astype('float32')
X_test = np.asarray(X_test).astype('float32')
Y_train = np.asarray(Y_train).astype('float32')
Y_test = np.asarray(Y_test).astype('float32')

X_train_scaled = np.asarray(X_train).astype('float32')
X_test_scaled = np.asarray(X_test).astype('float32')

scalers = []
for i in range(X_train.shape[2]):
    scalers.append(StandardScaler())
    X_train_scaled[:, :, i] = scalers[i].fit_transform(
        X_train_scaled[:, :, i])
for i in range(X_test.shape[2]):
    X_test_scaled[:, :, i] = scalers[i].transform(X_test_scaled[:, :, i])

model.fit(X_train_scaled, Y_train, epochs=50,
        batch_size=16, verbose=0)
# model.fit(X_train, Y_train, epochs=40,
#         batch_size=128, verbose=0)

y_pred = model.predict(X_test_scaled).reshape(-1, 1)
for i in range(len(y_pred)):
    y_pred[i, 0]=max(y_pred[i, 0], 0)

r2 = r2_score(Y_test, y_pred)
print(r2)


In [ ]:
for i in range(0, len(y_pred)-1, 24):
    print(r2_score(Y_test[i: i+24], y_pred[i: i+24]))


In [ ]:
for i in range(0, len(y_pred)-24, 24):
    print(r2_score(Y_test[i: i+24], y_pred[i: i+24]))

In [ ]:

r2_score(Y_test[2*24: 3*24], y_pred[2*24: 3*24])

In [ ]:

r2_score(Y_test[4*24: 5*24], y_pred[4*24: 5*24])

In [ ]:
temp_data_block[1]

In [ ]:
range(24)

In [ ]:
sns.set_theme(rc={'figure.figsize': (15, 6)},
              style='white', font='Arial', font_scale=1.1)

date_str="3/25/23"

num_lines = 3
line_values = [Y_test[3*24: 6*24], y_pred[3*24: 6*24]]
line_labels = ['Measured Bee Activity', 'Model Prediction']
line_colors = ['green', 'blue']

for i in range(2):
    line_values[i]=pd.DataFrame(line_values[i])
    line_values[i].columns = [line_labels[i]]
    sns.lineplot(data=line_values[i], palette=[line_colors[i]])
plt.xlabel('Hour')
plt.xticks(range(0, 73, 4))
plt.ylabel('Percentage of Foragers Outside')
plt.title(
    'Measured Bee Activity vs. Model Prediction (3/24/23-3/26/23)')
plt.savefig(os.path.join(directory, 'Figures',
            'hourly_predictions.jpg'), dpi=600, bbox_inches='tight')

plt.show()


In [ ]:
temp=pd.concat([pd.DataFrame(Y_test[3*24: 6*24]), pd.DataFrame(y_pred[3*24: 6*24])], axis=1)
temp.columns=['Measured Bee Activity', 'Model Prediction']
temp.to_csv(os.path.join(directory, processed_folder,
            'hourly_predictions.csv'))

In [ ]:
sns.set_theme(rc={'figure.figsize': (10, 6)},
                    style='white', font='Arial', font_scale=1.1)

num_lines=3
line_values=[Y_test[2*24: 3*24], y_pred[2*24: 3*24]]
line_labels=['Model Prediction', 'Actual Bee Activity']
line_colors=['green', 'blue']

for i in range(2):
    sns.lineplot(data=line_values[i], palette=[line_colors[i]])
plt.xlabel('Date')
plt.ylabel('Daily Bee Hours')
plt.title(
    'Daily Bee Hours: Model Prediction vs. Bee Hour Calculation (' + seasons[season]+')')
# plt.savefig(os.path.join(directory, 'Figures',
#             'bee_hour_scaled_comp1_'+seasons[season]+'.jpg'), dpi=600, bbox_inches='tight')

plt.show()


# Future prediction

In [ ]:
#6am-6pm
march_7=[0, 32, 247, 435, 643, 812, 854, 883, 873, 787, 588, 313, 105]

In [ ]:
months_num = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
weather_headers = ['Temp.', 'Dew Point', 'Humidity', 'Speed',
                   'Gust', 'Pressure', 'Rainfall', 'UV', 'Solar']

weather_file = 'weather_forecast_clean.csv'
weather_forecast = pd.read_csv(os.path.join(directory, processed_folder, weather_file),
                           parse_dates=['Datetime'], engine='python')
weather_forecast = weather_forecast.resample('1h', on='Datetime').mean()
# weather_forecast.columns = weather_headers
weather_forecast.insert(len(weather_forecast.columns),
                    'Datetime_col', weather_forecast.index.tolist())
weather_forecast


In [ ]:
weather_forecast[7463:]

In [ ]:
X.shape

In [ ]:
weather_columns = [0, 2, 3, 5, 6, 8]
# weather_columns=[0, 2, 8]
X = []
model_type = 1
season = 2
time_steps = 9
index = 0

for date in data_blocks[season]:
    for i in range(index, len(weather_data)):
        if weather_data.iloc[i].name == date-pd.Timedelta(hours=time_steps-1):
            temp = []
            # print('name', weather_data.iloc[i].name)
            for j in range(i, i+time_steps+24):
                temp.append(
                    weather_data.iloc[j, weather_columns].values)
            for hour in range(24):
                X.append(np.array(temp[hour:hour+time_steps]))
            index = i
            break
X = np.array(X)
Y = pd.DataFrame(bee_np[season][:, 1])

# drop_combinations = []

# for i in range(len(leave_out[season])+1):
#     for j in itertools.combinations(leave_out[season], i):
#         temp = []
#         for k in j:
#             temp.append(k)
#         drop_combinations.append(temp)

# printdrop_combinations)

model = Sequential([
    GRU(64, activation='relu', return_sequences=True,
        input_shape=(time_steps, len(weather_columns))),
    Dropout(0.2),
    GRU(64, activation='relu',
        input_shape=(time_steps, len(weather_columns))),
    Dropout(0.2),
    Dense(1)
])

# (learning_rate=0.01)
model.compile(metrics=['mse'], loss='mse', optimizer='adam')

X_train = X
Y_train = Y

X_train = np.asarray(X_train).astype('float32')
Y_train = np.asarray(Y_train).astype('float32')

scalers = []
for i in range(X_train.shape[2]):
    scalers.append(StandardScaler())
    X_train[:, :, i] = scalers[i].fit_transform(
        X_train[:, :, i])

model.fit(X_train, Y_train, epochs=50,
            batch_size=16, verbose=1)

In [ ]:
np.array([weather_forecast.iloc[7455:, [
                       0, 2, 3, 5, 6, 8]].to_numpy()]).shape


In [ ]:
X_test=np.array([weather_forecast.iloc[7455-(23-i):7455-(23-i)+9, [
    0, 2, 3, 5, 6, 8]].to_numpy() for i in range(24)])
for i in range(X_test.shape[2]):
    X_test[:, :, i] = scalers[i].transform(X_test[:, :, i])

y_pred = model.predict(X_test).reshape(-1, 1)


In [ ]:
y_pred

In [ ]:
plt.cla()

In [ ]:
forecast_y_pred=y_pred

In [ ]:
forecast_y_pred


In [ ]:
fig, ax=plt.subplots(figsize=(8, 8))
# sns.lineplot(data=pd.DataFrame(forecast_y_pred))
pd.DataFrame(forecast_y_pred).plot(ax=ax, legend=False)
plt.xticks([i for i in range(0, 24, 2)])
# ax.set_xticks([i for i in range(0, 24, 2)])
# ax.set_xticklabels([i for i in range(0, 24, 2)])
# ax.spines['left'].set_visible(True)
# ax.xaxspines['bottom'].set_visible(True)
plt.xlabel('Hour')
plt.ylabel('Percent of Bees Outside')
plt.title('Hourly Model Predictions for 03/07/2023')
plt.savefig(os.path.join(directory, 'Figures',
            'forecasted_activity.jpg'), dpi=600, bbox_inches='tight')

plt.show()


In [ ]:

y_pred = model.predict(X_test).reshape(-1, 1)

In [ ]:
plot_df[5:20]

In [ ]:
plot_df


In [ ]:
mse

In [ ]:
r2

In [ ]:
df[1].mean()

In [ ]:
saved_df=df

In [ ]:
len(holdon)

In [ ]:
X[[holdon]].shape

In [ ]:
season=2
subset_metrics.append([])
#k-fold cross validation
# neither precip nor barometric
full_weather_columns = [[0, 2, 3, 5, 6, 8], [
    0, 2, 3, 5, 8], [0, 2, 3, 6, 8], [0, 2, 3, 8]]
column_headers = [['LSTM-6', 'GRU-6'], ['LSTM-4+P',
                                        'GRU-4+P'], ['LSTM-4+R', 'GRU-4+R'], ['LSTM-4', 'GRU-4']]

for i in range(len(full_weather_columns)):
    weather_columns = full_weather_columns[i]
    df = []
    model_gru = Sequential([
        GRU(16, activation='relu', return_sequences=True,
            input_shape=(1, len(weather_columns))),
        Dropout(0.15),
        GRU(16, activation='relu', return_sequences=True,
            input_shape=(1, len(weather_columns))),
        Dropout(0.1),
        Dense(1)
    ])

    model_gru.compile(metrics=['mse'], loss='mse')

    model_lstm = Sequential([
        LSTM(16, activation='relu', return_sequences=True,
                input_shape=(1, len(weather_columns))),
        Dropout(0.15),
        LSTM(16, activation='relu', return_sequences=True,
                input_shape=(1, len(weather_columns))),
        Dropout(0.1),
        Dense(1)
    ])

    model_lstm.compile(metrics=['mse'], loss='mse')

    X = pd.DataFrame(weather_np[season][:, weather_columns])
    Y = pd.DataFrame(bee_np[season][:, 1])

    # model_mse = []
    # model_r2 = []
    kf = KFold(n_splits=tests)
    for train_index, test_index in kf.split(X):
        # X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = i+1)
        # print(train_index)
        # print(test_index)

        X_train = X.iloc[train_index]
        X_test = X.iloc[test_index]
        Y_train = Y.iloc[train_index]
        Y_test = Y.iloc[test_index]

        X_train = np.asarray(X_train).astype('float32')
        X_test = np.asarray(X_test).astype('float32')
        Y_train = np.asarray(Y_train).astype('float32')
        Y_test = np.asarray(Y_test).astype('float32')

        scaler = StandardScaler()
        X_train_scale = scaler.fit_transform(X_train)
        X_test_scale = scaler.transform(X_test)

        X_train_scale = X_train_scale.reshape(-1, 1, len(weather_columns))
        X_test_scale = X_test_scale.reshape(-1, 1, len(weather_columns))

        row = []

        model_gru.fit(X_train_scale, Y_train, epochs=3,
                        batch_size=16, verbose=0)
        model_lstm.fit(X_train_scale, Y_train, epochs=3,
                        batch_size=16, verbose=0)

        y_pred = model_lstm.predict(X_test_scale).reshape(-1, 1)
        mse = mean_squared_error(Y_test, y_pred)
        # model_mse.append(mse)
        row.append(mse)
        y_pred = model_gru.predict(X_test_scale).reshape(-1, 1)
        mse = mean_squared_error(Y_test, y_pred)
        # model_mse.append(mse)
        row.append(mse)

        df.append(row)

    subset_metrics[-1].append(pd.DataFrame(df, columns=column_headers[i]))
    # r2 = r2_score(Y_test, y_pred)
    # model_r2.append(r2)


# gru_mse_1=pd.DataFrame(model_mse)
# gru_r2_1=pd.DataFrame(model_r2)
# print(gbr_mse, gbr_mse.mean(), gbr_r2, gbr_r2.mean(), sep='\n')


# final figure generation

In [ ]:
len(bee_np[0])

In [ ]:
# scatterplot
#k-fold cross validation
weather_columns = [0, 2, 3, 5, 6, 8]

for i in range(len(full_weather_columns)):
    weather_columns = full_weather_columns[i]
    df = []
    model_gru = Sequential([
        GRU(16, activation='relu', return_sequences=True,
            input_shape=(1, len(weather_columns))),
        Dropout(0.15),
        GRU(16, activation='relu', return_sequences=True,
            input_shape=(1, len(weather_columns))),
        Dropout(0.1),
        Dense(1)
    ])

    model_gru.compile(metrics=['mse'], loss='mse')

    model_lstm = Sequential([
        LSTM(16, activation='relu', return_sequences=True,
             input_shape=(1, len(weather_columns))),
        Dropout(0.15),
        LSTM(16, activation='relu', return_sequences=True,
             input_shape=(1, len(weather_columns))),
        Dropout(0.1),
        Dense(1)
    ])

    model_lstm.compile(metrics=['mse'], loss='mse')

    X = pd.DataFrame(weather_np[season][:, weather_columns])
    Y = pd.DataFrame(bee_np[season][:, 1])

    # model_mse = []
    # model_r2 = []
    kf = KFold(n_splits=tests)
    for train_index, test_index in kf.split(X):
        # X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = i+1)
        # print(train_index)
        # print(test_index)

        X_train = X.iloc[train_index]
        X_test = X.iloc[test_index]
        Y_train = Y.iloc[train_index]
        Y_test = Y.iloc[test_index]

        X_train = np.asarray(X_train).astype('float32')
        X_test = np.asarray(X_test).astype('float32')
        Y_train = np.asarray(Y_train).astype('float32')
        Y_test = np.asarray(Y_test).astype('float32')

        scaler = StandardScaler()
        X_train_scale = scaler.fit_transform(X_train)
        X_test_scale = scaler.transform(X_test)

        X_train_scale = X_train_scale.reshape(-1, 1, len(weather_columns))
        X_test_scale = X_test_scale.reshape(-1, 1, len(weather_columns))

        row = []

        model_gru.fit(X_train_scale, Y_train, epochs=3,
                      batch_size=16, verbose=0)
        model_lstm.fit(X_train_scale, Y_train, epochs=3,
                       batch_size=16, verbose=0)

        y_pred = model_lstm.predict(X_test_scale).reshape(-1, 1)
        mse = mean_squared_error(Y_test, y_pred)
        # model_mse.append(mse)
        row.append(mse)
        y_pred = model_gru.predict(X_test_scale).reshape(-1, 1)
        mse = mean_squared_error(Y_test, y_pred)
        # model_mse.append(mse)
        row.append(mse)

        df.append(row)

    subset_metrics[-1].append(pd.DataFrame(df, columns=column_headers[i]))

In [ ]:
# bee hour predictions

In [ ]:
sns.boxplot(data=pd.concat(
    subset_metrics[0], axis=1), color='yellow', medianprops={"color": "black"})
plt.title('Model Accuracies on Feature Subsets (Summer)')
plt.ylabel('Model')
plt.ylabel('MSE')
# plt.xticks(rotation=90)
plt.savefig(os.path.join(directory, 'Figures',
            'features_summer.jpg'), dpi=600, bbox_inches='tight')
plt.show()


In [ ]:
sns.boxplot(data=pd.concat(
    subset_metrics[1], axis=1), color='yellow', medianprops={"color": "black"})
plt.title('Model Accuracies on Feature Subsets (Fall)')
plt.ylabel('Model')
plt.ylabel('MSE')
# plt.xticks(rotation=90)
plt.savefig(os.path.join(directory, 'Figures',
            'features_fall.jpg'), dpi=600, bbox_inches='tight')
plt.show()


In [ ]:
sns.boxplot(data=pd.concat(
    subset_metrics[-2], axis=1), color='yellow', medianprops={"color": "black"}, showfliers=False)
plt.title('Model Accuracies on Feature Subsets (Early Spring)')
plt.ylabel('Model')
plt.ylabel('MSE')
# plt.xticks(rotation=90)
plt.savefig(os.path.join(directory, 'Figures',
            'features_spring_no_rain.jpg'), dpi=600, bbox_inches='tight')
plt.show()


In [ ]:
sns.boxplot(data=pd.concat(
    subset_metrics[-1], axis=1), color='yellow', medianprops={"color": "black"}, showfliers=False)
plt.title('Model Accuracies on Feature Subsets (Early Spring)')
plt.ylabel('Model')
plt.ylabel('MSE')
# plt.xticks(rotation=90)
plt.savefig(os.path.join(directory, 'Figures',
            'features_spring_with_rain.jpg'), dpi=600, bbox_inches='tight')
plt.show()


In [ ]:
len(subset_metrics)

In [ ]:
df=pd.concat(subset_metrics[-2], axis=1)
for i in df.columns:
    print(df[i].mean())

In [ ]:
df=pd.concat(subset_metrics[-1], axis=1)
for i in df.columns:
    print(df[i].mean())

In [ ]:
sns.boxplot(data=pd.concat(
    subset_metrics[2], axis=1), color='yellow', medianprops={"color": "black"})
plt.title('Model Accuracies on Feature Subsets (Early Spring)')
plt.ylabel('Model')
plt.ylabel('MSE')
# plt.xticks(rotation=90)
plt.savefig(os.path.join(directory, 'Figures',
            'features_spring_with_rain.jpg'), dpi=600, bbox_inches='tight')
plt.show()


In [ ]:
fig
sns.boxplot(data=boxplot_df, color='yellow', medianprops={"color": "black"})
plt.ylabel('MSE')
# plt.xticks(rotation=90)
plt.show()

In [ ]:
season=0
weather_columns = [0, 2, 3, 5, 6, 8]

X = pd.DataFrame(weather_np[season][:, weather_columns])
Y = pd.DataFrame(bee_np[season][:, 1])

Y_train=Y.astype('float32')
scaler = StandardScaler()
X_scale = scaler.fit_transform(X).astype('float32')
# X_train_scale = X_train
# X_test_scale = X_test

# X_train_single = X_train_scale.reshape(-1, 1, 6)
# X_test_single = X_test_scale.reshape(-1,1,6)
X_train_single = X_scale.reshape(-1, 1, 6)

model_lstm = Sequential([
    LSTM(16, activation='relu', return_sequences=True, input_shape=(1, 6)),
    Dropout(0.15),
    LSTM(16, activation='relu', return_sequences=True, input_shape=(1, 6)),
    Dropout(0.1),
    Dense(1)
])

model_lstm.compile(metrics=['mse'], loss='mse')

n_epochs = 25
history = model_lstm.fit(X_train_single, Y_train, epochs=n_epochs, batch_size=16, verbose=1)


In [ ]:
weather_data

In [ ]:
weather_np[3][10][6:9]

In [ ]:
pd.to_datetime(str(weather_np[3][0][-1])[:10])

# Models

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
season=3

X = pd.DataFrame(weather_np[season][:, weather_columns])
Y = pd.DataFrame(bee_np[season][:, 1])
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 0)

scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)

model_gbr = GradientBoostingRegressor()

model_gbr.fit(X_train_scale, Y_train)
y_pred = model_gbr.predict(X_test_scale)
mse = mean_squared_error(Y_test, y_pred)
print('mse: '+str(mse))
print('R2 test data: ', model_gbr.score(X_test_scale, Y_test))

pipe = Pipeline([('scaler', scaler), ('model', model_gbr)])
cvs=cross_val_score(pipe, X, Y, cv=10)
print('R2 cross-validated: ', cvs, cvs.mean())


In [ ]:
from sklearn.ensemble import RandomForestRegressor

season=0

X = pd.DataFrame(weather_np[season][:, weather_columns])
# Y = pd.DataFrame(bee_np[season], columns=['Minutes', 'Bees', 'Egress', 'Ingress', 'Both', 'Datetime'])
Y = pd.DataFrame(bee_np[season][:, 1])
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=0)

scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)

model_rfr = RandomForestRegressor()

model_rfr.fit(X_train_scale, Y_train)
y_pred = model_rfr.predict(X_test_scale)
mse = mean_squared_error(Y_test, y_pred)
print('mse: '+str(mse))
print('R2 test data: ', model_rfr.score(X_test_scale, Y_test))

pipe = Pipeline([('scaler', scaler), ('model', model_rfr)])
print('R2 cross-validated: ', cross_val_score(pipe, X, Y, cv=10),
      cross_val_score(pipe, X, Y, cv=10).mean())


In [ ]:
cnt=0

for date in data_blocks[2]:
    cnt+=arr[date]['Trips Taken']

cnt

In [ ]:
from sklearn.ensemble import RandomForestRegressor

season=0

X = pd.DataFrame(weather_np[season][:, 8])
# Y = pd.DataFrame(bee_np[season], columns=['Minutes', 'Bees', 'Egress', 'Ingress', 'Both', 'Datetime'])
Y = pd.DataFrame(bee_np[season][:, 1])
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=0)

scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)

model_rfr = RandomForestRegressor()

model_rfr.fit(X_train_scale, Y_train)
y_pred = model_rfr.predict(X_test_scale)
mse = mean_squared_error(Y_test, y_pred)
print('mse: '+str(mse))
print('R2 test data: ', model_rfr.score(X_test_scale, Y_test))

pipe = Pipeline([('scaler', scaler), ('model', model_rfr)])
print('R2 cross-validated: ', cross_val_score(pipe, X, Y, cv=10),
      cross_val_score(pipe, X, Y, cv=10).mean())


In [ ]:
from sklearn.ensemble import RandomForestRegressor

season = 1

X = pd.DataFrame(weather_np[season][:, weather_columns])
# Y = pd.DataFrame(bee_np[season], columns=['Minutes', 'Bees', 'Egress', 'Ingress', 'Both', 'Datetime'])
Y = pd.DataFrame(bee_np[season][:, 1])
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=0)

scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)

model_rfr = RandomForestRegressor()

model_rfr.fit(X_train_scale, Y_train)
y_pred = model_rfr.predict(X_test_scale)
mse = mean_squared_error(Y_test, y_pred)
print('mse: '+str(mse))
print('R2 test data: ', model_rfr.score(X_test_scale, Y_test))

pipe = Pipeline([('scaler', scaler), ('model', model_rfr)])
print('R2 cross-validated: ', cross_val_score(pipe, X, Y, cv=10),
      cross_val_score(pipe, X, Y, cv=10).mean())


In [ ]:
from sklearn.ensemble import RandomForestRegressor

season=2

X = pd.DataFrame(weather_np[season][:, 3])
# Y = pd.DataFrame(bee_np[season], columns=['Minutes', 'Bees', 'Egress', 'Ingress', 'Both', 'Datetime'])
Y = pd.DataFrame(bee_np[season][:, 1])
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=0)

scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)

model_rfr = RandomForestRegressor()

model_rfr.fit(X_train_scale, Y_train)
y_pred = model_rfr.predict(X_test_scale)
mse = mean_squared_error(Y_test, y_pred)
print('mse: '+str(mse))
print('R2 test data: ', model_rfr.score(X_test_scale, Y_test))

pipe = Pipeline([('scaler', scaler), ('model', model_rfr)])
print('R2 cross-validated: ', cross_val_score(pipe, X, Y, cv=10),
      cross_val_score(pipe, X, Y, cv=10).mean())


In [ ]:
from sklearn.ensemble import RandomForestRegressor

season=3

X = pd.DataFrame(weather_np[season][:, weather_columns])
# Y = pd.DataFrame(bee_np[season], columns=['Minutes', 'Bees', 'Egress', 'Ingress', 'Both', 'Datetime'])
Y = pd.DataFrame(bee_np[season][:, 1])
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=0)

scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)

model_rfr = RandomForestRegressor()

model_rfr.fit(X_train_scale, Y_train)
y_pred = model_rfr.predict(X_test_scale)
mse = mean_squared_error(Y_test, y_pred)
print('mse: '+str(mse))
print('R2 test data: ', model_rfr.score(X_test_scale, Y_test))

pipe = Pipeline([('scaler', scaler), ('model', model_rfr)])
print('R2 cross-validated: ', cross_val_score(pipe, X, Y, cv=10),
      cross_val_score(pipe, X, Y, cv=10).mean())


In [ ]:
X.iloc[[0, 1, 2]]

In [ ]:
#k-fold cross validation

model_mse=[]
model_r2=[]
kf = KFold(n_splits=tests)
for train_index, test_index in kf.split(X):
    # X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = i+1)
    # print(train_index)
    # print(test_index)

    X_train = X.iloc[train_index]
    X_test = X.iloc[test_index]
    Y_train = Y.iloc[train_index]
    Y_test = Y.iloc[test_index]

    scaler = StandardScaler()
    X_train_scale = scaler.fit_transform(X_train)
    X_test_scale = scaler.transform(X_test)

    model_rfr.fit(X_train_scale, Y_train)
    y_pred = model_rfr.predict(X_test_scale)
    
    mse=mean_squared_error(Y_test, y_pred)
    model_mse.append(mse)

    r2=r2_score(Y_test, y_pred)
    model_r2.append(r2)

rfr_mse_1=pd.DataFrame(model_mse)
rfr_r2_2=pd.DataFrame(model_r2)
# print(gbr_mse, gbr_mse.mean(), gbr_r2, gbr_r2.mean(), sep='\n')


In [ ]:
rfr_r2_2.mean()

In [ ]:
rfr_r2_2.mean()

In [ ]:
rfr_r2_2


In [ ]:
rfr_r2_2


In [ ]:
rfr_r2_2


In [ ]:
#30 iterations
#15
model_mse = []
model_r2 = []

season=2

# for i in range(0, len(bee_np[season])-initial_hours-24, 24):
for i in range(0, len(bee_np[season])-24, 24):
    # X_train = X[:initial_hours+i]
    # Y_train = Y[:initial_hours+i]
    # X_test = X[initial_hours+i:initial_hours+i+24]
    # Y_test = Y[initial_hours+i:initial_hours+i+24]
    X_train = X.drop(index=[x for x in range(i, i+24)])
    Y_train = Y.drop(index=[x for x in range(i, i+24)])
    X_test = X[i:i+24]
    Y_test = Y[i:i+24]

    scaler = StandardScaler()
    X_train_single = scaler.fit_transform(X_train)
    X_test_single = scaler.transform(X_test)

    model_rfr = RandomForestRegressor()

    model_rfr.fit(X_train_single, Y_train)

    y_pred = model_rfr.predict(X_test_single)

    mse = mean_squared_error(Y_test, y_pred)
    # model_mse.append([weather_np[season][:, -1][initial_hours+i], mse])
    model_mse.append([weather_np[season][:, -1][i], mse])

    r2 = r2_score(Y_test, y_pred)
    # model_r2.append([bee_np[season][:, -1][initial_hours+i], r2])
    model_r2.append([weather_np[season][:, -1][i], r2])

model_r2=pd.DataFrame(model_r2)
model_mse=pd.DataFrame(model_mse)


In [ ]:
print(model_r2.iloc[:, 1].mean())
for i in range(len(model_r2.iloc[:, 1])):
    print(i, model_r2.iloc[i, 0], model_r2.iloc[i, 1])


In [ ]:
season = 0

X = pd.DataFrame(weather_np[season][:, weather_columns])
Y = pd.DataFrame(bee_np[season][:, 1], columns=['T'])

# X = pd.DataFrame(weather_np[:, weather_columns])
# Y = pd.DataFrame(bee_np, columns=['Y1', "Y2"])
# X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 0)

# X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 0)
X_train = X[int(len(X)*0.8):]
X_test = X[:len(X)-int(len(X)*0.8)]
Y_train = Y[int(len(Y)*0.8):]
Y_test = Y[:len(Y)-int(len(X)*0.8)]

scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)


In [ ]:
season = 3

X = pd.DataFrame(weather_np[season][:, weather_columns])
Y = pd.DataFrame(bee_np[season][:, 1])

# X = pd.DataFrame(weather_np[:, weather_columns])
# Y = pd.DataFrame(bee_np, columns=['Y1', "Y2"])
# X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 0)

# X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 0)
X_train = X[int(len(X)*0.8):]
X_test = X[:len(X)-int(len(X)*0.8)]
Y_train = Y[int(len(Y)*0.8):]
Y_test = Y[:len(Y)-int(len(X)*0.8)]

X_train = np.asarray(X_train).astype('float32')
X_test = np.asarray(X_test).astype('float32')
Y_train = np.asarray(Y_train).astype('float32')
Y_test = np.asarray(Y_test).astype('float32')

scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)
# X_train_scale = X_train
# X_test_scale = X_test

# X_train_single = X_train_scale.reshape(-1, 1, 6)
# X_test_single = X_test_scale.reshape(-1,1,6)
X_train_single = X_train_scale.reshape(-1, 1, 6)
X_test_single = X_test_scale.reshape(-1, 1, 6)

# only predict 1 variable
model_lstm = Sequential([
    LSTM(16, activation='relu', return_sequences=True, input_shape=(1, 6)),
    Dropout(0.15),
    LSTM(16, activation='relu', return_sequences=True, input_shape=(1, 6)),
    Dropout(0.1),
    Dense(1)
])

model_lstm.compile(metrics=['mse'], loss='mse')

model_lstm.summary()

n_epochs = 25
history = model_lstm.fit(X_train_single, Y_train, epochs=n_epochs, batch_size=16, validation_data=(
    X_test_single, Y_test), verbose=1)

fig, ax = plt.subplots()

pd.DataFrame(history.history['loss'], index=list(
    range(n_epochs)), columns=['Training Loss']).plot(ax=ax)
pd.DataFrame(history.history['val_loss'], index=list(
    range(n_epochs)), columns=['Validation Loss']).plot(ax=ax, c='r')

print(history.history['val_loss'][-1])


In [ ]:
weather_columns


In [ ]:
model_r2.mean()

In [ ]:
model_mse.mean()


In [ ]:
season = 0

X = pd.DataFrame(weather_np[season][:, weather_columns])
Y = pd.DataFrame(bee_np[season][:, 1])

# X = pd.DataFrame(weather_np[:, weather_columns])
# Y = pd.DataFrame(bee_np, columns=['Y1', "Y2"])
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=0)

# X = pd.DataFrame(weather_np[:, weather_columns])
# Y = pd.DataFrame(bee_np, columns=['Y1', "Y2"])
# X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 0)

# X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 0)
# X_train = X[int(len(X)*0.8):]
# X_test = X[:len(X)-int(len(X)*0.8)]
# Y_train = Y[int(len(Y)*0.8):]
# Y_test = Y[:len(Y)-int(len(X)*0.8)]

X_train = np.asarray(X_train).astype('float32')
X_test = np.asarray(X_test).astype('float32')
Y_train = np.asarray(Y_train).astype('float32')
Y_test = np.asarray(Y_test).astype('float32')

scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)
# X_train_scale = X_train
# X_test_scale = X_test

# X_train_single = X_train_scale.reshape(-1, 1, 6)
# X_test_single = X_test_scale.reshape(-1,1,6)
X_train_single = X_train_scale.reshape(-1, 1, 6)
X_test_single = X_test_scale.reshape(-1, 1, 6)

# only predict 1 variable
model_lstm = Sequential([
    LSTM(16, activation='relu', return_sequences=True, input_shape=(1, 6)),
    Dropout(0.15),
    LSTM(16, activation='relu', return_sequences=True, input_shape=(1, 6)),
    Dropout(0.1),
    Dense(1)
])

model_lstm.compile(metrics=['mse'], loss='mse', optimizer='Adam')

model_lstm.summary()

n_epochs = 20
history = model_lstm.fit(X_train_single, Y_train, epochs=n_epochs, batch_size=16, validation_data=(
    X_test_single, Y_test), verbose=1)

fig, ax = plt.subplots()

pd.DataFrame(history.history['loss'], index=list(
    range(n_epochs)), columns=['Training Loss']).plot(ax=ax)
pd.DataFrame(history.history['val_loss'], index=list(
    range(n_epochs)), columns=['Validation Loss']).plot(ax=ax, c='r')

print(history.history['val_loss'][-1])


In [ ]:
#k-fold cross validation

model_mse=[]
model_r2=[]
kf = KFold(n_splits=tests)
for train_index, test_index in kf.split(X):
    # X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = i+1)
    # print(train_index)
    # print(test_index)

    X_train = X.iloc[train_index]
    X_test = X.iloc[test_index]
    Y_train = Y.iloc[train_index]
    Y_test = Y.iloc[test_index]

    X_train = np.asarray(X_train).astype('float32')
    X_test = np.asarray(X_test).astype('float32')
    Y_train = np.asarray(Y_train).astype('float32')
    Y_test = np.asarray(Y_test).astype('float32')

    scaler = StandardScaler()
    X_train_scale = scaler.fit_transform(X_train)
    X_test_scale = scaler.transform(X_test)

    X_train_scale = X_train_scale.reshape(-1, 1, 6)
    X_test_scale = X_test_scale.reshape(-1, 1, 6)

    model_lstm.fit(X_train_scale, Y_train, epochs=3, batch_size=16, verbose=0)
    y_pred = model_lstm.predict(X_test_scale).reshape(-1, 1)
    
    mse=mean_squared_error(Y_test, y_pred)
    model_mse.append(mse)

    r2=r2_score(Y_test, y_pred)
    model_r2.append(r2)

lstm_mse_1=pd.DataFrame(model_mse)
lstm_r2_1=pd.DataFrame(model_r2)
# print(gbr_mse, gbr_mse.mean(), gbr_r2, gbr_r2.mean(), sep='\n')

In [ ]:
lstm_r2_1.mean()


In [ ]:
lstm_r2_1.mean()


In [ ]:
lstm_r2_1.mean()


In [ ]:
lstm_r2_1.mean()

In [ ]:
lstm_r2_1.mean()


In [ ]:
lstm_r2_1.mean()

In [ ]:
lstm_r2_1.mean()

In [ ]:
lstm_r2_1.mean()


In [ ]:
lstm_r2_1.mean()

In [ ]:
lstm_r2_1


In [ ]:
#walk-forward cv
#30 iterations
#15
model_mse = []
model_r2 = []

season = 3

initial_hours = 7*24

# for i in range(0, len(bee_np)+1-initial_hours-24, 24*8):
for i in range(0, len(bee_np[season])-initial_hours-24, 24*7):
    X_train = X[:initial_hours+i]
    Y_train = Y[:initial_hours+i]
    X_test = X[initial_hours+i:initial_hours+i+24]
    Y_test = Y[initial_hours+i:initial_hours+i+24]

    X_train = np.asarray(X_train).astype('float32')
    X_test = np.asarray(X_test).astype('float32')
    Y_train = np.asarray(Y_train).astype('float32')
    Y_test = np.asarray(Y_test).astype('float32')

    scaler = StandardScaler()
    X_train_scale = scaler.fit_transform(X_train)
    X_test_scale = scaler.transform(X_test)

    X_train_single = X_train_scale.reshape(-1, 1, 6)
    X_test_single = X_test_scale.reshape(-1, 1, 6)

    model_lstm.fit(X_train_single, Y_train, epochs=n_epochs,
                   batch_size=16, verbose=0)

    y_pred = model_lstm.predict(X_test_single).reshape(-1, 1)

    mse = mean_squared_error(Y_test, y_pred)
    model_mse.append([weather_np[season][:, -1][initial_hours+i], mse])

    r2 = r2_score(Y_test, y_pred)
    model_r2.append([bee_np[season][:, -1][initial_hours+i], r2])

model_r2 = pd.DataFrame(model_r2)
model_mse = pd.DataFrame(model_mse)


In [ ]:
model_r2.mean()


In [ ]:
model_r2.mean()

In [ ]:
model_mse.mean()

In [ ]:
model_mse.mean()

In [ ]:
# single day test
i=11*24

X_train = X[:initial_hours+i]
Y_train = Y[:initial_hours+i]
X_test = X[initial_hours+i:initial_hours+i+24]
Y_test = Y[initial_hours+i:initial_hours+i+24]


scaler = StandardScaler()
X_train_single = scaler.fit_transform(X_train)
X_test_single = scaler.transform(X_test)

# X_train_single = X_train_scale.reshape(-1, 1, 6)
# X_test_single = X_test_scale.reshape(-1, 1, 6)

model_rfr = RandomForestRegressor()

model_rfr.fit(X_train_single, Y_train)

y_pred = model_rfr.predict(X_test_single)

In [ ]:
len(bee_np[2])

In [ ]:
3294/6

In [ ]:
season = 3

X = pd.DataFrame(weather_np[season][:, weather_columns])
Y = pd.DataFrame(bee_np[season][:, 1])

# X = pd.DataFrame(weather_np[:, weather_columns])
# Y = pd.DataFrame(bee_np, columns=['Y1', "Y2"])
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 0)

# X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 0)
# X_train = X[int(len(X)*0.8):]
# X_test = X[:len(X)-int(len(X)*0.8)]
# Y_train = Y[int(len(Y)*0.8):]
# Y_test = Y[:len(Y)-int(len(X)*0.8)]

X_train = np.asarray(X_train).astype('float32')
X_test = np.asarray(X_test).astype('float32')
Y_train = np.asarray(Y_train).astype('float32')
Y_test = np.asarray(Y_test).astype('float32')

scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)
# X_train_scale = X_train
# X_test_scale = X_test

# X_train_single = X_train_scale.reshape(-1, 1, 6)
# X_test_single = X_test_scale.reshape(-1,1,6)
X_train_single = X_train_scale.reshape(-1, 1, len(weather_columns))
X_test_single = X_test_scale.reshape(-1, 1, len(weather_columns))

# only predict 1 variable
model_gru = Sequential([
    GRU(16, activation='relu', return_sequences=True, input_shape=(1, len(weather_columns))),
    Dropout(0.15),
    GRU(16, activation='relu', return_sequences=True, input_shape=(1, len(weather_columns))),
    Dropout(0.1),
    Dense(1)
])

model_gru.compile(metrics=['mse'], loss='mse')

model_gru.summary()

n_epochs = 15
history = model_gru.fit(X_train_single, Y_train, epochs=n_epochs, batch_size=16, validation_data=(
    X_test_single, Y_test), verbose=1)

fig, ax = plt.subplots()

pd.DataFrame(history.history['loss'], index=list(
    range(n_epochs)), columns=['Training Loss']).plot(ax=ax)
pd.DataFrame(history.history['val_loss'], index=list(
    range(n_epochs)), columns=['Validation Loss']).plot(ax=ax, c='r')

print(history.history['val_loss'][-1])


In [ ]:
weather_data.columns[weather_columns]


In [ ]:
weather_columns


In [ ]:
X

In [ ]:
#k-fold cross validation
# neither precip nor barometric
weather_columns=[0, 2, 3, 8]

model_gru = Sequential([
    GRU(16, activation='relu', return_sequences=True,
        input_shape=(1, len(weather_columns))),
    Dropout(0.15),
    GRU(16, activation='relu', return_sequences=True,
        input_shape=(1, len(weather_columns))),
    Dropout(0.1),
    Dense(1)
])

model_gru.compile(metrics=['mse'], loss='mse')
X = pd.DataFrame(weather_np[season][:, weather_columns])
Y = pd.DataFrame(bee_np[season][:, 1])

model_mse=[]
model_r2=[]
kf = KFold(n_splits=tests)
for train_index, test_index in kf.split(X):
    # X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = i+1)
    # print(train_index)
    # print(test_index)

    X_train = X.iloc[train_index]
    X_test = X.iloc[test_index]
    Y_train = Y.iloc[train_index]
    Y_test = Y.iloc[test_index]

    X_train = np.asarray(X_train).astype('float32')
    X_test = np.asarray(X_test).astype('float32')
    Y_train = np.asarray(Y_train).astype('float32')
    Y_test = np.asarray(Y_test).astype('float32')

    scaler = StandardScaler()
    X_train_scale = scaler.fit_transform(X_train)
    X_test_scale = scaler.transform(X_test)

    X_train_scale = X_train_scale.reshape(-1, 1, len(weather_columns))
    X_test_scale = X_test_scale.reshape(-1, 1, len(weather_columns))

    model_gru.fit(X_train_scale, Y_train, epochs=3, batch_size=16, verbose=0)
    y_pred = model_gru.predict(X_test_scale).reshape(-1, 1)
    
    mse=mean_squared_error(Y_test, y_pred)
    model_mse.append(mse)

    r2=r2_score(Y_test, y_pred)
    model_r2.append(r2)

no_precip_baro_r2=model_r2
no_precip_baro_mse=model_mse

# gru_mse_1=pd.DataFrame(model_mse)
# gru_r2_1=pd.DataFrame(model_r2)
# print(gbr_mse, gbr_mse.mean(), gbr_r2, gbr_r2.mean(), sep='\n')


In [ ]:
pd.DataFrame(no_precip_baro_r2).mean()

In [ ]:
pd.DataFrame(no_precip_baro_mse).mean()

In [ ]:
#k-fold cross validation
# no barometric
weather_columns = [0, 2, 3, 6, 8]

model_gru = Sequential([
    GRU(16, activation='relu', return_sequences=True,
        input_shape=(1, len(weather_columns))),
    Dropout(0.15),
    GRU(16, activation='relu', return_sequences=True,
        input_shape=(1, len(weather_columns))),
    Dropout(0.1),
    Dense(1)
])

model_gru.compile(metrics=['mse'], loss='mse')
X = pd.DataFrame(weather_np[season][:, weather_columns])
Y = pd.DataFrame(bee_np[season][:, 1])

model_mse = []
model_r2 = []
kf = KFold(n_splits=tests)
for train_index, test_index in kf.split(X):
    # X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = i+1)
    # print(train_index)
    # print(test_index)

    X_train = X.iloc[train_index]
    X_test = X.iloc[test_index]
    Y_train = Y.iloc[train_index]
    Y_test = Y.iloc[test_index]

    X_train = np.asarray(X_train).astype('float32')
    X_test = np.asarray(X_test).astype('float32')
    Y_train = np.asarray(Y_train).astype('float32')
    Y_test = np.asarray(Y_test).astype('float32')

    scaler = StandardScaler()
    X_train_scale = scaler.fit_transform(X_train)
    X_test_scale = scaler.transform(X_test)

    X_train_scale = X_train_scale.reshape(-1, 1, len(weather_columns))
    X_test_scale = X_test_scale.reshape(-1, 1, len(weather_columns))

    model_gru.fit(X_train_scale, Y_train, epochs=3, batch_size=16, verbose=0)
    y_pred = model_gru.predict(X_test_scale).reshape(-1, 1)

    mse = mean_squared_error(Y_test, y_pred)
    model_mse.append(mse)

    r2 = r2_score(Y_test, y_pred)
    model_r2.append(r2)

no_baro_r2=model_r2
no_baro_mse=model_mse

# gru_mse_1=pd.DataFrame(model_mse)
# gru_r2_1=pd.DataFrame(model_r2)
# print(gbr_mse, gbr_mse.mean(), gbr_r2, gbr_r2.mean(), sep='\n')


In [ ]:
pd.DataFrame(no_baro_r2).mean()

In [ ]:
pd.DataFrame(no_baro_mse).mean()

In [ ]:
#k-fold cross validation
# no precip
weather_columns = [0, 2, 3, 5, 8]

model_gru = Sequential([
    GRU(16, activation='relu', return_sequences=True,
        input_shape=(1, len(weather_columns))),
    Dropout(0.15),
    GRU(16, activation='relu', return_sequences=True,
        input_shape=(1, len(weather_columns))),
    Dropout(0.1),
    Dense(1)
])

model_gru.compile(metrics=['mse'], loss='mse')
X = pd.DataFrame(weather_np[season][:, weather_columns])
Y = pd.DataFrame(bee_np[season][:, 1])

model_mse = []
model_r2 = []
kf = KFold(n_splits=tests)
for train_index, test_index in kf.split(X):
    # X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = i+1)
    # print(train_index)
    # print(test_index)

    X_train = X.iloc[train_index]
    X_test = X.iloc[test_index]
    Y_train = Y.iloc[train_index]
    Y_test = Y.iloc[test_index]

    X_train = np.asarray(X_train).astype('float32')
    X_test = np.asarray(X_test).astype('float32')
    Y_train = np.asarray(Y_train).astype('float32')
    Y_test = np.asarray(Y_test).astype('float32')

    scaler = StandardScaler()
    X_train_scale = scaler.fit_transform(X_train)
    X_test_scale = scaler.transform(X_test)

    X_train_scale = X_train_scale.reshape(-1, 1, len(weather_columns))
    X_test_scale = X_test_scale.reshape(-1, 1, len(weather_columns))

    model_gru.fit(X_train_scale, Y_train, epochs=3,
                  batch_size=16, verbose=0)
    y_pred = model_gru.predict(X_test_scale).reshape(-1, 1)

    mse = mean_squared_error(Y_test, y_pred)
    model_mse.append(mse)

    r2 = r2_score(Y_test, y_pred)
    model_r2.append(r2)

no_precip_r2 = model_r2
no_precip_mse = model_mse

# gru_mse_1=pd.DataFrame(model_mse)
# gru_r2_1=pd.DataFrame(model_r2)
# print(gbr_mse, gbr_mse.mean(), gbr_r2, gbr_r2.mean(), sep='\n')


In [ ]:
pd.DataFrame(no_precip_r2).mean()

In [ ]:
pd.DataFrame(no_precip_mse).mean()

In [ ]:
#k-fold cross validation
# all 6
weather_columns = [0, 2, 3, 5, 6, 8]

model_gru = Sequential([
    GRU(16, activation='relu', return_sequences=True,
        input_shape=(1, len(weather_columns))),
    Dropout(0.15),
    GRU(16, activation='relu', return_sequences=True,
        input_shape=(1, len(weather_columns))),
    Dropout(0.1),
    Dense(1)
])

model_gru.compile(metrics=['mse'], loss='mse')
X = pd.DataFrame(weather_np[season][:, weather_columns])
Y = pd.DataFrame(bee_np[season][:, 1])

model_mse = []
model_r2 = []
kf = KFold(n_splits=tests)
for train_index, test_index in kf.split(X):
    # X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = i+1)
    # print(train_index)
    # print(test_index)

    X_train = X.iloc[train_index]
    X_test = X.iloc[test_index]
    Y_train = Y.iloc[train_index]
    Y_test = Y.iloc[test_index]

    X_train = np.asarray(X_train).astype('float32')
    X_test = np.asarray(X_test).astype('float32')
    Y_train = np.asarray(Y_train).astype('float32')
    Y_test = np.asarray(Y_test).astype('float32')

    scaler = StandardScaler()
    X_train_scale = scaler.fit_transform(X_train)
    X_test_scale = scaler.transform(X_test)

    X_train_scale = X_train_scale.reshape(-1, 1, len(weather_columns))
    X_test_scale = X_test_scale.reshape(-1, 1, len(weather_columns))

    model_gru.fit(X_train_scale, Y_train, epochs=3,
                  batch_size=16, verbose=0)
    y_pred = model_gru.predict(X_test_scale).reshape(-1, 1)

    mse = mean_squared_error(Y_test, y_pred)
    model_mse.append(mse)

    r2 = r2_score(Y_test, y_pred)
    model_r2.append(r2)

all_6_r2 = model_r2
all_6_mse = model_mse

# gru_mse_1=pd.DataFrame(model_mse)
# gru_r2_1=pd.DataFrame(model_r2)
# print(gbr_mse, gbr_mse.mean(), gbr_r2, gbr_r2.mean(), sep='\n')


In [ ]:
print(pd.DataFrame(all_6_mse).mean(), pd.DataFrame(all_6_r2).mean())


In [ ]:
#k-fold cross validation
# all 6
weather_columns = [0, 2, 5, 6, 8]

model_gru = Sequential([
    GRU(16, activation='relu', return_sequences=True,
        input_shape=(1, len(weather_columns))),
    Dropout(0.15),
    GRU(16, activation='relu', return_sequences=True,
        input_shape=(1, len(weather_columns))),
    Dropout(0.1),
    Dense(1)
])

model_gru.compile(metrics=['mse'], loss='mse')
X = pd.DataFrame(weather_np[season][:, weather_columns])
Y = pd.DataFrame(bee_np[season][:, 1])

model_mse = []
model_r2 = []
kf = KFold(n_splits=tests)
for train_index, test_index in kf.split(X):
    # X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = i+1)
    # print(train_index)
    # print(test_index)

    X_train = X.iloc[train_index]
    X_test = X.iloc[test_index]
    Y_train = Y.iloc[train_index]
    Y_test = Y.iloc[test_index]

    X_train = np.asarray(X_train).astype('float32')
    X_test = np.asarray(X_test).astype('float32')
    Y_train = np.asarray(Y_train).astype('float32')
    Y_test = np.asarray(Y_test).astype('float32')

    scaler = StandardScaler()
    X_train_scale = scaler.fit_transform(X_train)
    X_test_scale = scaler.transform(X_test)

    X_train_scale = X_train_scale.reshape(-1, 1, len(weather_columns))
    X_test_scale = X_test_scale.reshape(-1, 1, len(weather_columns))

    model_gru.fit(X_train_scale, Y_train, epochs=3,
                  batch_size=16, verbose=0)
    y_pred = model_gru.predict(X_test_scale).reshape(-1, 1)

    mse = mean_squared_error(Y_test, y_pred)
    model_mse.append(mse)

    r2 = r2_score(Y_test, y_pred)
    model_r2.append(r2)

no_wind_r2 = model_r2
no_wind_mse = model_mse

# gru_mse_1=pd.DataFrame(model_mse)
# gru_r2_1=pd.DataFrame(model_r2)
# print(gbr_mse, gbr_mse.mean(), gbr_r2, gbr_r2.mean(), sep='\n')


In [ ]:
print(pd.DataFrame(no_wind_mse).mean(), pd.DataFrame(no_wind_r2).mean())

In [ ]:
#k-fold cross validation
# just 3
weather_columns = [0, 2, 8]

model_gru = Sequential([
    GRU(16, activation='relu', return_sequences=True,
        input_shape=(1, len(weather_columns))),
    Dropout(0.15),
    GRU(16, activation='relu', return_sequences=True,
        input_shape=(1, len(weather_columns))),
    Dropout(0.1),
    Dense(1)
])

model_gru.compile(metrics=['mse'], loss='mse')
X = pd.DataFrame(weather_np[season][:, weather_columns])
Y = pd.DataFrame(bee_np[season][:, 1])

model_mse = []
model_r2 = []
kf = KFold(n_splits=tests)
for train_index, test_index in kf.split(X):
    # X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = i+1)
    # print(train_index)
    # print(test_index)

    X_train = X.iloc[train_index]
    X_test = X.iloc[test_index]
    Y_train = Y.iloc[train_index]
    Y_test = Y.iloc[test_index]

    X_train = np.asarray(X_train).astype('float32')
    X_test = np.asarray(X_test).astype('float32')
    Y_train = np.asarray(Y_train).astype('float32')
    Y_test = np.asarray(Y_test).astype('float32')

    scaler = StandardScaler()
    X_train_scale = scaler.fit_transform(X_train)
    X_test_scale = scaler.transform(X_test)

    X_train_scale = X_train_scale.reshape(-1, 1, len(weather_columns))
    X_test_scale = X_test_scale.reshape(-1, 1, len(weather_columns))

    model_gru.fit(X_train_scale, Y_train, epochs=3,
                  batch_size=16, verbose=0)
    y_pred = model_gru.predict(X_test_scale).reshape(-1, 1)

    mse = mean_squared_error(Y_test, y_pred)
    model_mse.append(mse)

    r2 = r2_score(Y_test, y_pred)
    model_r2.append(r2)

just_3_r2 = model_r2
just_3_mse = model_mse

# gru_mse_1=pd.DataFrame(model_mse)
# gru_r2_1=pd.DataFrame(model_r2)
# print(gbr_mse, gbr_mse.mean(), gbr_r2, gbr_r2.mean(), sep='\n')


In [ ]:
print(pd.DataFrame(just_3_mse).mean(), pd.DataFrame(just_3_r2).mean())

In [ ]:
weather_data.columns[5]

In [ ]:
#k-fold cross validation
# just 3
weather_columns = [0, 2, 5, 8]

model_gru = Sequential([
    GRU(16, activation='relu', return_sequences=True,
        input_shape=(1, len(weather_columns))),
    Dropout(0.15),
    GRU(16, activation='relu', return_sequences=True,
        input_shape=(1, len(weather_columns))),
    Dropout(0.1),
    Dense(1)
])

model_gru.compile(metrics=['mse'], loss='mse')
X = pd.DataFrame(weather_np[season][:, weather_columns])
Y = pd.DataFrame(bee_np[season][:, 1])

model_mse = []
model_r2 = []
kf = KFold(n_splits=tests)
for train_index, test_index in kf.split(X):
    # X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = i+1)
    # print(train_index)
    # print(test_index)

    X_train = X.iloc[train_index]
    X_test = X.iloc[test_index]
    Y_train = Y.iloc[train_index]
    Y_test = Y.iloc[test_index]

    X_train = np.asarray(X_train).astype('float32')
    X_test = np.asarray(X_test).astype('float32')
    Y_train = np.asarray(Y_train).astype('float32')
    Y_test = np.asarray(Y_test).astype('float32')

    scaler = StandardScaler()
    X_train_scale = scaler.fit_transform(X_train)
    X_test_scale = scaler.transform(X_test)

    X_train_scale = X_train_scale.reshape(-1, 1, len(weather_columns))
    X_test_scale = X_test_scale.reshape(-1, 1, len(weather_columns))

    model_gru.fit(X_train_scale, Y_train, epochs=3,
                  batch_size=16, verbose=0)
    y_pred = model_gru.predict(X_test_scale).reshape(-1, 1)

    mse = mean_squared_error(Y_test, y_pred)
    model_mse.append(mse)

    r2 = r2_score(Y_test, y_pred)
    model_r2.append(r2)

no_wind_precip_r2 = model_r2
no_wind_precip_mse = model_mse

# gru_mse_1=pd.DataFrame(model_mse)
# gru_r2_1=pd.DataFrame(model_r2)
# print(gbr_mse, gbr_mse.mean(), gbr_r2, gbr_r2.mean(), sep='\n')


In [ ]:
print(pd.DataFrame(no_wind_precip_mse).mean(), pd.DataFrame(no_wind_precip_r2).mean())


In [ ]:
#k-fold cross validation
# just 3
weather_columns = [0, 2, 6, 8]

model_gru = Sequential([
    GRU(16, activation='relu', return_sequences=True,
        input_shape=(1, len(weather_columns))),
    Dropout(0.15),
    GRU(16, activation='relu', return_sequences=True,
        input_shape=(1, len(weather_columns))),
    Dropout(0.1),
    Dense(1)
])

model_gru.compile(metrics=['mse'], loss='mse')
X = pd.DataFrame(weather_np[season][:, weather_columns])
Y = pd.DataFrame(bee_np[season][:, 1])

model_mse = []
model_r2 = []
kf = KFold(n_splits=tests)
for train_index, test_index in kf.split(X):
    # X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = i+1)
    # print(train_index)
    # print(test_index)

    X_train = X.iloc[train_index]
    X_test = X.iloc[test_index]
    Y_train = Y.iloc[train_index]
    Y_test = Y.iloc[test_index]

    X_train = np.asarray(X_train).astype('float32')
    X_test = np.asarray(X_test).astype('float32')
    Y_train = np.asarray(Y_train).astype('float32')
    Y_test = np.asarray(Y_test).astype('float32')

    scaler = StandardScaler()
    X_train_scale = scaler.fit_transform(X_train)
    X_test_scale = scaler.transform(X_test)

    X_train_scale = X_train_scale.reshape(-1, 1, len(weather_columns))
    X_test_scale = X_test_scale.reshape(-1, 1, len(weather_columns))

    model_gru.fit(X_train_scale, Y_train, epochs=3,
                  batch_size=16, verbose=0)
    y_pred = model_gru.predict(X_test_scale).reshape(-1, 1)

    mse = mean_squared_error(Y_test, y_pred)
    model_mse.append(mse)

    r2 = r2_score(Y_test, y_pred)
    model_r2.append(r2)

no_wind_baro_r2 = model_r2
no_wind_baro_mse = model_mse

# gru_mse_1=pd.DataFrame(model_mse)
# gru_r2_1=pd.DataFrame(model_r2)
# print(gbr_mse, gbr_mse.mean(), gbr_r2, gbr_r2.mean(), sep='\n')


In [ ]:
print(pd.DataFrame(no_wind_baro_mse).mean(), pd.DataFrame(no_wind_baro_r2).mean())

In [ ]:
features_selected = [no_precip_baro_r2, no_precip_baro_mse, no_baro_r2, no_baro_mse, no_precip_r2, no_precip_mse, all_6_r2,
                     all_6_mse, no_wind_r2, no_wind_mse, just_3_r2, just_3_mse, no_wind_precip_r2, no_wind_precip_mse, no_wind_baro_r2, no_wind_baro_mse]


In [ ]:
weather_np[0][0]

In [ ]:
#k-fold cross validation
# just 3
strong = [0, 2, 3, 8]
# weather_columns = [0, 2, 3, 8]
# strong = [0, 1, 2, 5]

for factor in strong:
    model_gru = Sequential([
        GRU(16, activation='relu', return_sequences=True,
            input_shape=(1, 1)),
        Dropout(0.15),
        GRU(16, activation='relu', return_sequences=True,
            input_shape=(1, 1)),
        Dropout(0.1),
        Dense(1)
    ])

    model_gru.compile(metrics=['mse'], loss='mse')
    X = pd.DataFrame(weather_np[season][:, factor])
    Y = pd.DataFrame(bee_np[season][:, 1])

    model_mse = []
    model_r2 = []
    kf = KFold(n_splits=tests)
    for train_index, test_index in kf.split(X):
        # X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = i+1)
        # print(train_index)
        # print(test_index)

        X_train = X.iloc[train_index]
        X_test = X.iloc[test_index]
        Y_train = Y.iloc[train_index]
        Y_test = Y.iloc[test_index]

        X_train = np.asarray(X_train).astype('float32')
        X_test = np.asarray(X_test).astype('float32')
        Y_train = np.asarray(Y_train).astype('float32')
        Y_test = np.asarray(Y_test).astype('float32')

        scaler = StandardScaler()
        X_train_scale = scaler.fit_transform(X_train)
        X_test_scale = scaler.transform(X_test)

        X_train_scale = X_train_scale.reshape(-1, 1, 1)
        X_test_scale = X_test_scale.reshape(-1, 1, 1)

        model_gru.fit(X_train_scale, Y_train, epochs=3,
                    batch_size=16, verbose=0)
        y_pred = model_gru.predict(X_test_scale).reshape(-1, 1)

        mse = mean_squared_error(Y_test, y_pred)
        model_mse.append(mse)

        r2 = r2_score(Y_test, y_pred)
        model_r2.append(r2)

    features_selected.append(model_r2)
    features_selected.append(model_mse)

# gru_mse_1=pd.DataFrame(model_mse)
# gru_r2_1=pd.DataFrame(model_r2)
# print(gbr_mse, gbr_mse.mean(), gbr_r2, gbr_r2.mean(), sep='\n')


In [ ]:
for i in range(len(features_selected)-8, len(features_selected), 2):
    print(pd.DataFrame(features_selected[i]).mean(), pd.DataFrame(features_selected[i+1]).mean())

In [ ]:
fig
sns.boxplot(data=boxplot_df, color='yellow', medianprops={"color": "black"})
plt.ylabel('MSE')
# plt.xticks(rotation=90)
plt.show()

In [ ]:
ratio={}
for date in arr:
    ratio[date]=arr[date]['Active Bees']/arr[date]['Total Bees']

In [ ]:
line_df=pd.DataFrame()
line_df2=pd.DataFrame()
for i in range(len(gru_r2_1)):
    date = gru_r2_1[0][i]
    line_df[date]=pd.Series(gru_r2_1[1][i])
    line_df2[date]=pd.Series(rain_accum[date])
    # indices.append(date)
    # vals.append([gru_r2_1[1][i], rain_accum[date]])
line_df=line_df.transpose()
line_df2=line_df2.transpose()

line_df.columns=['R-Squared Score']
line_df2.columns=['Daily Accumulated Precipitation']

In [ ]:
line_df=pd.DataFrame()
line_df2=pd.DataFrame()
for date in rain_accum:
    if date>=pd.to_datetime('2023-01-26'):
        line_df[date]=pd.Series(ratio[date])
        line_df2[date]=pd.Series(rain_accum[date])
    # indices.append(date)
    # vals.append([gru_r2_1[1][i], rain_accum[date]])
line_df=line_df.transpose()
line_df2=line_df2.transpose()

line_df.columns=['Daily Percent of Total Bees Active']
line_df2.columns=['Daily Accumulated Precipitation (inches)']

In [ ]:
pearsonr(line_df.iloc[:, 0].tolist(), line_df2.iloc[:, 0].tolist())


In [ ]:

fig, ax = plt.subplots(figsize=(12, 8))
# line1=sns.lineplot(data=line_df, ax=ax)
line1 = ax.plot(line_df, label='Daily Percent of Total Bees Active')

ax2 = ax.twinx()
# df_hourly["Speed"].plot(ax=ax2, c="r")
# sns.lineplot(data=line_df2, ax=ax2)
# ax2.legend([line2], ['R-Squared Score'], loc='upper left')
line2 = ax2.plot(line_df2, color='red', label='Daily Accumulated Precipitation (inches)')
# line2=sns.lineplot(data=line_df2, ax=ax2, palette=['red'])
lines = line1+line2
labs = [l.get_label() for l in lines]
ax.legend(lines, labs, loc='upper left')
# fig.legend([line1, line2], ['R-Squared Score', 'Daily Accumulated Precipitation'])
# ax.set_ylim(bottom=0, top=1)
# ax.set_ylabel('Time')
ax.set_ylabel('Daily Percent of Total Bees Active')
ax2.set_ylabel('Daily Accumilated Preciptation')
# plt.xticks([i for i in range(int((len(bee_np[0])-initial_hours)/24))])

plt.title('Daily Bee Activity vs. Precipitation')
plt.xlabel('Time')
plt.savefig(os.path.join(directory, 'Figures',
            'r2_vs_rain.jpg'), dpi=600, bbox_inches='tight')
plt.show()


In [ ]:
lstm_r2_1


In [ ]:
gru_r2_1.mean()

In [ ]:
indexed = pd.DataFrame(lstm_r2_1[1])
indexed.index = lstm_r2_1[0]
indexed.columns=['LSTM']
indexed2 = pd.DataFrame(gru_r2_1[1])
indexed2.index = gru_r2_1[0]
indexed2.columns = ['GRU']

fig, ax = plt.subplots(figsize=(12, 6))
sns.lineplot(data=indexed, ax=ax)
sns.lineplot(data=indexed2, ax=ax, palette=['red'])
ax.set_ylim(bottom=0, top=1)
plt.xlabel('Time')
plt.ylabel('R-Squared Score')
# plt.xticks([i for i in range(int((len(bee_np[0])-initial_hours)/24))])
plt.title('Initial Performance (R-Squared Score)')
plt.savefig(os.path.join(directory, 'Figures',
            'r2_initial.jpg'), dpi=600, bbox_inches='tight')
plt.show()


In [ ]:
indexed = pd.DataFrame(lstm_mse_1[1])
indexed.index = lstm_mse_1[0]
indexed.columns=['LSTM']
indexed2 = pd.DataFrame(gru_mse_1[1])
indexed2.index = gru_mse_1[0]
indexed2.columns = ['GRU']

fig, ax = plt.subplots(figsize=(12, 6))
sns.lineplot(data=indexed, ax=ax)
sns.lineplot(data=indexed2, ax=ax, palette=['red'])
ax.set_ylim(bottom=0, top=0.05)
plt.xlabel('Time')
plt.ylabel('Mean Squared Error')
# plt.xticks([i for i in range(int((len(bee_np[0])-initial_hours)/24))])
plt.title('Initial Performance (Mean Squared Error)')
plt.savefig(os.path.join(directory, 'Figures',
            'mse_initial.jpg'), dpi=600, bbox_inches='tight')
plt.show()


# NEW RNN TESTING

In [ ]:
pd.DataFrame(weather_np[season][:, [i for i in range(len(weather_np[0][0])-1)]])

In [ ]:
X_train.shape


In [ ]:
pd.DataFrame(weather_np[season][:, weather_columns]).to_numpy().shape

In [ ]:
pd.DataFrame(weather_np[season]
             [:, [i for i in range(len(weather_np[0][0])-1)]]).to_numpy().shape

In [ ]:
X_train.shape

In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import make_scorer

season = 0

X = pd.DataFrame(weather_np[season][:, [i for i in range(len(weather_np[0][0])-1)]])
# X = pd.DataFrame(weather_np[season][:, weather_columns])
Y = pd.DataFrame(bee_np[season][:, 1])

# X_train = X[int(len(X)*0.8):]
# X_test = X[:len(X)-int(len(X)*0.8)]
# Y_train = Y[int(len(Y)*0.8):]
# Y_test = Y[:len(Y)-int(len(X)*0.8)]

X_train = np.asarray(X).astype('float32')
# X_test = np.asarray(X_test).astype('float32')
Y_train = np.asarray(Y).astype('float32')
# Y_test = np.asarray(Y_test).astype('float32')

scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train)
# X_test_scale = scaler.transform(X_test)

X_train = X_train_scale.reshape(-1, 9)
X_test = X_test_scale.reshape(-1, 9)

# model_gru.compile(metrics=['mse'], loss='mse')

# model_gru.fit(X_train_single, Y_train, epochs=n_epochs,
#                 batch_size=16, verbose=0)

# y_pred = model_gru.predict(X_test_single).reshape(-1, 1)

# mse = mean_squared_error(Y_test, y_pred)
# model_mse.append([weather_np[season][:, -1][initial_hours+i], mse])

# r2 = r2_score(Y_test, y_pred)
# model_r2.append([bee_np[season][:, -1][initial_hours+i], r2])

# model = Sequential([
#     LSTM(16, activation='relu', return_sequences=True, input_shape=(1, 9)),
#     Dropout(0.15),
#     LSTM(16, activation='relu', return_sequences=True, input_shape=(1, 9)),
#     Dropout(0.1),
#     Dense(1)
# ])

model=RandomForestRegressor()

mse_scorer=make_scorer(mean_squared_error, greater_is_better=False)

# Define the SequentialFeatureSelector object for forward selection
sfs = SequentialFeatureSelector(estimator=model, n_features_to_select=None, scoring=mse_scorer, direction='forward', cv=None)

# scores = cross_val_score(sfs, X_train, Y_train, cv=5, scoring='neg_mean_squared_error')

# Fit the SequentialFeatureSelector object to the training data
sfs.fit(X_train, Y_train)

# Get the selected feature indices
selected_features = sfs.get_support(indices=True)

# Print the selected feature indices
print("Selected feature indices:", selected_features)


In [ ]:
list(selected_features)

In [ ]:
for i in list(selected_features):
    print(weather_data.columns[i])

# NEW RNN TESTING

In [ ]:
from sklearn.ensemble import RandomForestRegressor

X = pd.DataFrame(weather_np[:, weather_columns])
Y = pd.DataFrame(bee_np)
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=0)

scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)

model_rfr = RandomForestRegressor()

model_rfr.fit(X_train_scale, Y_train)
y_pred = model_rfr.predict(X_test_scale)
mse = mean_squared_error(Y_test, y_pred)
print('mse: '+str(mse))
print('R2 test data: ', model_rfr.score(X_test_scale, Y_test))

pipe = Pipeline([('scaler', scaler), ('model', model_rfr)])
print('R2 cross-validated: ', cross_val_score(pipe, X, Y, cv=10),
      cross_val_score(pipe, X, Y, cv=10).mean())


In [ ]:
model_mse=[]
model_r2=[]
for i in range(tests):
    temp_X_train, temp_X_test, temp_Y_train, temp_Y_test = train_test_split(X,Y, test_size = 0.2, random_state = i+1)

    scaler = StandardScaler()
    temp_X_test_scale = scaler.fit_transform(temp_X_test)

    temp_X_test_single = temp_X_test_scale
    temp_y_pred = model_rfr.predict(temp_X_test_single)
    
    mse=mean_squared_error(temp_y_pred, temp_Y_test)
    model_mse.append(mse)

    r2=r2_score(temp_y_pred, temp_Y_test)
    model_r2.append(r2)

rfr_mse=pd.DataFrame(model_mse)
rfr_r2=pd.DataFrame(model_r2)
print(rfr_mse, rfr_mse.mean(), rfr_r2, rfr_r2.mean(), sep='\n')


In [ ]:
season = 0

X = pd.DataFrame(weather_np[season][:, weather_columns])
Y = pd.DataFrame(bee_np[season][:, 1])

# X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 0)
X_train = X[int(len(X)*0.8):]
X_test = X[:len(X)-int(len(X)*0.8)]
Y_train = Y[int(len(Y)*0.8):]
Y_test = Y[:len(Y)-int(len(X)*0.8)]

scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)

X_train_single = X_train_scale.reshape(-1, 1, 6)
X_test_single = X_test_scale.reshape(-1,1,6)

# only predict 1 variable
model_lstm = Sequential([
    LSTM(16, activation='relu', return_sequences=True, input_shape=(1, 6)),
    Dropout(0.15),
    LSTM(16, activation='relu', return_sequences=True, input_shape=(1, 6)),
    Dropout(0.1),
    Dense(1)
])

model_lstm.compile(metrics=['mse'], loss='mse')

model_lstm.summary()

n_epochs = 15
history = model_lstm.fit(X_train_single, Y_train, epochs=n_epochs, batch_size=16, validation_data=(
    X_test_single, Y_test), verbose=1)

fig, ax = plt.subplots()

pd.DataFrame(history.history['loss'], index=list(
    range(n_epochs)), columns=['Training Loss']).plot(ax=ax)
pd.DataFrame(history.history['val_loss'], index=list(
    range(n_epochs)), columns=['Validation Loss']).plot(ax=ax, c='r')

print(history.history['val_loss'][-1])


In [ ]:
#30 iterations
#15
model_mse = []
model_r2 = []

for i in range(0, len(bee_np[season])-initial_hours-24, 24):
    X_train = X[:initial_hours+i]
    Y_train = Y[:initial_hours+i]
    X_test = X[initial_hours+i:initial_hours+i+24]
    Y_test = Y[initial_hours+i:initial_hours+i+24]

    scaler = StandardScaler()
    X_train_scale = scaler.fit_transform(X_train)
    X_test_scale = scaler.transform(X_test)

    X_train_single = X_train_scale.reshape(-1, 1, 6)
    X_test_single = X_test_scale.reshape(-1, 1, 6)

    model_lstm.compile(metrics=['mse'], loss='mse')

    n_epochs = 15
    model_lstm.fit(X_train_single, Y_train, epochs=n_epochs,
                   batch_size=16, verbose=0)

    y_pred = model_lstm.predict(X_test_single).reshape(-1, 1)

    mse = mean_squared_error(y_pred, Y_test)
    model_mse.append(mse)

    r2 = r2_score(y_pred, Y_test)
    model_r2.append(r2)

model_mse=pd.DataFrame(model_mse)
model_r2=pd.DataFrame(model_r2)


In [ ]:
pd.DataFrame(model_mse).mean()

In [ ]:
pd.DataFrame(model_mse)

In [ ]:
lstm_mse=pd.DataFrame(model_mse)
lstm_r2=model_r2

In [ ]:
model_r2=pd.DataFrame(model_r2)

In [ ]:
g = sns.lineplot(data = model_r2, x=model_r2.index.tolist(), y=model_r2[0].tolist())
# g.set_yticks(range(11))
# g.set_yticklabels([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1])

In [ ]:
season = 2

X = pd.DataFrame(weather_np[season][:, weather_columns])
Y = pd.DataFrame(bee_np[season][:, 1])

# X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 0)
X_train = X[int(len(X)*0.8):]
X_test = X[:len(X)-int(len(X)*0.8)]
Y_train = Y[int(len(Y)*0.8):]
Y_test = Y[:len(Y)-int(len(X)*0.8)]

scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)

X_train_single = X_train_scale.reshape(-1, 1, 6)
X_test_single = X_test_scale.reshape(-1,1,6)

# only predict 1 variable
model_gru1 = Sequential([
    GRU(16, activation='relu', return_sequences=True, input_shape=(1, 6)),
    Dropout(0.15),
    GRU(16, activation='relu', return_sequences=True, input_shape=(1, 6)),
    Dropout(0.1),
    Dense(1)
])

model_gru1.compile(metrics=['mse'], loss='mse')

model_gru1.summary()

n_epochs = 15
history = model_lstm.fit(X_train_single, Y_train, epochs=n_epochs, batch_size=16, validation_data=(
    X_test_single, Y_test), verbose=1)

fig, ax = plt.subplots()

pd.DataFrame(history.history['loss'], index=list(
    range(n_epochs)), columns=['Training Loss']).plot(ax=ax)
pd.DataFrame(history.history['val_loss'], index=list(
    range(n_epochs)), columns=['Validation Loss']).plot(ax=ax, c='r')

print(history.history['val_loss'][-1])


In [ ]:
#30 iterations
#15
model_mse = []
model_r2 = []

# for i in range(0, len(bee_np)+1-initial_hours-24, 24*8):
for i in range(0, len(bee_np[season])-initial_hours-24, 24):
    X_train=X[:initial_hours+i]
    Y_train=Y[:initial_hours+i]
    X_test=X[initial_hours+i:initial_hours+i+24]
    Y_test=Y[initial_hours+i:initial_hours+i+24]

    scaler = StandardScaler()
    X_train_scale = scaler.fit_transform(X_train)
    X_test_scale = scaler.transform(X_test)

    X_train_single = X_train_scale.reshape(-1, 1, 6)
    X_test_single = X_test_scale.reshape(-1,1,6)

    model_gru1.compile(metrics=['mse'], loss='mse')

    n_epochs = 15
    model_gru1.fit(X_train_single, Y_train, epochs=n_epochs, batch_size=16, verbose=0)

    y_pred = model_gru1.predict(X_test_single).reshape(-1, 1)

    mse = mean_squared_error(y_pred, Y_test)
    model_mse.append(mse)

    r2 = r2_score(y_pred, Y_test)
    model_r2.append(r2)

    # fig, ax = plt.subplots()

    # pd.DataFrame(history.history['loss'], index=list(range(n_epochs)), columns=['Training Loss']).plot(ax=ax)
    # pd.DataFrame(history.history['val_loss'], index=list(range(n_epochs)), columns=['Validation Loss']).plot(ax=ax, c='r')

    # print(history.history['val_loss'][-1])

In [ ]:
gru3_r2


In [ ]:
gru3_mse=pd.DataFrame(model_mse)
gru3_r2=pd.DataFrame(model_r2)

sns.lineplot(data=gru3_r2, x=gru3_r2.index.tolist(),
             y=gru3_r2[0].tolist())


In [ ]:
gru2_mse=pd.DataFrame(model_mse)
gru2_r2=pd.DataFrame(model_r2)

In [ ]:
sns.lineplot(data=gru2_r2, x=gru2_r2.index.tolist(),
                 y=gru2_r2[0].tolist())


In [ ]:
gru_mse=pd.DataFrame(model_mse)
gru_r2=pd.DataFrame(model_r2)

In [ ]:
gru_mse


In [ ]:
gru_r2


In [ ]:
sns.lineplot(data=gru_mse, x=gru_mse.index.tolist(),
                 y=gru_mse[0].tolist())

In [ ]:
sns.lineplot(data=gru_r2, x=gru_r2.index.tolist(),
                 y=gru_r2[0].tolist())


In [ ]:
model_mse


In [ ]:
lstm_r2


In [ ]:
#30 iterations
#15
model_mse = []
model_r2 = []

for i in range(0, len(bee_np)+1-initial_hours-24, 24*7):
# for i in range(0, 3350-initial_hours-24, 24*7):
    X_train=X[:initial_hours+i]
    Y_train=Y[:initial_hours+i]
    X_test=X[initial_hours+i:initial_hours+i+24*7]
    Y_test=Y[initial_hours+i:initial_hours+i+24*7]

    scaler = StandardScaler()
    X_train_scale = scaler.fit_transform(X_train)
    X_test_scale = scaler.transform(X_test)

    X_train_single = X_train_scale.reshape(-1, 1, 6)
    X_test_single = X_test_scale.reshape(-1,1,6)

    model_lstm.compile(metrics=['mse'], loss='mse')

    n_epochs = 15
    model_lstm.fit(X_train_single, Y_train[['Bees Outside']], epochs=n_epochs, batch_size=16, verbose=0)

    y_pred = model_lstm.predict(X_test_single).reshape(-1, 1)

    mse = mean_squared_error(y_pred, Y_test['Bees Outside'])
    model_mse.append(mse)

    r2 = r2_score(y_pred, Y_test['Bees Outside'])
    model_r2.append(r2)

    # fig, ax = plt.subplots()

    # pd.DataFrame(history.history['loss'], index=list(range(n_epochs)), columns=['Training Loss']).plot(ax=ax)
    # pd.DataFrame(history.history['val_loss'], index=list(range(n_epochs)), columns=['Validation Loss']).plot(ax=ax, c='r')

    # print(history.history['val_loss'][-1])

In [ ]:
model_mse


In [ ]:
pd.DataFrame(model_mse).mean()

In [ ]:
model_r2


In [ ]:
pd.DataFrame(model_r2).mean()


In [ ]:
model_mse = []
model_r2 = []
for i in range(tests):
    temp_X_train, temp_X_test, temp_Y_train, temp_Y_test = train_test_split(
        X, Y, test_size=0.2, random_state=i+1)

    scaler = StandardScaler()
    temp_X_train_scale = scaler.fit_transform(temp_X_train)
    temp_X_test_scale = scaler.transform(temp_X_test)

    model_lstm.compile(metrics=['mse'], loss='mse')

    n_epochs = 15
    model_lstm.fit(X_train_single, Y_train[['Bees Outside']], epochs=n_epochs, batch_size=16, verbose=0)

    temp_X_test_single = temp_X_test_scale.reshape(-1, 1, 6)
    temp_y_pred = model_lstm.predict(temp_X_test_single).reshape(-1, 1)

    mse = mean_squared_error(temp_y_pred, temp_Y_test['Bees Outside'])
    model_mse.append(mse)

    r2 = r2_score(temp_y_pred, temp_Y_test['Bees Outside'])
    model_r2.append(r2)

lstm1_mse = pd.DataFrame(model_mse)
lstm1_r2 = pd.DataFrame(model_r2)
print(lstm1_mse, lstm1_mse.mean(), lstm1_r2, lstm1_r2.mean(), sep='\n')


In [ ]:
# only predict 1 variable
model_gru1 = Sequential([
    GRU(16, activation='relu', return_sequences=True, input_shape=(1, 6)),
    Dropout(0.15),
    GRU(16, activation='relu', return_sequences=True, input_shape=(1, 6)),
    Dropout(0.1),
    Dense(1)
])

model_gru1.compile(metrics=['mse'], loss='mse')

model_gru1.summary()

n_epochs = 30
history = model_gru1.fit(X_train_single, Y_train[['Bees Outside']], epochs=n_epochs, batch_size=16, validation_data=(
    X_test_single, Y_test[['Bees Outside']]), verbose=1)

fig, ax = plt.subplots()

pd.DataFrame(history.history['loss'], index=list(range(n_epochs)), columns=['Training Loss']).plot(ax=ax)
pd.DataFrame(history.history['val_loss'], index=list(range(n_epochs)), columns=['Validation Loss']).plot(ax=ax, c='r')

print(history.history['val_loss'][-1])

In [ ]:
model_mse = []
model_r2 = []
for i in range(tests):
    temp_X_train, temp_X_test, temp_Y_train, temp_Y_test = train_test_split(
        X, Y, test_size=0.2, random_state=i+1)

    scaler = StandardScaler()
    temp_X_test_scale = scaler.fit_transform(temp_X_test)

    temp_X_test_single = temp_X_test_scale.reshape(-1, 1, 6)
    temp_y_pred = model_gru1.predict(temp_X_test_single).reshape(-1, 1)

    mse = mean_squared_error(temp_y_pred, temp_Y_test['Bees Outside'])
    model_mse.append(mse)

    r2 = r2_score(temp_y_pred, temp_Y_test['Bees Outside'])
    model_r2.append(r2)

gru1_mse = pd.DataFrame(model_mse)
gru1_r2 = pd.DataFrame(model_r2)
print(gru1_mse, gru1_mse.mean(), gru1_r2, gru1_r2.mean(), sep='\n')


In [ ]:
boxplot_df = pd.concat([rfr_mse, gbr_mse, lstm1_mse, gru1_mse], axis=1)
boxplot_df


In [ ]:
boxplot_df.columns=['Random Forest', 'Gradient Boosting', 'LSTM', 'GRU']
boxplot_df


In [ ]:
sns.reset_orig()


In [ ]:
sns.set_theme(palette='muted', rc={"figure.figsize":(8, 12)})
sns.boxplot(data=boxplot_df, color='yellow', medianprops={"color": "black"})
plt.ylabel('MSE')
# plt.xticks(rotation=90)
plt.show()



In [ ]:
boxplot_df = pd.concat([rfr_r2, gbr_r2, lstm1_r2, gru1_r2], axis=1)
boxplot_df


In [ ]:
boxplot_df.columns=['Random Forest', 'Gradient Boosting', 'LSTM', 'GRU']
boxplot_df


In [ ]:
sns.reset_orig()


In [ ]:
sns.set_theme(palette='muted', rc={"figure.figsize":(8, 12)})
sns.boxplot(data=boxplot_df, color='yellow', medianprops={"color": "black"})
plt.ylabel('R2 Score')
# plt.xticks(rotation=90)
plt.show()



In [ ]:
print(pd.DataFrame([[float(rfr_mse.mean()), float(gbr_mse.mean()), float(lstm1_mse.mean()), float(gru1_mse.mean())], 
              [float(rfr_r2.mean()), float(gbr_r2.mean()), float(lstm1_r2.mean()), float(gru1_r2.mean())]],
    index=['Average MSE', 'Average R2 Score'], 
    columns=['Random Forest', 'Gradient Boosting', 'LSTM', 'GRU']).to_markdown())

|                  |   Random Forest |   Gradient Boosting |     LSTM |      GRU |
|:-----------------|----------------:|--------------------:|---------:|---------:|
| Average MSE      |       0.0127412 |           0.0200705 | 0.025342 | 0.023604 |
| Average R2 Score |       0.746717  |           0.56804   | 0.414296 | 0.40879  |

# END OF INITIAL TESTING #

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

X = pd.DataFrame(weather_np[:, weather_columns])
Y = pd.DataFrame(bee_np[:,1])
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 0)

scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)

model_gbr = GradientBoostingRegressor()

model_gbr.fit(X_train_scale, Y_train)
y_pred = model_gbr.predict(X_test_scale)
mse = mean_squared_error(Y_test, y_pred)
print('mse: '+str(mse))
print('R2 test data: ', model_gbr.score(X_test_scale, Y_test))

# pipe = Pipeline([('scaler', scaler), ('model', model)])
# print('R2 cross-validated: ', cross_val_score(pipe, X, Y, cv=10),
#       cross_val_score(pipe, X, Y, cv=10).mean())


In [ ]:
model_mse=[]
model_r2=[]
for i in range(tests):
    temp_X_train, temp_X_test, temp_Y_train, temp_Y_test = train_test_split(X,Y, test_size = 0.2, random_state = i+1)

    scaler = StandardScaler()
    temp_X_test_scale = scaler.fit_transform(temp_X_test)

    temp_X_test_single = temp_X_test_scale
    temp_y_pred = model_gbr.predict(temp_X_test_single)
    
    mse=mean_squared_error(temp_y_pred, temp_Y_test)
    model_mse.append(mse)

    r2=r2_score(temp_y_pred, temp_Y_test)
    model_r2.append(r2)

gbr_mse=pd.DataFrame(model_mse)
gbr_r2=pd.DataFrame(model_r2)
print(gbr_mse, gbr_mse.mean(), gbr_r2, gbr_r2.mean(), sep='\n')

In [ ]:
from sklearn.ensemble import RandomForestRegressor

X = pd.DataFrame(weather_np[:, weather_columns])
Y = pd.DataFrame(bee_np[:, 1])
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=0)

scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)

model_rfr = RandomForestRegressor()

model_rfr.fit(X_train_scale, Y_train)
y_pred = model_rfr.predict(X_test_scale)
mse = mean_squared_error(Y_test, y_pred)
print('mse: '+str(mse))
print('R2 test data: ', model_rfr.score(X_test_scale, Y_test))

pipe = Pipeline([('scaler', scaler), ('model', model_rfr)])
print('R2 cross-validated: ', cross_val_score(pipe, X, Y, cv=10),
      cross_val_score(pipe, X, Y, cv=10).mean())


In [ ]:
model_mse=[]
model_r2=[]
for i in range(tests):
    temp_X_train, temp_X_test, temp_Y_train, temp_Y_test = train_test_split(X,Y, test_size = 0.2, random_state = i+1)

    scaler = StandardScaler()
    temp_X_test_scale = scaler.fit_transform(temp_X_test)

    temp_X_test_single = temp_X_test_scale
    temp_y_pred = model_rfr.predict(temp_X_test_single)
    
    mse=mean_squared_error(temp_y_pred, temp_Y_test)
    model_mse.append(mse)

    r2=r2_score(temp_y_pred, temp_Y_test)
    model_r2.append(r2)

rfr_mse=pd.DataFrame(model_mse)
rfr_r2=pd.DataFrame(model_r2)
print(rfr_mse, rfr_mse.mean(), rfr_r2, rfr_r2.mean(), sep='\n')


In [ ]:
pipe = Pipeline([('scaler', scaler), ('model', model_rfr)])
cvs=cross_val_score(pipe, X, Y, cv=20)
print('R2 cross-validated: ', cvs, cvs.mean())

In [ ]:
X = pd.DataFrame(weather_np[:, weather_columns])
Y = pd.DataFrame(bee_np, columns=['Minutes Outside', 'Bees Outside'])
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 0)

scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)

X_train_single = X_train_scale.reshape(-1, 1, 6)
X_test_single = X_test_scale.reshape(-1,1,6)

In [ ]:
# only predict 1 variable
model_lstm = Sequential([
    LSTM(16, activation='relu', return_sequences=True, input_shape=(1, 6)),
    Dropout(0.15),
    LSTM(16, activation='relu', return_sequences=True, input_shape=(1, 6)),
    Dropout(0.1),
    Dense(1)
])

model_lstm.compile(metrics=['mse'], loss='mse')

model_lstm.summary()

n_epochs = 10
history = model_lstm.fit(X_train_single, Y_train[['Bees Outside']], epochs=n_epochs, batch_size=16, validation_data=(
    X_test_single, Y_test[['Bees Outside']]), verbose=1)

fig, ax = plt.subplots()

pd.DataFrame(history.history['loss'], index=list(range(n_epochs)), columns=['Training Loss']).plot(ax=ax)
pd.DataFrame(history.history['val_loss'], index=list(range(n_epochs)), columns=['Validation Loss']).plot(ax=ax, c='r')

print(history.history['val_loss'][-1])

In [ ]:
model_mse = []
model_r2 = []
for i in range(tests):
    temp_X_train, temp_X_test, temp_Y_train, temp_Y_test = train_test_split(
        X, Y, test_size=0.2, random_state=i+1)

    scaler = StandardScaler()
    temp_X_test_scale = scaler.fit_transform(temp_X_test)

    temp_X_test_single = temp_X_test_scale.reshape(-1, 1, 6)
    temp_y_pred = model_lstm.predict(temp_X_test_single).reshape(-1, 1)

    mse = mean_squared_error(temp_y_pred, temp_Y_test['Bees Outside'])
    model_mse.append(mse)

    r2 = r2_score(temp_y_pred, temp_Y_test['Bees Outside'])
    model_r2.append(r2)

lstm1_mse = pd.DataFrame(model_mse)
lstm1_r2 = pd.DataFrame(model_r2)
print(lstm1_mse, lstm1_mse.mean(), lstm1_r2, lstm1_r2.mean(), sep='\n')


In [ ]:
# only predict 1 variable
model_gru1 = Sequential([
    GRU(16, activation='relu', return_sequences=True, input_shape=(1, 6)),
    Dropout(0.15),
    GRU(16, activation='relu', return_sequences=True, input_shape=(1, 6)),
    Dropout(0.1),
    Dense(1)
])

model_gru1.compile(metrics=['mse'], loss='mse')

model_gru1.summary()

n_epochs = 30
history = model_gru1.fit(X_train_single, Y_train[['Bees Outside']], epochs=n_epochs, batch_size=16, validation_data=(
    X_test_single, Y_test[['Bees Outside']]), verbose=1)

fig, ax = plt.subplots()

pd.DataFrame(history.history['loss'], index=list(range(n_epochs)), columns=['Training Loss']).plot(ax=ax)
pd.DataFrame(history.history['val_loss'], index=list(range(n_epochs)), columns=['Validation Loss']).plot(ax=ax, c='r')

print(history.history['val_loss'][-1])

In [ ]:
model_mse = []
model_r2 = []
for i in range(tests):
    temp_X_train, temp_X_test, temp_Y_train, temp_Y_test = train_test_split(
        X, Y, test_size=0.2, random_state=i+1)

    scaler = StandardScaler()
    temp_X_test_scale = scaler.fit_transform(temp_X_test)

    temp_X_test_single = temp_X_test_scale.reshape(-1, 1, 6)
    temp_y_pred = model_gru1.predict(temp_X_test_single).reshape(-1, 1)

    mse = mean_squared_error(temp_y_pred, temp_Y_test['Bees Outside'])
    model_mse.append(mse)

    r2 = r2_score(temp_y_pred, temp_Y_test['Bees Outside'])
    model_r2.append(r2)

gru1_mse = pd.DataFrame(model_mse)
gru1_r2 = pd.DataFrame(model_r2)
print(gru1_mse, gru1_mse.mean(), gru1_r2, gru1_r2.mean(), sep='\n')


In [ ]:

scaler = StandardScaler()
X_scale=scaler.fit_transform(X)
X_scale.shape

In [ ]:
X

In [ ]:
weather_data

In [ ]:
weather_data.iloc[0].name

In [ ]:
np.array(x)

In [ ]:
x = []
x.append([])
x[-1].append(weather_data.iloc[4, weather_columns].values)
x[-1].append(weather_data.iloc[5, weather_columns].values)
x[-1].append(weather_data.iloc[6, weather_columns].values)
x.append([])
x[-1].append(weather_data.iloc[7, weather_columns].values)
x[-1].append(weather_data.iloc[8, weather_columns].values)
x[-1].append(weather_data.iloc[9, weather_columns].values)


In [ ]:
np.array(x).shape

In [ ]:
x[1][0:2]

In [ ]:
np.array(temp[0:0+time_steps]).shape

In [ ]:
time_steps=5
index=0
X2=[]

temp=[]

date=pd.to_datetime('2022-05-02')

for i in range(index, len(weather_data)):
    if weather_data.iloc[i].name==date-pd.Timedelta(hours=time_steps-1):
        print('name', weather_data.iloc[i].name)
        for j in range(i, i+time_steps+24):
            temp.append(weather_data.iloc[j, weather_columns].values)
        index=i
    
for hour in range(24):
    X2.append(np.array(temp[hour:hour+time_steps]))

# X2=np.array(X2)


In [ ]:

temp=[]

date=pd.to_datetime('2022-05-03')

for i in range(index, len(weather_data)):
    if weather_data.iloc[i].name==date-pd.Timedelta(hours=time_steps-1):
        print('name', weather_data.iloc[i].name)
        for j in range(i, i+time_steps+24):
            temp.append(weather_data.iloc[j, weather_columns].values)
        index=i
    
for hour in range(24):
    X2.append(np.array(temp[hour:hour+time_steps]))

In [ ]:
X2=np.array(X2)

In [ ]:
X2.shape

In [ ]:
X2[47]

In [ ]:
weather_data.iloc[48:48+24, weather_columns]

In [ ]:
len(weather_data)


In [ ]:
data_blocks[-1][0]


In [ ]:
for i in range(len(weather_data)):
    if weather_data.iloc[i].name==data_blocks[-1][0]-pd.Timedelta(hours=time_steps-1):
        print(i)
        break

In [ ]:
testso=np.array([1, 2, 3, 4, 5, 6]).reshape(1, 3, 2)

In [ ]:
type(testso[0][0][0])

In [ ]:
X_test[0].shape

In [ ]:
temp_list=[]
for i in X_test[0]:
    temp_list.append(i)

In [ ]:
temp_list=np.array(temp_list).reshape(1, 24, 6)

In [ ]:
model_gru.predict(temp_list)


In [ ]:
#model and validation cells
season = 3

time_steps=24
index=0
X2=[]
# Y2=Y.iloc[time_steps-1:]

for date in data_blocks[-1]:
    for i in range(index, len(weather_data)):
        if weather_data.iloc[i].name==date-pd.Timedelta(hours=time_steps-1):
            temp=[]
            # print('name', weather_data.iloc[i].name)
            for j in range(i, i+time_steps+24):
                temp.append(weather_data.iloc[j, weather_columns].values)
            for hour in range(24):
                X2.append(np.array(temp[hour:hour+time_steps]))
            index=i
            break
X2=np.array(X2)

X=X2
# X = pd.DataFrame(weather_np[season][:, weather_columns])
Y = pd.DataFrame(bee_np[season][:, 1])

# X = pd.DataFrame(weather_np[:, weather_columns])
# Y = pd.DataFrame(bee_np, columns=['Y1', "Y2"])
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 0)

# X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 0)
# X_train = X[int(len(X)*0.8):]
# X_test = X[:len(X)-int(len(X)*0.8)]
# Y_train = Y[int(len(Y)*0.8):]
# Y_test = Y[:len(Y)-int(len(X)*0.8)]

X_train = np.asarray(X_train).astype('float32')
X_test = np.asarray(X_test).astype('float32')
Y_train = np.asarray(Y_train).astype('float32')
Y_test = np.asarray(Y_test).astype('float32')

# scaler = StandardScaler()
# X_train_scale = scaler.fit_transform(X_train)
# X_test_scale = scaler.transform(X_test)
# X_train_scale = X_train
# X_test_scale = X_test

# X_train_single = X_train_scale.reshape(-1, 1, 6)
# X_test_single = X_test_scale.reshape(-1,1,6)
# X_train_single = X_train_scale.reshape(-1, time_steps, len(weather_columns))
# X_test_single = X_test_scale.reshape(-1, time_steps, len(weather_columns))

# only predict 1 variable
model_gru = Sequential([
    GRU(16, activation='relu', return_sequences=True, input_shape=(time_steps, len(weather_columns))),
    Dropout(0.15),
    GRU(16, activation='relu', input_shape=(time_steps, len(weather_columns))),
    Dropout(0.1),
    Dense(1)
])

model_gru.compile(metrics=['mse'], loss='mse')

model_gru.summary()

n_epochs = 20
history = model_gru.fit(X_train, Y_train, epochs=n_epochs, batch_size=16, validation_data=(
    X_test, Y_test), verbose=1)

fig, ax = plt.subplots()

pd.DataFrame(history.history['loss'], index=list(
    range(n_epochs)), columns=['Training Loss']).plot(ax=ax)
pd.DataFrame(history.history['val_loss'], index=list(
    range(n_epochs)), columns=['Validation Loss']).plot(ax=ax, c='r')

print(history.history['val_loss'][-1])


In [ ]:
#30 iterations
#15
model_mse = []
model_r2 = []

season=3

initial_hours = 7*24

# X = pd.DataFrame(weather_np[season][:, weather_columns])
# Y = pd.DataFrame(bee_np[season][:, 1])

# for i in range(0, len(bee_np)+1-initial_hours-24, 24*8):
for i in range(0, len(bee_np[season])-initial_hours, 24):
    X_train = X[:initial_hours+i]
    Y_train = Y[:initial_hours+i]
    X_test = X[initial_hours+i:initial_hours+i+24]
    Y_test = Y[initial_hours+i:initial_hours+i+24]

    X_train = np.asarray(X_train).astype('float32')
    X_test = np.asarray(X_test).astype('float32')
    Y_train = np.asarray(Y_train).astype('float32')
    Y_test = np.asarray(Y_test).astype('float32')

    # print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)
    # print(type(X_train), type(X_test), type(Y_train), type(Y_test))

    # scaler = StandardScaler()
    # X_train_scale = scaler.fit_transform(X_train)
    # X_test_scale = scaler.transform(X_test)
    
    # X_train_single = X_train_scale.reshape(-1, 1, len(weather_columns))
    # X_test_single = X_test_scale.reshape(-1, 1, len(weather_columns))

    model_gru.fit(X_train, Y_train, epochs=n_epochs,
                   batch_size=16, verbose=0)

    # y_pred = model_gru.predict(X_test).reshape(-1, 1)
    y_pred = model_gru.predict(X_test).reshape(-1, 1)

    mse = mean_squared_error(Y_test, y_pred)
    
    model_mse.append([weather_np[season][:, -1][initial_hours+i], mse])

    r2 = r2_score(Y_test, y_pred)
    model_r2.append([bee_np[season][:, -1][initial_hours+i], r2])

model_r2=pd.DataFrame(model_r2)
model_mse=pd.DataFrame(model_mse)


In [ ]:
gru_r2_1=model_r2

In [ ]:
model_r2.to_csv(os.path.join(directory, '1Test.csv'))

In [ ]:
remove_dates = [pd.to_datetime('2022-12-09') +
                pd.Timedelta(days=i) for i in range(20)]
remove_dates.append(pd.to_datetime('2022-07-20'))
remove_dates.append(pd.to_datetime('2022-09-26'))
remove_dates.append(pd.to_datetime('2022-11-18'))
remove_dates.append(pd.to_datetime('2022-09-26'))
remove_dates.append(pd.to_datetime('2023-02-06'))
remove_dates.append(pd.to_datetime('2023-02-22'))
remove_dates.append(pd.to_datetime('2023-02-25'))
remove_dates.append(pd.to_datetime('2023-03-01'))
# remove_dates.append(pd.to_datetime('2023-03-01'))
#over .4 inches
high_rain = [pd.to_datetime('2022-11-01 00:00:00'),
             pd.to_datetime('2022-11-08 00:00:00'),
             pd.to_datetime('2023-02-03 00:00:00'),
             pd.to_datetime('2023-02-04 00:00:00'),
             pd.to_datetime('2023-02-05 00:00:00'),
             pd.to_datetime('2023-02-11 00:00:00'),
             pd.to_datetime('2023-02-23 00:00:00'),
             pd.to_datetime('2023-02-24 00:00:00'),
             pd.to_datetime('2023-02-26 00:00:00')]

for date in high_rain:
    remove_dates.append(date)


In [ ]:
pd.to_datetime('2023-02-24 00:00:00') in remove_dates


In [ ]:
new_r2=[]
for i in range(len(gru_r2_1)):
    if not gru_r2_1.iloc[i, 0] in remove_dates:
        new_r2.append(gru_r2_1.iloc[i])

In [ ]:
pd.DataFrame(new_r2).to_csv(os.path.join(directory, '1Test.csv'))


In [ ]:
pd.DataFrame(new_r2).mean()

In [ ]:
# time_steps=48
# X = pd.DataFrame(weather_np[season][:, weather_columns])
# Y = pd.DataFrame(bee_np[season][:, 1])

# scaler = StandardScaler()
# X_scale=pd.DataFrame(scaler.fit_transform(X))
# X2 = pd.concat([X_scale.shift(n) for n in range(time_steps)], axis=1).dropna()
# X2 = X2.values.reshape(-1,time_steps, len(weather_columns))
# Y2 = Y.iloc[time_steps-1:]
# Y_shift=pd.concat([Y.shift(n) for n in range(time_steps)], axis=1).dropna()

# # X_train, X_test, Y_train, Y_test = train_test_split(X2,Y2, test_size = 0.2, random_state = 0)

# # scaler = StandardScaler()
# # X_train_scale = scaler.fit_transform(X_train)
# # X_test_scale = scaler.transform(X_test)

In [ ]:
Y_shift

In [ ]:
X2.shape

In [ ]:
weather_headers

In [ ]:
X_test.shape

In [ ]:
X_train[:, :, [0, 1, 2, 3, 5]].shape

In [ ]:
X_train.shape

In [ ]:
[i for i in range(len(weather_columns)) if i!=2]

In [ ]:
time_steps=1

X = pd.DataFrame(weather_np[season][:, weather_columns]).to_numpy().reshape(-1, time_steps, len(weather_columns))
Y = pd.DataFrame(bee_np[season][:, 1])

In [ ]:
dropped_feature


In [ ]:
X[:, :, [i for i in range(len(weather_columns)) if i!=dropped_feature]][0][0]

In [ ]:
X[0][0]

In [ ]:
X.shape

In [ ]:
Y.shape

In [ ]:
temp = model_r2.append([weather_np[season][:, -1][initial_hours], 1.4])


In [ ]:
temp

In [ ]:
X_perm[0]

In [ ]:
# permutation feature importance
feature_mse = []
feature_r2 = []

for dropped_feature in range(len(weather_columns)):
    model_gru = Sequential([
        GRU(16, activation='relu', return_sequences=True,
            input_shape=(1, len(weather_columns)-1)),
        Dropout(0.15),
        GRU(16, activation='relu', input_shape=(1, len(weather_columns)-1)),
        Dropout(0.1),
        Dense(1)
    ])

    model_gru.compile(metrics=['mse'], loss='mse')

    X_perm = X[:, :, [i for i in range(
        len(weather_columns)) if i != dropped_feature]]
    model_r2 = []
    model_mse = []

    model_r2.append(X_perm[0])
    model_mse.append(X_perm[0])

    feature_mse.append(model_mse)
    feature_r2.append(model_r2)


In [ ]:
X[0]


In [ ]:
feature_mse


In [ ]:
model_r2 = []
model_mse = []

X = pd.DataFrame(weather_np[season][:, weather_columns])
Y = pd.DataFrame(bee_np[season][:, 1])

model_gru = Sequential([
    GRU(16, activation='relu', return_sequences=True, input_shape=(1, len(weather_columns))),
    Dropout(0.15),
    GRU(16, activation='relu', input_shape=(1, len(weather_columns))),
    Dropout(0.1),
    Dense(1)
])

model_gru.compile(metrics=['mse'], loss='mse')

model_gru.summary()

for i in range(0, len(bee_np[season])-initial_hours, 24*8):
    X_train = X[:initial_hours+i]
    Y_train = Y[:initial_hours+i]
    X_test = X[initial_hours+i:initial_hours+i+24]
    Y_test = Y[initial_hours+i:initial_hours+i+24]

    X_train = np.asarray(X_train).astype('float32').reshape(-1, 1, 6)
    X_test = np.asarray(X_test).astype('float32').reshape(-1, 1, 6)
    Y_train = np.asarray(Y_train).astype('float32')
    Y_test = np.asarray(Y_test).astype('float32')

    # print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)
    # print(type(X_train), type(X_test), type(Y_train), type(Y_test))

    # scaler = StandardScaler()
    # X_train_scale = scaler.fit_transform(X_train)
    # X_test_scale = scaler.transform(X_test)

    # X_train_single = X_train_scale.reshape(-1, 1, len(weather_columns))
    # X_test_single = X_test_scale.reshape(-1, 1, len(weather_columns))

    model_gru.fit(X_train, Y_train, epochs=n_epochs,
                    batch_size=16, verbose=0)

    # y_pred = model_gru.predict(X_test).reshape(-1, 1)
    y_pred = model_gru.predict(X_test).reshape(-1, 1)

    mse = mean_squared_error(Y_test, y_pred)

    model_mse.append([weather_np[season][:, -1][initial_hours+i], mse])

    r2 = r2_score(Y_test, y_pred)
    model_r2.append([bee_np[season][:, -1][initial_hours+i], r2])

feature_mse.append(model_mse)
feature_r2.append(model_r2)


In [ ]:
import skopt

In [ ]:
# permutation feature importance
feature_mse=[]
feature_r2=[]

for dropped_feature in range(len(weather_columns)):
    model_gru = Sequential([
        GRU(16, activation='relu', return_sequences=True, input_shape=(1, len(weather_columns)-1)),
        Dropout(0.15),
        GRU(16, activation='relu', input_shape=(1, len(weather_columns)-1)),
        Dropout(0.1),
        Dense(1)
    ])

    model_gru.compile(metrics=['mse'], loss='mse')

    X_perm = X[:, :, [i for i in range(len(weather_columns)) if i!=dropped_feature]]
    model_r2=[]
    model_mse=[]

# adjust
    for i in range(0, len(bee_np[season])-initial_hours, 24*8):
        X_train = X_perm[:initial_hours+i]
        Y_train = Y[:initial_hours+i]
        X_test = X_perm[initial_hours+i:initial_hours+i+24]
        Y_test = Y[initial_hours+i:initial_hours+i+24]

        X_train = np.asarray(X_train).astype('float32')
        X_test = np.asarray(X_test).astype('float32')
        Y_train = np.asarray(Y_train).astype('float32')
        Y_test = np.asarray(Y_test).astype('float32')
        

        # print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)
        # print(type(X_train), type(X_test), type(Y_train), type(Y_test))

        # scaler = StandardScaler()
        # X_train_scale = scaler.fit_transform(X_train)
        # X_test_scale = scaler.transform(X_test)
        
        # X_train_single = X_train_scale.reshape(-1, 1, len(weather_columns))
        # X_test_single = X_test_scale.reshape(-1, 1, len(weather_columns))

        model_gru.fit(X_train, Y_train, epochs=n_epochs,
                    batch_size=16, verbose=0)

        # y_pred = model_gru.predict(X_test).reshape(-1, 1)
        y_pred = model_gru.predict(X_test).reshape(-1, 1)

        mse = mean_squared_error(Y_test, y_pred)
        
        model_mse.append([weather_np[season][:, -1][initial_hours+i], mse])

        r2 = r2_score(Y_test, y_pred)
        model_r2.append([bee_np[season][:, -1][initial_hours+i], r2])

    feature_mse.append(model_mse)
    feature_r2.append(model_r2)


In [ ]:
len(feature_r2[2])

In [ ]:
feature_r2[1]

In [ ]:
pd.DataFrame(feature_r2[0])


In [ ]:
pd.DataFrame(feature_r2[3]).iloc[:, 1]

In [ ]:
weather_data.columns[weather_columns]

In [ ]:
for i in range(len(feature_r2)):
    print(pd.DataFrame(feature_r2[i]).drop(index=[14]).iloc[:, 1].mean())
    # print(pd.DataFrame(i[:, 1]).mean())


In [ ]:
for i in range(len(feature_r2)):
    print(pd.DataFrame(feature_r2[i]).drop(index=[14]).iloc[:, 1].mean())
    # print(pd.DataFrame(i[:, 1]).mean())

In [ ]:

    n_epochs = 15
    history = model_gru.fit(X_train_single, Y_train, epochs=n_epochs, batch_size=16, validation_data=(
        X_test_single, Y_test), verbose=1)


    # Calculate feature importances
    feature_importances = np.zeros(X_val.shape[2])
    for i in range(X_val.shape[2]):
        X_val_permuted = X_val.copy()
        np.random.shuffle(X_val_permuted[:, :, i])
        mse_permuted = mean_squared_error(Y_val, model.predict(X_val_permuted))
        feature_importances[i] = baseline_mse - mse_permuted

    # Rank features by importance and select subset
    ranked_features = np.argsort(feature_importances)[::-1]
    selected_features = []
    for i in ranked_features:
        selected_features.append(i)
        X_train_subset = X_train[:, :, selected_features]
        X_val_subset = X_val[:, :, selected_features]
        model.fit(X_train_subset, Y_train, epochs=25, batch_size=16, verbose=1)
        mse_subset = mean_squared_error(Y_val, model.predict(X_val_subset))
        if mse_subset > baseline_mse:
            break
        baseline_mse = mse_subset

    print("Selected features:", selected_features)

In [ ]:
from keras.layers import Add
from keras.models import Model

In [ ]:
combined = Add()([model_lstm.output, model_gru.output])
combined = Dense(1)(combined)

combined_model = Model([model_lstm.input, model_gru.input], combined)

combined_model.compile(metrics=['mse'], loss='mse')

combined_model.summary()

n_epochs = 20
history = combined_model.fit([X_train, X_train], Y_train[['Minutes Outside']], epochs=n_epochs, batch_size=16, validation_data = ([X_test, X_test], Y_test[['Minutes Outside']]), verbose=0)

In [ ]:
combined_model.compile()